In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install requests SPARQLWrapper Flask
!pip install transformers torch
!pip install nltk transformers
!pip install transformers torch nltk
!pip install certifi --upgrade
!pip install requests SPARQLWrapper Flask urllib3
!pip install requests SPARQLWrapper Flask urllib3 pandas
!pip install -r requirements.txt
!pip install spacy
!pip install simplemma stanza
!pip install nltk
!pip install transformers[torch] tokenizers==0.13.3
!pip install transformers torch nltk
!python -m spacy download it_core_news_sm
!python -m spacy download it_core_news_sm
!python -m spacy download it_core_news_lg  # Per italiano
!python -m spacy download en_core_web_lg   # Per inglese
!git clone https://github.com/tuo-username/dbpedia-spotlight-wrapper.git
!git clone https://github.com/tuo-username/dbpedia-spotlight-wrapper.git

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
  Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
INFO: pip is looking at multiple versions of transformers[torch] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
  Using cached transformers-4.48.2-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.48.1-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.48.0-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
INFO: pip is still looking at multiple versions of transformers[torch] to determine which version is compatible with other requirements. This c

In [ ]:
# Verifica se un URI esiste
uri = "http://dbpedia.org/resource/Rome"
exists = wrapper._check_uri_exists(uri)
print(f"L'URI {uri} {'esiste' if exists else 'non esiste'}")

In [ ]:
# Ottieni l'URI aggiornato per una label
label = "Roma"
old_uri = "http://it.dbpedia.org/resource/Roma"
updated_uri = wrapper._get_updated_uri(label, old_uri)
print(f"URI aggiornato: {updated_uri}")

In [ ]:
!pip install spacy
!python -m spacy download it_core_news_lg  # Per italiano
!python -m spacy download en_core_web_lg   # Per inglese
!pip install stanza
!pip install simplemma
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os

# Importazione di spaCy
import spacy

# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy

# Modifica alla funzione export_entities_to_txt per mostrare tutte le entità spaCy
def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, original_text=None):
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Mostra tutte le entità riconosciute da spaCy, non solo quelle uniche.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy da utilizzare (opzionale)
        original_text: Testo originale da analizzare con spaCy (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]

    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in wikidata_entities:
        lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")

    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback per compatibilità con la versione precedente
        spacy_only_entities = [e for e in entities if e.get('source') == 'spacy']

        if not spacy_only_entities:
            # Se non ci sono entità spaCy uniche, segnala che è necessario passare il riconoscitore
            if spacy_recognizer is None or original_text is None:
                lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
                lines.append("Modificare la chiamata a export_entities_to_txt includendo i parametri spacy_recognizer e original_text.")
            else:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
        else:
            # Mostra le entità spaCy uniche
            for entity in spacy_only_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")

    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities



# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }


# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

if __name__ == "__main__":
    # Crea un'istanza del linker di entità migliorato
    linker = EntityLinkerWithSpacy(language="it")

    # Testo di esempio esteso - usa lo stesso testo che stavi analizzando prima
    text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
           " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
           " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
           " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
           " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
           " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
           " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
           " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
           " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
           " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

    print(f"Analisi del testo:\n{text}\n")

    # Trova entità
    entities, stats = linker.find_entities(text)

    print(f"\nTrovate {stats['total_entities']} entità:")
    for entity in entities:
        source = entity.get('source', 'sconosciuta')
        uri = entity.get('dbpedia_uri', 'N/A')
        types = ', '.join(entity.get('types', ['Sconosciuto']))

        print(f"- '{entity['text']}' → {uri}")
        print(f"  Tipo: {types}")
        if 'wikidata_id' in entity:
            print(f"  Wikidata: {entity.get('wikidata_url', 'N/A')}")
        print(f"  Fonte: {source}")
        print("")

    # Verifica URI
    uri_results = linker.verify_uris_for_text(text)

    print("\nVerifica URI:")
    for word, info in uri_results['results'].items():
        print(f"- La parola '{word}' → URI: {info['uri']} esiste")

    # Mostra entità spaCy senza URI
    if 'spacy_results' in uri_results and uri_results['spacy_results']:
        print("\nEntità riconosciute solo da spaCy (senza URI):")
        for word, info in uri_results['spacy_results'].items():
            print(f"- '{word}' → Etichetta: {info.get('label', '')}, Tipi: {', '.join(info.get('types', []))}")

    # Mostra solo le entità con URI DBpedia
    print("\nEntità con URI DBpedia:")
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    for entity in entities_with_dbpedia:
        print(f"- '{entity['text']}' → {entity['dbpedia_uri']}")

    # Assicurati che questa riga venga eseguita per generare il file
    output_file_path = export_entities_to_txt(
    entities,
    stats,
    "entita.txt",
    spacy_recognizer=linker.wikidata.spacy_recognizer,  # Passa il riconoscitore spaCy
    original_text=text  # Passa il testo originale
    )

    print(f"\nFile di output salvato in: {output_file_path}")

    print("\nStatistiche:")
    for key, value in stats.items():
        print(f"- {key}: {value}")

    # Verifica aggiuntiva per il file
    import os
    if os.path.exists("entita.txt"):
        file_size = os.path.getsize("entita.txt")
        print(f"\nVerifica file: 'entita_italia.txt' esiste e ha dimensione {file_size} bytes")
    else:
        print("\nATTENZIONE: Il file 'entita_italia.txt' non è stato creato!")
        # Prova a identificare il problema
        try:
            with open("test_write_permission.txt", "w") as f:
                f.write("Test")
            print("La directory è scrivibile, il problema è altrove")
            os.remove("test_write_permission.txt")
        except Exception as e:
            print(f"Errore nell'accesso alla directory: {e}")
            print("Prova a specificare un percorso completo per il file di output")
            # Tenta con percorso nella home directory dell'utente
            home_dir = os.path.expanduser("~")
            output_path = os.path.join(home_dir, "entita.txt")
            try:
                export_entities_to_txt(entities, stats, output_path)
                print(f"File salvato con successo in: {output_path}")
            except Exception as e2:
                print(f"Errore anche con percorso alternativo: {e2}")

Modello spaCy it_core_news_lg caricato con successo.
Analisi del testo:
L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico. Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento. Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio. Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export. Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa. Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo. La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo. Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia. Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponent

In [ ]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os

# Importazione di spaCy
import spacy

# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy

# Modifica alla funzione export_entities_to_txt per includere le entità del riconoscitore italiano
def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, italian_recognizer=None, original_text=None):
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Include anche le entità riconosciute da spaCy e dal riconoscitore italiano.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy (opzionale)
        italian_recognizer: Riconoscitore italiano (opzionale)
        original_text: Testo originale da analizzare (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]
    spacy_entities = [e for e in entities if e.get('source') == 'spacy']
    italian_entities = [e for e in entities if e.get('source') in ['italian_nlp', 'stanza', 'stanza_pos', 'geo_rule']]

    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in wikidata_entities:
        lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")

    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_all_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_all_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback alle entità spaCy già filtrate
        if not spacy_entities:
            lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
        else:
            for entity in spacy_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore italiano
    lines.append("ITALIAN NLP ENTITIES:")
    lines.append("--------------------")

    # Se abbiamo un riconoscitore italiano e un testo originale, ottieni tutte le entità
    if italian_recognizer and original_text:
        try:
            italian_all_entities = italian_recognizer.recognize_entities(original_text)

            if not italian_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore italiano nel testo.")
            else:
                for entity in italian_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore italiano: {e}")
    else:
        # Fallback alle entità italiane già filtrate
        if not italian_entities:
            lines.append("Per mostrare tutte le entità del riconoscitore italiano, è necessario passare il riconoscitore e il testo originale.")
        else:
            for entity in italian_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")

    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities


class ItalianNLPRecognizer:
    """
    Riconoscitore di entità ottimizzato per l'italiano in ambiente Colab.
    Usa una combinazione di Stanza (versione Python di Stanford CoreNLP)
    e altre librerie per l'italiano.
    """

    def __init__(self, use_gpu: bool = False):
        """
        Inizializza il riconoscitore per l'italiano.

        Args:
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        self.use_gpu = use_gpu
        self.is_available = False
        self.stanza_nlp = None
        self.initialized = False

        # Inizializza le risorse
        self._initialize_resources()

    def _initialize_resources(self):
        """Inizializza Stanza e altre risorse per l'italiano."""
        try:
            import stanza
            import simplemma
            import nltk
            from nltk.tokenize import word_tokenize

            # Scarica i modelli di Stanza per l'italiano
            stanza.download('it')

            # Inizializza il pipeline Stanza per l'italiano
            self.stanza_nlp = stanza.Pipeline(
                lang='it',
                processors='tokenize,mwt,pos,lemma,ner',
                use_gpu=self.use_gpu
            )

            # Scarica le risorse NLTK necessarie
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('averaged_perceptron_tagger')

            self.is_available = True
            self.initialized = True
            print("Riconoscitore NLP italiano inizializzato correttamente con Stanza.")

        except Exception as e:
            print(f"Errore nell'inizializzazione del riconoscitore italiano: {e}")
            print("Assicurati di aver eseguito le installazioni necessarie in Colab.")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo italiano utilizzando Stanza.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.stanza_nlp:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con Stanza
            doc = self.stanza_nlp(text)

            # Estrai le entità
            for sent in doc.sentences:
                for ent in sent.ents:
                    entity = {
                        'text': ent.text,
                        'label': ent.type,
                        'start_char': -1,  # Stanza non fornisce direttamente le posizioni dei caratteri
                        'end_char': -1,
                        'source': 'stanza'
                    }
                    entities.append(entity)

                # Aggiungi anche i nomi propri (POS tag == PROPN)
                for token in sent.tokens:
                    for word in token.words:
                        if word.upos == 'PROPN' and not any(e['text'] == word.text for e in entities):
                            entity = {
                                'text': word.text,
                                'label': 'PROPN',
                                'start_char': -1,
                                'end_char': -1,
                                'source': 'stanza_pos'
                            }
                            entities.append(entity)

            # Cerca anche entità geografiche italiane specifiche usando regole
            entities.extend(self._extract_italian_geo_entities(text))

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Stanza: {e}")

        return entities

    def _extract_italian_geo_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Estrae entità geografiche italiane usando regole.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità geografiche
        """
        import re
        entities = []

        # Regioni italiane
        regions = [
            "Abruzzo", "Basilicata", "Calabria", "Campania", "Emilia-Romagna",
            "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardia", "Marche",
            "Molise", "Piemonte", "Puglia", "Sardegna", "Sicilia", "Toscana",
            "Trentino-Alto Adige", "Umbria", "Valle d'Aosta", "Veneto"
        ]

        # Pattern per laghi, montagne, fiumi italiani
        geo_patterns = [
            r'\b(?:Lago|Monte|Fiume|Isola|Golfo|Capo|Valle|Parco Nazionale|Mar) [A-Z][a-zàèéìòù]+\b',
            r'\b(?:Alpi|Appennini|Dolomiti|Maremma|Laguna|Costa|Riviera) [A-Z][a-zàèéìòù]*\b'
        ]

        # Cerca regioni
        for region in regions:
            if re.search(r'\b' + re.escape(region) + r'\b', text):
                entity = {
                    'text': region,
                    'label': 'GPE',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        # Cerca pattern geografici
        for pattern in geo_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                entity = {
                    'text': match.group(0),
                    'label': 'LOC',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        return entities





# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }



# Estendi la classe EntityLinkerWithSpacy per incorporare il riconoscitore italiano
class EntityLinkerItalian(EntityLinkerWithSpacy):
    """
    Versione estesa di EntityLinker ottimizzata per l'italiano in ambiente Colab.
    """

    def __init__(self, language: str = "it", use_gpu: bool = False):
        """
        Inizializza il linker di entità con supporto NLP per l'italiano.

        Args:
            language: Lingua preferita per i risultati (deve essere "it")
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        # Controlla che la lingua sia italiana
        if language != "it":
            print("Attenzione: questo riconoscitore è ottimizzato per l'italiano. Impostiamo language='it'.")
            language = "it"

        # Inizializza la classe base (con spaCy)
        super().__init__(language)

        # Aggiungi il riconoscitore italiano
        self.italian_recognizer = ItalianNLPRecognizer(use_gpu=use_gpu)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando Wikidata, spaCy e il riconoscitore italiano.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Prima ottieni i risultati dalla classe base (Wikidata + spaCy)
        enriched_entities, stats = super().find_entities(text)
        existing_texts = {e.get('text', '').lower() for e in enriched_entities if 'text' in e}

        # Aggiungi le entità del riconoscitore italiano se disponibile
        italian_only_entities = self._get_italian_only_entities(text, existing_texts)
        enriched_entities.extend(italian_only_entities)

        # Aggiorna le statistiche
        stats['italian_only_entities'] = len(italian_only_entities)
        stats['total_entities'] = len(enriched_entities)

        return enriched_entities, stats

    def _get_italian_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore italiano e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo dal riconoscitore italiano
        """
        # Verifica che il riconoscitore italiano sia disponibile
        if not self.italian_recognizer.is_available:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore italiano
        italian_entities = self.italian_recognizer.recognize_entities(text)
        print(f"Riconoscitore italiano ha trovato {len(italian_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        italian_only = []
        for entity in italian_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore italiano: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                italian_type = self._map_italian_label_to_type(entity['label'])

                italian_entity = {
                    'text': entity_text,
                    'types': [italian_type],
                    'source': 'italian_nlp',
                    'label': entity['label']
                }

                italian_only.append(italian_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(italian_only)} sono uniche (non trovate da altre fonti)")
        return italian_only

    def _map_italian_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore italiano a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        mapping = {
            'PERSON': 'Person',
            'PER': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'MISC': 'Miscellaneous',
            'PROPN': 'ProperNoun',
            'WORK_OF_ART': 'Artwork',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estende il metodo della classe base per includere i risultati del riconoscitore italiano.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        # Ottieni i risultati di base
        uri_results = super().verify_uris_for_text(text)

        # Aggiungi una sezione per le entità del riconoscitore italiano senza URI
        italian_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'italian_nlp' and entity['text'] not in uri_results['results']:
                italian_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        # Aggiungi la sezione del riconoscitore italiano ai risultati
        uri_results['italian_results'] = italian_results

        return uri_results




# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

if __name__ == "__main__":
    # Crea un'istanza del linker di entità migliorato
#    linker = EntityLinkerWithSpacy(language="it")
    linker = EntityLinkerItalian(language="it", use_gpu=False)


    # Testo di esempio esteso - usa lo stesso testo che stavi analizzando prima
    text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
           " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
           " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
           " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
           " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
           " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
           " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
           " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
           " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
           " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

    print(f"Analisi del testo:\n{text}\n")

    # Trova entità
    entities, stats = linker.find_entities(text)

    print(f"\nTrovate {stats['total_entities']} entità:")
    for entity in entities:
        source = entity.get('source', 'sconosciuta')
        uri = entity.get('dbpedia_uri', 'N/A')
        types = ', '.join(entity.get('types', ['Sconosciuto']))

        print(f"- '{entity['text']}' → {uri}")
        print(f"  Tipo: {types}")
        if 'wikidata_id' in entity:
            print(f"  Wikidata: {entity.get('wikidata_url', 'N/A')}")
        print(f"  Fonte: {source}")
        print("")

    # Verifica URI
    uri_results = linker.verify_uris_for_text(text)

    print("\nVerifica URI:")
    for word, info in uri_results['results'].items():
        print(f"- La parola '{word}' → URI: {info['uri']} esiste")

    # Mostra entità spaCy senza URI
    if 'spacy_results' in uri_results and uri_results['spacy_results']:
        print("\nEntità riconosciute solo da spaCy (senza URI):")
        for word, info in uri_results['spacy_results'].items():
            print(f"- '{word}' → Etichetta: {info.get('label', '')}, Tipi: {', '.join(info.get('types', []))}")

    # Mostra solo le entità con URI DBpedia
    print("\nEntità con URI DBpedia:")
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    for entity in entities_with_dbpedia:
        print(f"- '{entity['text']}' → {entity['dbpedia_uri']}")

    # Esporta le entità
    output_file_path = export_entities_to_txt(
        entities,
        stats,
        "entita.txt",
        spacy_recognizer=linker.wikidata.spacy_recognizer,
        italian_recognizer=linker.italian_recognizer,
        original_text=text
    )

    print(f"\nFile di output salvato in: {output_file_path}")

    print("\nStatistiche:")
    for key, value in stats.items():
        print(f"- {key}: {value}")

    # Verifica aggiuntiva per il file
    import os
    if os.path.exists("entita.txt"):
        file_size = os.path.getsize("entita.txt")
        print(f"\nVerifica file: 'entita.txt' esiste e ha dimensione {file_size} bytes")
    else:
        print("\nATTENZIONE: Il file 'entita.txt' non è stato creato!")
        # Prova a identificare il problema
        try:
            with open("test_write_permission.txt", "w") as f:
                f.write("Test")
            print("La directory è scrivibile, il problema è altrove")
            os.remove("test_write_permission.txt")
        except Exception as e:
            print(f"Errore nell'accesso alla directory: {e}")
            print("Prova a specificare un percorso completo per il file di output")
            # Tenta con percorso nella home directory dell'utente
            home_dir = os.path.expanduser("~")
            output_path = os.path.join(home_dir, "entita.txt")
            try:
                export_entities_to_txt(entities, stats, output_path)
                print(f"File salvato con successo in: {output_path}")
            except Exception as e2:
                print(f"Errore anche con percorso alternativo: {e2}")

Modello spaCy it_core_news_lg caricato con successo.


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: it (Italian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/it/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| ner       | fbk               |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Riconoscitore NLP italiano inizializzato correttamente con Stanza.
Analisi del testo:
L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico. Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento. Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio. Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export. Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa. Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo. La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo. Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia. Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più g

In [ ]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict, Any, Optional, Tuple, Set # Added missing Any type hint
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch


# Importazione di spaCy
import spacy

# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy

# Modifica alla funzione export_entities_to_txt per includere le entità del riconoscitore italiano
#def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, italian_recognizer=None, original_text=None):

def export_entities_to_txt(
    entities,
    stats,
    output_file="entita_trovate.txt",
    spacy_recognizer=None,
    italian_recognizer=None,
    transformers_recognizer=None,  # Nuovo parametro
    original_text=None
    ):
    # ... resto del codice invariato ...
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Include anche le entità riconosciute da spaCy e dal riconoscitore italiano.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy (opzionale)
        italian_recognizer: Riconoscitore italiano (opzionale)
        original_text: Testo originale da analizzare (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]
    spacy_entities = [e for e in entities if e.get('source') == 'spacy']
    italian_entities = [e for e in entities if e.get('source') in ['italian_nlp', 'stanza', 'stanza_pos', 'geo_rule']]
#    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']


    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in wikidata_entities:
        lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")

    # Aggiungi questa parte nella funzione export_entities_to_txt
    # Sezione entità del riconoscitore Transformer NER
    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']

    lines.append("TRANSFORMER NER ENTITIES:")
    lines.append("-------------------------")
    if transformers_entities:
        for entity in transformers_entities:
            lines.append(f"- '{entity['text']}' → Etichetta: {entity.get('label', 'N/A')}, Punteggio: {entity.get('score', 'N/A')}")
    else:
        lines.append("Nessuna entità rilevata dal riconoscitore Transformer NER.")
    lines.append("")

    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_all_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_all_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback alle entità spaCy già filtrate
        if not spacy_entities:
            lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
        else:
            for entity in spacy_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore italiano
    lines.append("ITALIAN NLP ENTITIES:")
    lines.append("--------------------")

    # Se abbiamo un riconoscitore italiano e un testo originale, ottieni tutte le entità
    if italian_recognizer and original_text:
        try:
            italian_all_entities = italian_recognizer.recognize_entities(original_text)

            if not italian_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore italiano nel testo.")
            else:
                for entity in italian_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore italiano: {e}")
    else:
        # Fallback alle entità italiane già filtrate
        if not italian_entities:
            lines.append("Per mostrare tutte le entità del riconoscitore italiano, è necessario passare il riconoscitore e il testo originale.")
        else:
            for entity in italian_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")

    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities


class ItalianNLPRecognizer:
    """
    Riconoscitore di entità ottimizzato per l'italiano in ambiente Colab.
    Usa una combinazione di Stanza (versione Python di Stanford CoreNLP)
    e altre librerie per l'italiano.
    """

    def __init__(self, use_gpu: bool = False):
        """
        Inizializza il riconoscitore per l'italiano.

        Args:
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        self.use_gpu = use_gpu
        self.is_available = False
        self.stanza_nlp = None
        self.initialized = False

        # Inizializza le risorse
        self._initialize_resources()

    def _initialize_resources(self):
        """Inizializza Stanza e altre risorse per l'italiano."""
        try:
            import stanza
            import simplemma
            import nltk
            from nltk.tokenize import word_tokenize

            # Scarica i modelli di Stanza per l'italiano
            stanza.download('it')

            # Inizializza il pipeline Stanza per l'italiano
            self.stanza_nlp = stanza.Pipeline(
                lang='it',
                processors='tokenize,mwt,pos,lemma,ner',
                use_gpu=self.use_gpu
            )

            # Scarica le risorse NLTK necessarie
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('averaged_perceptron_tagger')

            self.is_available = True
            self.initialized = True
            print("Riconoscitore NLP italiano inizializzato correttamente con Stanza.")

        except Exception as e:
            print(f"Errore nell'inizializzazione del riconoscitore italiano: {e}")
            print("Assicurati di aver eseguito le installazioni necessarie in Colab.")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo italiano utilizzando Stanza.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.stanza_nlp:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con Stanza
            doc = self.stanza_nlp(text)

            # Estrai le entità
            for sent in doc.sentences:
                for ent in sent.ents:
                    entity = {
                        'text': ent.text,
                        'label': ent.type,
                        'start_char': -1,  # Stanza non fornisce direttamente le posizioni dei caratteri
                        'end_char': -1,
                        'source': 'stanza'
                    }
                    entities.append(entity)

                # Aggiungi anche i nomi propri (POS tag == PROPN)
                for token in sent.tokens:
                    for word in token.words:
                        if word.upos == 'PROPN' and not any(e['text'] == word.text for e in entities):
                            entity = {
                                'text': word.text,
                                'label': 'PROPN',
                                'start_char': -1,
                                'end_char': -1,
                                'source': 'stanza_pos'
                            }
                            entities.append(entity)

            # Cerca anche entità geografiche italiane specifiche usando regole
            entities.extend(self._extract_italian_geo_entities(text))

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Stanza: {e}")

        return entities

    def _extract_italian_geo_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Estrae entità geografiche italiane usando regole.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità geografiche
        """
        import re
        entities = []

        # Regioni italiane
        regions = [
            "Abruzzo", "Basilicata", "Calabria", "Campania", "Emilia-Romagna",
            "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardia", "Marche",
            "Molise", "Piemonte", "Puglia", "Sardegna", "Sicilia", "Toscana",
            "Trentino-Alto Adige", "Umbria", "Valle d'Aosta", "Veneto"
        ]

        # Pattern per laghi, montagne, fiumi italiani
        geo_patterns = [
            r'\b(?:Lago|Monte|Fiume|Isola|Golfo|Capo|Valle|Parco Nazionale|Mar) [A-Z][a-zàèéìòù]+\b',
            r'\b(?:Alpi|Appennini|Dolomiti|Maremma|Laguna|Costa|Riviera) [A-Z][a-zàèéìòù]*\b'
        ]

        # Cerca regioni
        for region in regions:
            if re.search(r'\b' + re.escape(region) + r'\b', text):
                entity = {
                    'text': region,
                    'label': 'GPE',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        # Cerca pattern geografici
        for pattern in geo_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                entity = {
                    'text': match.group(0),
                    'label': 'LOC',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        return entities


# Alternative per modelli italiani NER
model_options = [
    "5had3/bert-base-italian-cased-ner",  # Un'alternativa affidabile
    "MilaNLProc/bert-italian-cased-ner",  # Altro modello italiano per NER
    "dslim/bert-base-NER"  # Modello generico che funziona bene
]


class TransformersNERRecognizer:
    def __init__(self,
                 model_name: str = None,
                 language: str = "it",
                 use_gpu: bool = False):
        """
        Inizializza il riconoscitore di entità con un modello Transformer.

        Args:
            model_name: Nome del modello pre-addestrato per NER
            language: Lingua del modello (default: italiano)
            use_gpu: Se utilizzare l'accelerazione GPU
        """
        self.language = language
        self.use_gpu = use_gpu

        # Dizionario di mapping per modelli in diverse lingue
        self.language_models = {
            "it": [
                "5had3/bert-base-italian-cased-ner",
                "MilaNLProc/bert-italian-cased-ner",
                "dslim/bert-base-NER"
            ],
            "en": ["dslim/bert-base-NER"],
            "de": ["RafLorem/bert-base-german-NER"],
            "fr": ["Jean-Baptiste/camembert-ner"]
        }

        # Se non specificato, scegli un modello predefinito
        if model_name is None:
            model_name = self._select_best_model()

        self.model_name = model_name
        self.is_available = False
        self.ner_pipeline = None

        # Inizializza il modello
        self._load_model()

    def _select_best_model(self) -> str:
        """
        Seleziona il miglior modello disponibile per la lingua.

        Returns:
            Nome del modello
        """
        models = self.language_models.get(self.language, self.language_models["en"])

        for model in models:
            try:
                # Prova a caricare il modello
                AutoTokenizer.from_pretrained(model)
                return model
            except Exception:
                continue

        # Fallback
        return "dslim/bert-base-NER"

    def _load_model(self):
        """
        Carica il modello Transformer per il riconoscimento di entità.
        """
        try:
            # Determina il device
            device = 0 if self.use_gpu and torch.cuda.is_available() else -1

            # Carica tokenizer e modello
            tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            model = AutoModelForTokenClassification.from_pretrained(self.model_name)

            # Crea pipeline NER
            self.ner_pipeline = pipeline(
                "ner",
                model=model,
                tokenizer=tokenizer,
                device=device
            )

            self.is_available = True
            print(f"Modello Transformer NER caricato: {self.model_name}")

        except Exception as e:
            print(f"Errore nel caricamento del modello Transformer NER: {e}")
            print("Assicurati di avere installato transformers e torch.")
            print("Prova a installare i modelli con:")
            print("pip install transformers torch")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando un modello Transformer.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.ner_pipeline:
            print("Riconoscitore Transformer NER non disponibile.")
            return []

        entities = []

        try:
            # Esegui NER
            ner_results = self.ner_pipeline(text)

            # Raggruppa entità consecutive con lo stesso tipo
            grouped_entities = []
            current_entity = None

            for result in ner_results:
                if result['entity'].startswith('B-'):
                    # Nuova entità
                    if current_entity:
                        grouped_entities.append(current_entity)

                    current_entity = {
                        'text': result['word'],
                        'label': result['entity'][2:],
                        'start_char': result['start'],
                        'end_char': result['end'],
                        'score': result['score']
                    }

                elif result['entity'].startswith('I-') and current_entity:
                    # Continua l'entità precedente
                    if result['entity'][2:] == current_entity['label']:
                        current_entity['text'] += result['word'].replace('##', '')
                        current_entity['end_char'] = result['end']
                        current_entity['score'] = max(current_entity['score'], result['score'])

                else:
                    # Caso di fallback
                    if current_entity:
                        grouped_entities.append(current_entity)
                        current_entity = None

            # Aggiungi ultima entità
            if current_entity:
                grouped_entities.append(current_entity)

            # Aggiungi source
            for entity in grouped_entities:
                entity['source'] = 'transformers_ner'
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Transformer NER: {e}")

        return entities

    def set_language(self, language: str):
        """
        Cambia la lingua del riconoscitore.

        Args:
            language: Nuova lingua da impostare
        """
        if language != self.language:
            self.language = language
            self._load_model()

    def get_supported_languages(self) -> List[str]:
        """
        Restituisce le lingue supportate.

        Returns:
            Lista di codici lingua supportati
        """
        return list(self.language_models.keys())



# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }



# Estendi la classe EntityLinkerWithSpacy per incorporare il riconoscitore italiano
class EntityLinkerItalian(EntityLinkerWithSpacy):
    """
    Versione estesa di EntityLinker ottimizzata per l'italiano in ambiente Colab.
    Integra riconoscitori multipli: Wikidata, spaCy, Stanza NLP e Transformer NER.
    """

    def __init__(self, language: str = "it", use_gpu: bool = False):
        """
        Inizializza il linker di entità con supporto NLP per l'italiano.

        Args:
            language: Lingua preferita per i risultati (deve essere "it")
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        # Controlla che la lingua sia italiana
        if language != "it":
            print("Attenzione: questo riconoscitore è ottimizzato per l'italiano. Impostiamo language='it'.")
            language = "it"

        # Inizializza la classe base (con spaCy)
        super().__init__(language)

        # Aggiungi il riconoscitore italiano Stanza NLP
        self.italian_recognizer = ItalianNLPRecognizer(use_gpu=use_gpu)

        # Aggiungi il riconoscitore Transformer NER
        try:
            from transformers import AutoTokenizer  # Verifica disponibilità
            self.transformers_recognizer = TransformersNERRecognizer(
                model_name="dbmdz/bert-base-italian-cased-ner",
                language=language,
                use_gpu=use_gpu
            )
        except ImportError:
            print("Transformer NER non disponibile. Installare 'transformers' e 'torch'.")
            self.transformers_recognizer = None

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando Wikidata, spaCy, Stanza NLP e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Debug del riconoscitore Transformer
        print(f"DEBUG: Stato riconoscitore Transformer - Disponibile: {hasattr(self, 'transformers_recognizer')}")
        if hasattr(self, 'transformers_recognizer'):
            print(f"DEBUG: Riconoscitore Transformer - Is Available: {self.transformers_recognizer.is_available}")

        # Prima ottieni i risultati dalla classe base (Wikidata + spaCy)
        enriched_entities, stats = super().find_entities(text)
        existing_texts = {e.get('text', '').lower() for e in enriched_entities if 'text' in e}

        # Aggiungi le entità del riconoscitore italiano se disponibile
        italian_only_entities = self._get_italian_only_entities(text, existing_texts)
        enriched_entities.extend(italian_only_entities)

        # Aggiungi le entità dal riconoscitore Transformer NER
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer and self.transformers_recognizer.is_available:
            print("DEBUG: Tentativo di trovare entità Transformer NER")
            transformers_only_entities = self._get_transformers_only_entities(text, existing_texts)
            print(f"DEBUG: Entità Transformer NER trovate: {len(transformers_only_entities)}")
            enriched_entities.extend(transformers_only_entities)
            stats['transformers_only_entities'] = len(transformers_only_entities)
        else:
            print("DEBUG: Riconoscitore Transformer NER non disponibile")

        # Aggiorna le statistiche
        stats['italian_only_entities'] = len(italian_only_entities)
        stats['total_entities'] = len(enriched_entities)

        return enriched_entities, stats

    def _get_italian_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore italiano e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo dal riconoscitore italiano
        """
        # Verifica che il riconoscitore italiano sia disponibile
        if not self.italian_recognizer.is_available:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore italiano
        italian_entities = self.italian_recognizer.recognize_entities(text)
        print(f"Riconoscitore italiano ha trovato {len(italian_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        italian_only = []
        for entity in italian_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore italiano: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                italian_type = self._map_italian_label_to_type(entity['label'])

                italian_entity = {
                    'text': entity_text,
                    'types': [italian_type],
                    'source': 'italian_nlp',
                    'label': entity['label']
                }

                italian_only.append(italian_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(italian_only)} sono uniche (non trovate da altre fonti)")
        return italian_only

    def _get_transformers_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore Transformer NER
        e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo da Transformer NER
        """
        # Verifica che il riconoscitore Transformer sia disponibile
        if not self.transformers_recognizer or not self.transformers_recognizer.is_available:
            print("Riconoscitore Transformer NER non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore Transformer NER
        transformers_entities = self.transformers_recognizer.recognize_entities(text)
        print(f"DEBUG: Riconoscitore Transformer NER ha trovato {len(transformers_entities)} entità nel testo")

        # Stampa dettagli delle entità trovate
        for entity in transformers_entities:
            print(f"DEBUG: Entità trovata - Testo: {entity['text']}, Etichetta: {entity['label']}, Punteggio: {entity.get('score', 'N/A')}")

        # Stampa testi esistenti
        print(f"DEBUG: Testi esistenti: {existing_texts}")

        # Filtra per entità uniche non trovate da altre fonti
        transformers_only = []
        for entity in transformers_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            print(f"DEBUG: Verifica entità - Testo: {entity_text}, Lowercase: {entity_lower}")

            # Rimuovi la condizione di validità temporaneamente
            if entity_lower not in existing_texts:
                print(f"DEBUG: Entità unica dal riconoscitore Transformer NER: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                transformers_type = self._map_transformers_label_to_type(entity['label'])

                transformers_entity = {
                    'text': entity_text,
                    'types': [transformers_type],
                    'source': 'transformers_ner',
                    'label': entity['label'],
                    'score': entity.get('score', 0)
                }

                transformers_only.append(transformers_entity)
                existing_texts.add(entity_lower)

        print(f"DEBUG: Entità Transformer NER uniche trovate: {len(transformers_only)}")
        return transformers_only

    def _map_italian_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore italiano a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        mapping = {
            'PERSON': 'Person',
            'PER': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'MISC': 'Miscellaneous',
            'PROPN': 'ProperNoun',
            'WORK_OF_ART': 'Artwork',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def _map_transformers_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore Transformer NER a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        # (simile a quella di altri riconoscitori)
        mapping = {
            'PER': 'Person',
            'LOC': 'Location',
            'ORG': 'Organization',
            'MISC': 'Miscellaneous',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'PERSON': 'Person',
            'ORGANIZATION': 'Organization',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estende il metodo della classe base per includere i risultati
        del riconoscitore italiano e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        # Ottieni i risultati di base
        uri_results = super().verify_uris_for_text(text)

        # Aggiungi una sezione per le entità del riconoscitore italiano senza URI
        italian_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'italian_nlp' and entity['text'] not in uri_results['results']:
                italian_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        # Aggiungi una sezione per le entità del riconoscitore Transformer NER senza URI
        transformers_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'transformers_ner' and entity['text'] not in uri_results['results']:
                transformers_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', []),
                    'score': entity.get('score', 0)
                }

        # Aggiungi le sezioni ai risultati
        uri_results['italian_results'] = italian_results
        uri_results['transformers_results'] = transformers_results

        return uri_results


# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

if __name__ == "__main__":
    # Crea un'istanza del linker di entità migliorato
#    linker = EntityLinkerWithSpacy(language="it")
    linker = EntityLinkerItalian(language="it", use_gpu=False)


    # Testo di esempio esteso - usa lo stesso testo che stavi analizzando prima
    text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
           " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
           " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
           " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
           " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
           " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
           " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
           " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
           " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
           " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

    print(f"Analisi del testo:\n{text}\n")

    # Trova entità
    entities, stats = linker.find_entities(text)

    print(f"\nTrovate {stats['total_entities']} entità:")
    for entity in entities:
        source = entity.get('source', 'sconosciuta')
        uri = entity.get('dbpedia_uri', 'N/A')
        types = ', '.join(entity.get('types', ['Sconosciuto']))

        print(f"- '{entity['text']}' → {uri}")
        print(f"  Tipo: {types}")
        if 'wikidata_id' in entity:
            print(f"  Wikidata: {entity.get('wikidata_url', 'N/A')}")
        print(f"  Fonte: {source}")
        print("")

    # Verifica URI
    uri_results = linker.verify_uris_for_text(text)

    print("\nVerifica URI:")
    for word, info in uri_results['results'].items():
        print(f"- La parola '{word}' → URI: {info['uri']} esiste")

    # Mostra entità spaCy senza URI
    if 'spacy_results' in uri_results and uri_results['spacy_results']:
        print("\nEntità riconosciute solo da spaCy (senza URI):")
        for word, info in uri_results['spacy_results'].items():
            print(f"- '{word}' → Etichetta: {info.get('label', '')}, Tipi: {', '.join(info.get('types', []))}")

    # Mostra solo le entità con URI DBpedia
    print("\nEntità con URI DBpedia:")
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    for entity in entities_with_dbpedia:
        print(f"- '{entity['text']}' → {entity['dbpedia_uri']}")

    # Esporta le entità
    output_file_path = export_entities_to_txt(
    entities,
    stats,
    "entita.txt",
    spacy_recognizer=linker.wikidata.spacy_recognizer,
    italian_recognizer=linker.italian_recognizer,
    transformers_recognizer=linker.transformers_recognizer,  # Aggiungi questo
    original_text=text
    )

    print(f"\nFile di output salvato in: {output_file_path}")

    print("\nStatistiche:")
    for key, value in stats.items():
        print(f"- {key}: {value}")

    # Verifica aggiuntiva per il file
    import os
    if os.path.exists("entita.txt"):
        file_size = os.path.getsize("entita.txt")
        print(f"\nVerifica file: 'entita.txt' esiste e ha dimensione {file_size} bytes")
    else:
        print("\nATTENZIONE: Il file 'entita.txt' non è stato creato!")
        # Prova a identificare il problema
        try:
            with open("test_write_permission.txt", "w") as f:
                f.write("Test")
            print("La directory è scrivibile, il problema è altrove")
            os.remove("test_write_permission.txt")
        except Exception as e:
            print(f"Errore nell'accesso alla directory: {e}")
            print("Prova a specificare un percorso completo per il file di output")
            # Tenta con percorso nella home directory dell'utente
            home_dir = os.path.expanduser("~")
            output_path = os.path.join(home_dir, "entita.txt")
            try:
                export_entities_to_txt(entities, stats, output_path)
                print(f"File salvato con successo in: {output_path}")
            except Exception as e2:
                print(f"Errore anche con percorso alternativo: {e2}")

Modello spaCy it_core_news_lg caricato con successo.


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: it (Italian) ...
INFO:stanza:File exists: /root/stanza_resources/it/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| ner       | fbk               |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Riconoscitore NLP italiano inizializzato correttamente con Stanza.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Errore nel caricamento del modello Transformer NER: dbmdz/bert-base-italian-cased-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Assicurati di avere installato transformers e torch.
Prova a installare i modelli con:
pip install transformers torch
Analisi del testo:
L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico. Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento. Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio. Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export. Le università italiane, come la Sapienza e il Politecnico di Milano, 

In [ ]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict, Any, Optional, Tuple, Set # Added missing Any type hint
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch


# Importazione di spaCy
import spacy

# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy

# Modifica alla funzione export_entities_to_txt per includere le entità del riconoscitore italiano
#def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, italian_recognizer=None, original_text=None):

def export_entities_to_txt(
    entities,
    stats,
    output_file="entita.txt",
    spacy_recognizer=None,
    italian_recognizer=None,
    transformers_recognizer=None,  # Nuovo parametro
    original_text=None
    ):
    # ... resto del codice invariato ...
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Include anche le entità riconosciute da spaCy e dal riconoscitore italiano.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy (opzionale)
        italian_recognizer: Riconoscitore italiano (opzionale)
        original_text: Testo originale da analizzare (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]
    spacy_entities = [e for e in entities if e.get('source') == 'spacy']
    italian_entities = [e for e in entities if e.get('source') in ['italian_nlp', 'stanza', 'stanza_pos', 'geo_rule']]
#    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']


    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in wikidata_entities:
        lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")


    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_all_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_all_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback alle entità spaCy già filtrate
        if not spacy_entities:
            lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
        else:
            for entity in spacy_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore italiano
    lines.append("ITALIAN NLP ENTITIES:")
    lines.append("--------------------")

    # Se abbiamo un riconoscitore italiano e un testo originale, ottieni tutte le entità
    if italian_recognizer and original_text:
        try:
            italian_all_entities = italian_recognizer.recognize_entities(original_text)

            if not italian_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore italiano nel testo.")
            else:
                for entity in italian_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore italiano: {e}")
    else:
        # Fallback alle entità italiane già filtrate
        if not italian_entities:
            lines.append("Per mostrare tutte le entità del riconoscitore italiano, è necessario passare il riconoscitore e il testo originale.")
        else:
            for entity in italian_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")





    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities


class ItalianNLPRecognizer:
    """
    Riconoscitore di entità ottimizzato per l'italiano in ambiente Colab.
    Usa una combinazione di Stanza (versione Python di Stanford CoreNLP)
    e altre librerie per l'italiano.
    """

    def __init__(self, use_gpu: bool = False):
        """
        Inizializza il riconoscitore per l'italiano.

        Args:
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        self.use_gpu = use_gpu
        self.is_available = False
        self.stanza_nlp = None
        self.initialized = False

        # Inizializza le risorse
        self._initialize_resources()

    def _initialize_resources(self):
        """Inizializza Stanza e altre risorse per l'italiano."""
        try:
            import stanza
            import simplemma
            import nltk
            from nltk.tokenize import word_tokenize

            # Scarica i modelli di Stanza per l'italiano
            stanza.download('it')

            # Inizializza il pipeline Stanza per l'italiano
            self.stanza_nlp = stanza.Pipeline(
                lang='it',
                processors='tokenize,mwt,pos,lemma,ner',
                use_gpu=self.use_gpu
            )

            # Scarica le risorse NLTK necessarie
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('averaged_perceptron_tagger')

            self.is_available = True
            self.initialized = True
            print("Riconoscitore NLP italiano inizializzato correttamente con Stanza.")

        except Exception as e:
            print(f"Errore nell'inizializzazione del riconoscitore italiano: {e}")
            print("Assicurati di aver eseguito le installazioni necessarie in Colab.")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo italiano utilizzando Stanza.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.stanza_nlp:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con Stanza
            doc = self.stanza_nlp(text)

            # Estrai le entità
            for sent in doc.sentences:
                for ent in sent.ents:
                    entity = {
                        'text': ent.text,
                        'label': ent.type,
                        'start_char': -1,  # Stanza non fornisce direttamente le posizioni dei caratteri
                        'end_char': -1,
                        'source': 'stanza'
                    }
                    entities.append(entity)

                # Aggiungi anche i nomi propri (POS tag == PROPN)
                for token in sent.tokens:
                    for word in token.words:
                        if word.upos == 'PROPN' and not any(e['text'] == word.text for e in entities):
                            entity = {
                                'text': word.text,
                                'label': 'PROPN',
                                'start_char': -1,
                                'end_char': -1,
                                'source': 'stanza_pos'
                            }
                            entities.append(entity)

            # Cerca anche entità geografiche italiane specifiche usando regole
            entities.extend(self._extract_italian_geo_entities(text))

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Stanza: {e}")

        return entities

    def _extract_italian_geo_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Estrae entità geografiche italiane usando regole.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità geografiche
        """
        import re
        entities = []

        # Regioni italiane
        regions = [
            "Abruzzo", "Basilicata", "Calabria", "Campania", "Emilia-Romagna",
            "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardia", "Marche",
            "Molise", "Piemonte", "Puglia", "Sardegna", "Sicilia", "Toscana",
            "Trentino-Alto Adige", "Umbria", "Valle d'Aosta", "Veneto"
        ]

        # Pattern per laghi, montagne, fiumi italiani
        geo_patterns = [
            r'\b(?:Lago|Monte|Fiume|Isola|Golfo|Capo|Valle|Parco Nazionale|Mar) [A-Z][a-zàèéìòù]+\b',
            r'\b(?:Alpi|Appennini|Dolomiti|Maremma|Laguna|Costa|Riviera) [A-Z][a-zàèéìòù]*\b'
        ]

        # Cerca regioni
        for region in regions:
            if re.search(r'\b' + re.escape(region) + r'\b', text):
                entity = {
                    'text': region,
                    'label': 'GPE',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        # Cerca pattern geografici
        for pattern in geo_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                entity = {
                    'text': match.group(0),
                    'label': 'LOC',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        return entities


# Alternative per modelli italiani NER
model_options = [
    "5had3/bert-base-italian-cased-ner",  # Un'alternativa affidabile
    "MilaNLProc/bert-italian-cased-ner",  # Altro modello italiano per NER
    "dslim/bert-base-NER"  # Modello generico che funziona bene
]


class TransformersNERRecognizer:
    def __init__(self,
                 model_name: str = None,
                 language: str = "it",
                 use_gpu: bool = False):
        """
        Inizializza il riconoscitore di entità con un modello Transformer.

        Args:
            model_name: Nome del modello pre-addestrato per NER
            language: Lingua del modello (default: italiano)
            use_gpu: Se utilizzare l'accelerazione GPU
        """
        self.language = language
        self.use_gpu = use_gpu

        # Dizionario di mapping per modelli in diverse lingue
        self.language_models = {
            "it": [
                "5had3/bert-base-italian-cased-ner",
                "MilaNLProc/bert-italian-cased-ner",
                "dslim/bert-base-NER"
            ],
            "en": ["dslim/bert-base-NER"],
            "de": ["RafLorem/bert-base-german-NER"],
            "fr": ["Jean-Baptiste/camembert-ner"]
        }

        # Se non specificato, scegli un modello predefinito
        if model_name is None:
            model_name = self._select_best_model()

        self.model_name = model_name
        self.is_available = False
        self.ner_pipeline = None

        # Inizializza il modello
        self._load_model()

    def _select_best_model(self) -> str:
        """
        Seleziona il miglior modello disponibile per la lingua.

        Returns:
            Nome del modello
        """
        models = self.language_models.get(self.language, self.language_models["en"])

        for model in models:
            try:
                # Prova a caricare il modello
                AutoTokenizer.from_pretrained(model)
                return model
            except Exception:
                continue

        # Fallback
        return "dslim/bert-base-NER"

    def _load_model(self):
        """
        Carica il modello Transformer per il riconoscimento di entità.
        """
        try:
            # Lista di modelli italiani alternativi
            alternative_models = [
                "5had3/bert-base-italian-cased-ner",
                "MilaNLProc/bert-italian-cased-ner",
                "Davlan/bert-base-multilingual-cased-ner-hrl"
            ]

            # Prova a caricare uno dei modelli
            for model_name in alternative_models:
                try:
                    # Determina il device
                    device = 0 if self.use_gpu and torch.cuda.is_available() else -1

                    # Carica tokenizer e modello
                    tokenizer = AutoTokenizer.from_pretrained(model_name)
                    model = AutoModelForTokenClassification.from_pretrained(model_name)

                    # Crea pipeline NER
                    self.ner_pipeline = pipeline(
                        "ner",
                        model=model,
                        tokenizer=tokenizer,
                        device=device
                    )

                    self.is_available = True
                    self.model_name = model_name
                    print(f"Modello Transformer NER caricato: {model_name}")
                    return

                except Exception as inner_e:
                    print(f"Tentativo fallito con modello {model_name}: {inner_e}")

            # Se nessun modello funziona
            raise Exception("Nessun modello NER disponibile")

        except Exception as e:
            print(f"Errore nel caricamento del modello Transformer NER: {e}")
            print("Assicurati di avere installato transformers e torch.")
            print("Prova a installare i modelli con:")
            print("pip install transformers torch")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando un modello Transformer.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.ner_pipeline:
            print("Riconoscitore Transformer NER non disponibile.")
            return []

        entities = []

        try:
            # Esegui NER
            ner_results = self.ner_pipeline(text)

            # Raggruppa entità consecutive con lo stesso tipo
            grouped_entities = []
            current_entity = None

            for result in ner_results:
                if result['entity'].startswith('B-'):
                    # Nuova entità
                    if current_entity:
                        grouped_entities.append(current_entity)

                    current_entity = {
                        'text': result['word'],
                        'label': result['entity'][2:],
                        'start_char': result['start'],
                        'end_char': result['end'],
                        'score': result['score']
                    }

                elif result['entity'].startswith('I-') and current_entity:
                    # Continua l'entità precedente
                    if result['entity'][2:] == current_entity['label']:
                        current_entity['text'] += result['word'].replace('##', '')
                        current_entity['end_char'] = result['end']
                        current_entity['score'] = max(current_entity['score'], result['score'])

                else:
                    # Caso di fallback
                    if current_entity:
                        grouped_entities.append(current_entity)
                        current_entity = None

            # Aggiungi ultima entità
            if current_entity:
                grouped_entities.append(current_entity)

            # Aggiungi source
            for entity in grouped_entities:
                entity['source'] = 'transformers_ner'
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Transformer NER: {e}")

        return entities

    def set_language(self, language: str):
        """
        Cambia la lingua del riconoscitore.

        Args:
            language: Nuova lingua da impostare
        """
        if language != self.language:
            self.language = language
            self._load_model()

    def get_supported_languages(self) -> List[str]:
        """
        Restituisce le lingue supportate.

        Returns:
            Lista di codici lingua supportati
        """
        return list(self.language_models.keys())



# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }



# Estendi la classe EntityLinkerWithSpacy per incorporare il riconoscitore italiano
class EntityLinkerItalian(EntityLinkerWithSpacy):
    """
    Versione estesa di EntityLinker ottimizzata per l'italiano in ambiente Colab.
    Integra riconoscitori multipli: Wikidata, spaCy, Stanza NLP e Transformer NER.
    """

    def __init__(self, language: str = "it", use_gpu: bool = False):
        """
        Inizializza il linker di entità con supporto NLP per l'italiano.

        Args:
            language: Lingua preferita per i risultati (deve essere "it")
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        # Controlla che la lingua sia italiana
        if language != "it":
            print("Attenzione: questo riconoscitore è ottimizzato per l'italiano. Impostiamo language='it'.")
            language = "it"

        # Inizializza la classe base (con spaCy)
        super().__init__(language)

        # Aggiungi il riconoscitore italiano Stanza NLP
        self.italian_recognizer = ItalianNLPRecognizer(use_gpu=use_gpu)

        # Aggiungi il riconoscitore Transformer NER
        try:
            from transformers import AutoTokenizer  # Verifica disponibilità
            self.transformers_recognizer = TransformersNERRecognizer(
                model_name="dbmdz/bert-base-italian-cased-ner",
                language=language,
                use_gpu=use_gpu
            )
        except ImportError:
            print("Transformer NER non disponibile. Installare 'transformers' e 'torch'.")
            self.transformers_recognizer = None

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando Wikidata, spaCy, Stanza NLP e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Debug del riconoscitore Transformer
        print(f"DEBUG: Stato riconoscitore Transformer - Disponibile: {hasattr(self, 'transformers_recognizer')}")
        if hasattr(self, 'transformers_recognizer'):
            print(f"DEBUG: Riconoscitore Transformer - Is Available: {self.transformers_recognizer.is_available}")

        # Prima ottieni i risultati dalla classe base (Wikidata + spaCy)
        enriched_entities, stats = super().find_entities(text)
        existing_texts = {e.get('text', '').lower() for e in enriched_entities if 'text' in e}

        # Aggiungi le entità del riconoscitore italiano se disponibile
        italian_only_entities = self._get_italian_only_entities(text, existing_texts)
        enriched_entities.extend(italian_only_entities)

        # Aggiungi le entità dal riconoscitore Transformer NER
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer and self.transformers_recognizer.is_available:
            print("DEBUG: Tentativo di trovare entità Transformer NER")
            transformers_only_entities = self._get_transformers_only_entities(text, existing_texts)
            print(f"DEBUG: Entità Transformer NER trovate: {len(transformers_only_entities)}")
            enriched_entities.extend(transformers_only_entities)
            stats['transformers_only_entities'] = len(transformers_only_entities)
        else:
            print("DEBUG: Riconoscitore Transformer NER non disponibile")

        # Aggiorna le statistiche
        stats['italian_only_entities'] = len(italian_only_entities)
        stats['total_entities'] = len(enriched_entities)

        return enriched_entities, stats

    def _get_italian_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore italiano e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo dal riconoscitore italiano
        """
        # Verifica che il riconoscitore italiano sia disponibile
        if not self.italian_recognizer.is_available:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore italiano
        italian_entities = self.italian_recognizer.recognize_entities(text)
        print(f"Riconoscitore italiano ha trovato {len(italian_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        italian_only = []
        for entity in italian_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore italiano: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                italian_type = self._map_italian_label_to_type(entity['label'])

                italian_entity = {
                    'text': entity_text,
                    'types': [italian_type],
                    'source': 'italian_nlp',
                    'label': entity['label']
                }

                italian_only.append(italian_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(italian_only)} sono uniche (non trovate da altre fonti)")
        return italian_only

    def _get_transformers_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore Transformer NER
        e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo da Transformer NER
        """
        # Verifica che il riconoscitore Transformer sia disponibile
        if not self.transformers_recognizer or not self.transformers_recognizer.is_available:
            print("Riconoscitore Transformer NER non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore Transformer NER
        transformers_entities = self.transformers_recognizer.recognize_entities(text)
        print(f"DEBUG: Riconoscitore Transformer NER ha trovato {len(transformers_entities)} entità nel testo")

        # Stampa dettagli delle entità trovate
        for entity in transformers_entities:
            print(f"DEBUG: Entità trovata - Testo: {entity['text']}, Etichetta: {entity['label']}, Punteggio: {entity.get('score', 'N/A')}")

        # Stampa testi esistenti
        print(f"DEBUG: Testi esistenti: {existing_texts}")

        # Filtra per entità uniche non trovate da altre fonti
        transformers_only = []
        for entity in transformers_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            print(f"DEBUG: Verifica entità - Testo: {entity_text}, Lowercase: {entity_lower}")

            # Rimuovi la condizione di validità temporaneamente
            if entity_lower not in existing_texts:
                print(f"DEBUG: Entità unica dal riconoscitore Transformer NER: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                transformers_type = self._map_transformers_label_to_type(entity['label'])

                transformers_entity = {
                    'text': entity_text,
                    'types': [transformers_type],
                    'source': 'transformers_ner',
                    'label': entity['label'],
                    'score': entity.get('score', 0)
                }

                transformers_only.append(transformers_entity)
                existing_texts.add(entity_lower)

        print(f"DEBUG: Entità Transformer NER uniche trovate: {len(transformers_only)}")
        return transformers_only

    def _map_italian_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore italiano a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        mapping = {
            'PERSON': 'Person',
            'PER': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'MISC': 'Miscellaneous',
            'PROPN': 'ProperNoun',
            'WORK_OF_ART': 'Artwork',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def _map_transformers_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore Transformer NER a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        # (simile a quella di altri riconoscitori)
        mapping = {
            'PER': 'Person',
            'LOC': 'Location',
            'ORG': 'Organization',
            'MISC': 'Miscellaneous',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'PERSON': 'Person',
            'ORGANIZATION': 'Organization',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estende il metodo della classe base per includere i risultati
        del riconoscitore italiano e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        # Ottieni i risultati di base
        uri_results = super().verify_uris_for_text(text)

        # Aggiungi una sezione per le entità del riconoscitore italiano senza URI
        italian_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'italian_nlp' and entity['text'] not in uri_results['results']:
                italian_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        # Aggiungi una sezione per le entità del riconoscitore Transformer NER senza URI
        transformers_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'transformers_ner' and entity['text'] not in uri_results['results']:
                transformers_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', []),
                    'score': entity.get('score', 0)
                }

        # Aggiungi le sezioni ai risultati
        uri_results['italian_results'] = italian_results
        uri_results['transformers_results'] = transformers_results

        return uri_results


# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

if __name__ == "__main__":
    # Crea un'istanza del linker di entità migliorato
#    linker = EntityLinkerWithSpacy(language="it")
    linker = EntityLinkerItalian(language="it", use_gpu=False)


    # Testo di esempio esteso - usa lo stesso testo che stavi analizzando prima
    text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
           " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
           " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
           " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
           " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
           " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
           " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
           " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
           " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
           " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

    print(f"Analisi del testo:\n{text}\n")

    # Trova entità
    entities, stats = linker.find_entities(text)

    print(f"\nTrovate {stats['total_entities']} entità:")
    for entity in entities:
        source = entity.get('source', 'sconosciuta')
        uri = entity.get('dbpedia_uri', 'N/A')
        types = ', '.join(entity.get('types', ['Sconosciuto']))

        print(f"- '{entity['text']}' → {uri}")
        print(f"  Tipo: {types}")
        if 'wikidata_id' in entity:
            print(f"  Wikidata: {entity.get('wikidata_url', 'N/A')}")
        print(f"  Fonte: {source}")
        print("")

    # Verifica URI
    uri_results = linker.verify_uris_for_text(text)

    print("\nVerifica URI:")
    for word, info in uri_results['results'].items():
        print(f"- La parola '{word}' → URI: {info['uri']} esiste")

    # Mostra entità spaCy senza URI
    if 'spacy_results' in uri_results and uri_results['spacy_results']:
        print("\nEntità riconosciute solo da spaCy (senza URI):")
        for word, info in uri_results['spacy_results'].items():
            print(f"- '{word}' → Etichetta: {info.get('label', '')}, Tipi: {', '.join(info.get('types', []))}")

    # Mostra solo le entità con URI DBpedia
    print("\nEntità con URI DBpedia:")
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    for entity in entities_with_dbpedia:
        print(f"- '{entity['text']}' → {entity['dbpedia_uri']}")

    # Esporta le entità
    output_file_path = export_entities_to_txt(
    entities,
    stats,
    "entita.txt",
    spacy_recognizer=linker.wikidata.spacy_recognizer,
    italian_recognizer=linker.italian_recognizer,
    transformers_recognizer=linker.transformers_recognizer,  # Aggiungi questo
    original_text=text
    )

    print(f"\nFile di output salvato in: {output_file_path}")

    print("\nStatistiche:")
    for key, value in stats.items():
        print(f"- {key}: {value}")

    # Verifica aggiuntiva per il file
    import os
    if os.path.exists("entita.txt"):
        file_size = os.path.getsize("entita.txt")
        print(f"\nVerifica file: 'entita.txt' esiste e ha dimensione {file_size} bytes")
    else:
        print("\nATTENZIONE: Il file 'entita.txt' non è stato creato!")
        # Prova a identificare il problema
        try:
            with open("test_write_permission.txt", "w") as f:
                f.write("Test")
            print("La directory è scrivibile, il problema è altrove")
            os.remove("test_write_permission.txt")
        except Exception as e:
            print(f"Errore nell'accesso alla directory: {e}")
            print("Prova a specificare un percorso completo per il file di output")
            # Tenta con percorso nella home directory dell'utente
            home_dir = os.path.expanduser("~")
            output_path = os.path.join(home_dir, "entita.txt")
            try:
                export_entities_to_txt(entities, stats, output_path)
                print(f"File salvato con successo in: {output_path}")
            except Exception as e2:
                print(f"Errore anche con percorso alternativo: {e2}")

Modello spaCy it_core_news_lg caricato con successo.


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: it (Italian) ...
INFO:stanza:File exists: /root/stanza_resources/it/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| ner       | fbk               |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Riconoscitore NLP italiano inizializzato correttamente con Stanza.
Tentativo fallito con modello 5had3/bert-base-italian-cased-ner: 5had3/bert-base-italian-cased-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Tentativo fallito con modello MilaNLProc/bert-italian-cased-ner: MilaNLProc/bert-italian-cased-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`


Device set to use cpu


Modello Transformer NER caricato: Davlan/bert-base-multilingual-cased-ner-hrl
Analisi del testo:
L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico. Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento. Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio. Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export. Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa. Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo. La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo. Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia. Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono 

In [ ]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict, Any, Optional, Tuple, Set # Added missing Any type hint
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch


# Importazione di spaCy
import spacy

# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy

# Modifica alla funzione export_entities_to_txt per includere le entità del riconoscitore italiano
#def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, italian_recognizer=None, original_text=None):

def export_entities_to_txt(
    entities,
    stats,
    output_file="entita_trovate.txt",
    spacy_recognizer=None,
    italian_recognizer=None,
    transformers_recognizer=None,  # Nuovo parametro
    original_text=None
    ):
    # ... resto del codice invariato ...
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Include anche le entità riconosciute da spaCy e dal riconoscitore italiano.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy (opzionale)
        italian_recognizer: Riconoscitore italiano (opzionale)
        original_text: Testo originale da analizzare (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]
    spacy_entities = [e for e in entities if e.get('source') == 'spacy']
    italian_entities = [e for e in entities if e.get('source') in ['italian_nlp', 'stanza', 'stanza_pos', 'geo_rule']]
#    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']


    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in wikidata_entities:
        lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")

    # Aggiungi questa parte nella funzione export_entities_to_txt
    # Sezione entità del riconoscitore Transformer NER
    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']

    lines.append("TRANSFORMER NER ENTITIES:")
    lines.append("-------------------------")
    if transformers_entities:
        for entity in transformers_entities:
            lines.append(f"- '{entity['text']}' → Etichetta: {entity.get('label', 'N/A')}, Punteggio: {entity.get('score', 'N/A')}")
    else:
        lines.append("Nessuna entità rilevata dal riconoscitore Transformer NER.")
    lines.append("")

    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_all_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_all_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback alle entità spaCy già filtrate
        if not spacy_entities:
            lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
        else:
            for entity in spacy_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore italiano
    lines.append("ITALIAN NLP ENTITIES:")
    lines.append("--------------------")

    # Se abbiamo un riconoscitore italiano e un testo originale, ottieni tutte le entità
    if italian_recognizer and original_text:
        try:
            italian_all_entities = italian_recognizer.recognize_entities(original_text)

            if not italian_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore italiano nel testo.")
            else:
                for entity in italian_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore italiano: {e}")
    else:
        # Fallback alle entità italiane già filtrate
        if not italian_entities:
            lines.append("Per mostrare tutte le entità del riconoscitore italiano, è necessario passare il riconoscitore e il testo originale.")
        else:
            for entity in italian_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")

    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities


class ItalianNLPRecognizer:
    """
    Riconoscitore di entità ottimizzato per l'italiano in ambiente Colab.
    Usa una combinazione di Stanza (versione Python di Stanford CoreNLP)
    e altre librerie per l'italiano.
    """

    def __init__(self, use_gpu: bool = False):
        """
        Inizializza il riconoscitore per l'italiano.

        Args:
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        self.use_gpu = use_gpu
        self.is_available = False
        self.stanza_nlp = None
        self.initialized = False

        # Inizializza le risorse
        self._initialize_resources()

    def _initialize_resources(self):
        """Inizializza Stanza e altre risorse per l'italiano."""
        try:
            import stanza
            import simplemma
            import nltk
            from nltk.tokenize import word_tokenize

            # Scarica i modelli di Stanza per l'italiano
            stanza.download('it')

            # Inizializza il pipeline Stanza per l'italiano
            self.stanza_nlp = stanza.Pipeline(
                lang='it',
                processors='tokenize,mwt,pos,lemma,ner',
                use_gpu=self.use_gpu
            )

            # Scarica le risorse NLTK necessarie
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('averaged_perceptron_tagger')

            self.is_available = True
            self.initialized = True
            print("Riconoscitore NLP italiano inizializzato correttamente con Stanza.")

        except Exception as e:
            print(f"Errore nell'inizializzazione del riconoscitore italiano: {e}")
            print("Assicurati di aver eseguito le installazioni necessarie in Colab.")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo italiano utilizzando Stanza.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.stanza_nlp:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con Stanza
            doc = self.stanza_nlp(text)

            # Estrai le entità
            for sent in doc.sentences:
                for ent in sent.ents:
                    entity = {
                        'text': ent.text,
                        'label': ent.type,
                        'start_char': -1,  # Stanza non fornisce direttamente le posizioni dei caratteri
                        'end_char': -1,
                        'source': 'stanza'
                    }
                    entities.append(entity)

                # Aggiungi anche i nomi propri (POS tag == PROPN)
                for token in sent.tokens:
                    for word in token.words:
                        if word.upos == 'PROPN' and not any(e['text'] == word.text for e in entities):
                            entity = {
                                'text': word.text,
                                'label': 'PROPN',
                                'start_char': -1,
                                'end_char': -1,
                                'source': 'stanza_pos'
                            }
                            entities.append(entity)

            # Cerca anche entità geografiche italiane specifiche usando regole
            entities.extend(self._extract_italian_geo_entities(text))

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Stanza: {e}")

        return entities

    def _extract_italian_geo_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Estrae entità geografiche italiane usando regole.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità geografiche
        """
        import re
        entities = []

        # Regioni italiane
        regions = [
            "Abruzzo", "Basilicata", "Calabria", "Campania", "Emilia-Romagna",
            "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardia", "Marche",
            "Molise", "Piemonte", "Puglia", "Sardegna", "Sicilia", "Toscana",
            "Trentino-Alto Adige", "Umbria", "Valle d'Aosta", "Veneto"
        ]

        # Pattern per laghi, montagne, fiumi italiani
        geo_patterns = [
            r'\b(?:Lago|Monte|Fiume|Isola|Golfo|Capo|Valle|Parco Nazionale|Mar) [A-Z][a-zàèéìòù]+\b',
            r'\b(?:Alpi|Appennini|Dolomiti|Maremma|Laguna|Costa|Riviera) [A-Z][a-zàèéìòù]*\b'
        ]

        # Cerca regioni
        for region in regions:
            if re.search(r'\b' + re.escape(region) + r'\b', text):
                entity = {
                    'text': region,
                    'label': 'GPE',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        # Cerca pattern geografici
        for pattern in geo_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                entity = {
                    'text': match.group(0),
                    'label': 'LOC',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        return entities


# Alternative per modelli italiani NER
model_options = [
    "5had3/bert-base-italian-cased-ner",  # Un'alternativa affidabile
    "MilaNLProc/bert-italian-cased-ner",  # Altro modello italiano per NER
    "dslim/bert-base-NER"  # Modello generico che funziona bene
]


class TransformersNERRecognizer:
    def __init__(self,
                 model_name: str = None,
                 language: str = "it",
                 use_gpu: bool = False):
        """
        Inizializza il riconoscitore di entità con un modello Transformer.

        Args:
            model_name: Nome del modello pre-addestrato per NER
            language: Lingua del modello (default: italiano)
            use_gpu: Se utilizzare l'accelerazione GPU
        """
        self.language = language
        self.use_gpu = use_gpu

        # Dizionario di mapping per modelli in diverse lingue
        self.language_models = {
            "it": [
                "5had3/bert-base-italian-cased-ner",
                "MilaNLProc/bert-italian-cased-ner",
                "dslim/bert-base-NER"
            ],
            "en": ["dslim/bert-base-NER"],
            "de": ["RafLorem/bert-base-german-NER"],
            "fr": ["Jean-Baptiste/camembert-ner"]
        }

        # Se non specificato, scegli un modello predefinito
        if model_name is None:
            model_name = self._select_best_model()

        self.model_name = model_name
        self.is_available = False
        self.ner_pipeline = None

        # Inizializza il modello
        self._load_model()

    def _select_best_model(self) -> str:
        """
        Seleziona il miglior modello disponibile per la lingua.

        Returns:
            Nome del modello
        """
        models = self.language_models.get(self.language, self.language_models["en"])

        for model in models:
            try:
                # Prova a caricare il modello
                AutoTokenizer.from_pretrained(model)
                return model
            except Exception:
                continue

        # Fallback
        return "dslim/bert-base-NER"

    def _load_model(self):
        """
        Carica il modello Transformer per il riconoscimento di entità.
        """
        try:
            # Determina il device
            device = 0 if self.use_gpu and torch.cuda.is_available() else -1

            # Carica tokenizer e modello
            tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            model = AutoModelForTokenClassification.from_pretrained(self.model_name)

            # Crea pipeline NER
            self.ner_pipeline = pipeline(
                "ner",
                model=model,
                tokenizer=tokenizer,
                device=device
            )

            self.is_available = True
            print(f"Modello Transformer NER caricato: {self.model_name}")

        except Exception as e:
            print(f"Errore nel caricamento del modello Transformer NER: {e}")
            print("Assicurati di avere installato transformers e torch.")
            print("Prova a installare i modelli con:")
            print("pip install transformers torch")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando un modello Transformer.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.ner_pipeline:
            print("Riconoscitore Transformer NER non disponibile.")
            return []

        entities = []

        try:
            # Esegui NER
            ner_results = self.ner_pipeline(text)

            # Raggruppa entità consecutive con lo stesso tipo
            grouped_entities = []
            current_entity = None

            for result in ner_results:
                if result['entity'].startswith('B-'):
                    # Nuova entità
                    if current_entity:
                        grouped_entities.append(current_entity)

                    current_entity = {
                        'text': result['word'],
                        'label': result['entity'][2:],
                        'start_char': result['start'],
                        'end_char': result['end'],
                        'score': result['score']
                    }

                elif result['entity'].startswith('I-') and current_entity:
                    # Continua l'entità precedente
                    if result['entity'][2:] == current_entity['label']:
                        current_entity['text'] += result['word'].replace('##', '')
                        current_entity['end_char'] = result['end']
                        current_entity['score'] = max(current_entity['score'], result['score'])

                else:
                    # Caso di fallback
                    if current_entity:
                        grouped_entities.append(current_entity)
                        current_entity = None

            # Aggiungi ultima entità
            if current_entity:
                grouped_entities.append(current_entity)

            # Aggiungi source
            for entity in grouped_entities:
                entity['source'] = 'transformers_ner'
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Transformer NER: {e}")

        return entities

    def set_language(self, language: str):
        """
        Cambia la lingua del riconoscitore.

        Args:
            language: Nuova lingua da impostare
        """
        if language != self.language:
            self.language = language
            self._load_model()

    def get_supported_languages(self) -> List[str]:
        """
        Restituisce le lingue supportate.

        Returns:
            Lista di codici lingua supportati
        """
        return list(self.language_models.keys())



# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }



# Estendi la classe EntityLinkerWithSpacy per incorporare il riconoscitore italiano
class EntityLinkerItalian(EntityLinkerWithSpacy):
    """
    Versione estesa di EntityLinker ottimizzata per l'italiano in ambiente Colab.
    Integra riconoscitori multipli: Wikidata, spaCy, Stanza NLP e Transformer NER.
    """

    def __init__(self, language: str = "it", use_gpu: bool = False):
        """
        Inizializza il linker di entità con supporto NLP per l'italiano.

        Args:
            language: Lingua preferita per i risultati (deve essere "it")
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        # Controlla che la lingua sia italiana
        if language != "it":
            print("Attenzione: questo riconoscitore è ottimizzato per l'italiano. Impostiamo language='it'.")
            language = "it"

        # Inizializza la classe base (con spaCy)
        super().__init__(language)

        # Aggiungi il riconoscitore italiano Stanza NLP
        self.italian_recognizer = ItalianNLPRecognizer(use_gpu=use_gpu)

        # Aggiungi il riconoscitore Transformer NER
        try:
            from transformers import AutoTokenizer  # Verifica disponibilità
            self.transformers_recognizer = TransformersNERRecognizer(
                model_name="dbmdz/bert-base-italian-cased-ner",
                language=language,
                use_gpu=use_gpu
            )
        except ImportError:
            print("Transformer NER non disponibile. Installare 'transformers' e 'torch'.")
            self.transformers_recognizer = None

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando Wikidata, spaCy, Stanza NLP e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Prima ottieni i risultati dalla classe base (Wikidata + spaCy)
        enriched_entities, stats = super().find_entities(text)
        existing_texts = {e.get('text', '').lower() for e in enriched_entities if 'text' in e}

        # Aggiungi le entità del riconoscitore italiano se disponibile
        italian_only_entities = self._get_italian_only_entities(text, existing_texts)
        enriched_entities.extend(italian_only_entities)

        # Aggiungi le entità dal riconoscitore Transformer NER
        if self.transformers_recognizer and self.transformers_recognizer.is_available:
            transformers_only_entities = self._get_transformers_only_entities(text, existing_texts)
            enriched_entities.extend(transformers_only_entities)
            stats['transformers_only_entities'] = len(transformers_only_entities)

        # Aggiorna le statistiche
        stats['italian_only_entities'] = len(italian_only_entities)
        stats['total_entities'] = len(enriched_entities)

        return enriched_entities, stats

    def _get_italian_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore italiano e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo dal riconoscitore italiano
        """
        # Verifica che il riconoscitore italiano sia disponibile
        if not self.italian_recognizer.is_available:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore italiano
        italian_entities = self.italian_recognizer.recognize_entities(text)
        print(f"Riconoscitore italiano ha trovato {len(italian_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        italian_only = []
        for entity in italian_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore italiano: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                italian_type = self._map_italian_label_to_type(entity['label'])

                italian_entity = {
                    'text': entity_text,
                    'types': [italian_type],
                    'source': 'italian_nlp',
                    'label': entity['label']
                }

                italian_only.append(italian_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(italian_only)} sono uniche (non trovate da altre fonti)")
        return italian_only

    def _get_transformers_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore Transformer NER
        e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo da Transformer NER
        """
        # Verifica che il riconoscitore Transformer sia disponibile
        if not self.transformers_recognizer or not self.transformers_recognizer.is_available:
            print("Riconoscitore Transformer NER non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore Transformer NER
        transformers_entities = self.transformers_recognizer.recognize_entities(text)
        print(f"Riconoscitore Transformer NER ha trovato {len(transformers_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        transformers_only = []
        for entity in transformers_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore Transformer NER: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                transformers_type = self._map_transformers_label_to_type(entity['label'])

                transformers_entity = {
                    'text': entity_text,
                    'types': [transformers_type],
                    'source': 'transformers_ner',
                    'label': entity['label'],
                    'score': entity.get('score', 0)
                }

                transformers_only.append(transformers_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(transformers_only)} sono uniche (non trovate da altre fonti)")
        return transformers_only

    def _map_italian_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore italiano a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        mapping = {
            'PERSON': 'Person',
            'PER': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'MISC': 'Miscellaneous',
            'PROPN': 'ProperNoun',
            'WORK_OF_ART': 'Artwork',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def _map_transformers_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore Transformer NER a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        # (simile a quella di altri riconoscitori)
        mapping = {
            'PER': 'Person',
            'LOC': 'Location',
            'ORG': 'Organization',
            'MISC': 'Miscellaneous',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'PERSON': 'Person',
            'ORGANIZATION': 'Organization',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estende il metodo della classe base per includere i risultati
        del riconoscitore italiano e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        # Ottieni i risultati di base
        uri_results = super().verify_uris_for_text(text)

        # Aggiungi una sezione per le entità del riconoscitore italiano senza URI
        italian_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'italian_nlp' and entity['text'] not in uri_results['results']:
                italian_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        # Aggiungi una sezione per le entità del riconoscitore Transformer NER senza URI
        transformers_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'transformers_ner' and entity['text'] not in uri_results['results']:
                transformers_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', []),
                    'score': entity.get('score', 0)
                }

        # Aggiungi le sezioni ai risultati
        uri_results['italian_results'] = italian_results
        uri_results['transformers_results'] = transformers_results

        return uri_results


# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

if __name__ == "__main__":
    # Crea un'istanza del linker di entità migliorato
#    linker = EntityLinkerWithSpacy(language="it")
    linker = EntityLinkerItalian(language="it", use_gpu=False)


    # Testo di esempio esteso - usa lo stesso testo che stavi analizzando prima
    text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
           " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
           " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
           " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
           " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
           " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
           " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
           " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
           " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
           " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

    print(f"Analisi del testo:\n{text}\n")

    # Trova entità
    entities, stats = linker.find_entities(text)

    print(f"\nTrovate {stats['total_entities']} entità:")
    for entity in entities:
        source = entity.get('source', 'sconosciuta')
        uri = entity.get('dbpedia_uri', 'N/A')
        types = ', '.join(entity.get('types', ['Sconosciuto']))

        print(f"- '{entity['text']}' → {uri}")
        print(f"  Tipo: {types}")
        if 'wikidata_id' in entity:
            print(f"  Wikidata: {entity.get('wikidata_url', 'N/A')}")
        print(f"  Fonte: {source}")
        print("")

    # Verifica URI
    uri_results = linker.verify_uris_for_text(text)

    print("\nVerifica URI:")
    for word, info in uri_results['results'].items():
        print(f"- La parola '{word}' → URI: {info['uri']} esiste")

    # Mostra entità spaCy senza URI
    if 'spacy_results' in uri_results and uri_results['spacy_results']:
        print("\nEntità riconosciute solo da spaCy (senza URI):")
        for word, info in uri_results['spacy_results'].items():
            print(f"- '{word}' → Etichetta: {info.get('label', '')}, Tipi: {', '.join(info.get('types', []))}")

    # Mostra solo le entità con URI DBpedia
    print("\nEntità con URI DBpedia:")
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    for entity in entities_with_dbpedia:
        print(f"- '{entity['text']}' → {entity['dbpedia_uri']}")

    # Esporta le entità
    output_file_path = export_entities_to_txt(
    entities,
    stats,
    "entita.txt",
    spacy_recognizer=linker.wikidata.spacy_recognizer,
    italian_recognizer=linker.italian_recognizer,
    transformers_recognizer=linker.transformers_recognizer,  # Aggiungi questo
    original_text=text
    )

    print(f"\nFile di output salvato in: {output_file_path}")

    print("\nStatistiche:")
    for key, value in stats.items():
        print(f"- {key}: {value}")

    # Verifica aggiuntiva per il file
    import os
    if os.path.exists("entita.txt"):
        file_size = os.path.getsize("entita.txt")
        print(f"\nVerifica file: 'entita.txt' esiste e ha dimensione {file_size} bytes")
    else:
        print("\nATTENZIONE: Il file 'entita.txt' non è stato creato!")
        # Prova a identificare il problema
        try:
            with open("test_write_permission.txt", "w") as f:
                f.write("Test")
            print("La directory è scrivibile, il problema è altrove")
            os.remove("test_write_permission.txt")
        except Exception as e:
            print(f"Errore nell'accesso alla directory: {e}")
            print("Prova a specificare un percorso completo per il file di output")
            # Tenta con percorso nella home directory dell'utente
            home_dir = os.path.expanduser("~")
            output_path = os.path.join(home_dir, "entita.txt")
            try:
                export_entities_to_txt(entities, stats, output_path)
                print(f"File salvato con successo in: {output_path}")
            except Exception as e2:
                print(f"Errore anche con percorso alternativo: {e2}")

Modello spaCy it_core_news_lg caricato con successo.


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: it (Italian) ...
INFO:stanza:File exists: /root/stanza_resources/it/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| ner       | fbk               |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Riconoscitore NLP italiano inizializzato correttamente con Stanza.
Errore nel caricamento del modello Transformer NER: dbmdz/bert-base-italian-cased-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Assicurati di avere installato transformers e torch.
Prova a installare i modelli con:
pip install transformers torch
Analisi del testo:
L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico. Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento. Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio. Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export. Le

In [ ]:
# Verifica caricamento del modello
recognizer = TransformersNERRecognizer(language="it", use_gpu=False)

# Testo di test più esplicito
test_text = "Mario Rossi lavora per la Banca d'Italia a Roma e ha fondato una startup tecnologica a Milano."
entities = recognizer.recognize_entities(test_text)

print("Entità trovate:")
for entity in entities:
    print(f"- {entity['text']} (Tipo: {entity['label']}, Punteggio: {entity['score']})")

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Modello Transformer NER caricato: dslim/bert-base-NER
Entità trovate:
- MarioRossi (Tipo: PER, Punteggio: 0.9997310042381287)
- Roma (Tipo: LOC, Punteggio: 0.9841041564941406)
- Milano (Tipo: LOC, Punteggio: 0.9881179928779602)


In [ ]:
!pip install transformers[torch] tokenizers==0.13.3

INFO: pip is looking at multiple versions of transformers[torch] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.0 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers[torch] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict, Any, Optional, Tuple, Set # Added missing Any type hint
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch


# Importazione di spaCy
import spacy
# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy
# Modifica alla funzione export_entities_to_txt per includere le entità del riconoscitore italiano
#def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, italian_recognizer=None, original_text=None):

def export_entities_to_txt(
    entities,
    stats,
    output_file="entita.txt",
    spacy_recognizer=None,
    italian_recognizer=None,
    transformers_recognizer=None,
    original_text=None
    ):
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Include anche le entità riconosciute da spaCy, dal riconoscitore italiano e dal riconoscitore Transformer NER.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy (opzionale)
        italian_recognizer: Riconoscitore italiano (opzionale)
        transformers_recognizer: Riconoscitore Transformer NER (opzionale)
        original_text: Testo originale da analizzare (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]
    spacy_entities = [e for e in entities if e.get('source') == 'spacy']
    italian_entities = [e for e in entities if e.get('source') in ['italian_nlp', 'stanza', 'stanza_pos', 'geo_rule']]
    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']  # Assicurati che questa riga non sia commentata

    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in entities:  # Iterate through all entities
        if entity.get('source') in ['wikidata', 'wikidata+spacy']:  # Check if the entity is from Wikidata
            lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")  # Access Wikidata-specific attributes only if present
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")

    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_all_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_all_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback alle entità spaCy già filtrate
        if not spacy_entities:
            lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
        else:
            for entity in spacy_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore italiano
    lines.append("ITALIAN NLP ENTITIES:")
    lines.append("--------------------")

    # Se abbiamo un riconoscitore italiano e un testo originale, ottieni tutte le entità
    if italian_recognizer and original_text:
        try:
            italian_all_entities = italian_recognizer.recognize_entities(original_text)

            if not italian_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore italiano nel testo.")
            else:
                for entity in italian_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore italiano: {e}")
    else:
        # Fallback alle entità italiane già filtrate
        if not italian_entities:
            lines.append("Per mostrare tutte le entità del riconoscitore italiano, è necessario passare il riconoscitore e il testo originale.")
        else:
            for entity in italian_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore Transformer NER
    lines.append("TRANSFORMER NER ENTITIES:")
    lines.append("-------------------------")

    # Se abbiamo un riconoscitore Transformer e un testo originale, ottieni tutte le entità
    if transformers_recognizer and original_text and hasattr(transformers_recognizer, 'recognize_entities'):
        try:
            print("DEBUG: Tentativo di ottenere entità dal riconoscitore Transformer")
            transformers_all_entities = transformers_recognizer.recognize_entities(original_text)

            if not transformers_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore Transformer NER.")
            else:
                print(f"DEBUG: Trovate {len(transformers_all_entities)} entità da Transformer")
                # Rimuovi duplicati mantenendo l'ordine originale
                unique_transformers_entities = []
                seen = set()
                for entity in transformers_all_entities:
                    if entity['text'] not in seen:
                        unique_transformers_entities.append(entity)
                        seen.add(entity['text'])

                for entity in unique_transformers_entities:
                    # Usa il primo tipo disponibile o 'N/A'
                    label = entity.get('label', entity.get('types', ['N/A'])[0] if entity.get('types') else 'N/A')
                    score = entity.get('score', 'N/A')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}, Punteggio: {score}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore Transformer: {e}")
            print(f"DEBUG: Errore nell'ottenere entità Transformer: {e}")
    else:
        # Fallback alle entità Transformer già filtrate nel metodo find_entities
        if not transformers_entities:
            if not transformers_recognizer:
                lines.append("Riconoscitore Transformer NER non disponibile.")
            elif not hasattr(transformers_recognizer, 'recognize_entities'):
                lines.append("Il riconoscitore Transformer NER non ha il metodo recognize_entities.")
            else:
                lines.append("Per mostrare tutte le entità del riconoscitore Transformer, è necessario passare il riconoscitore e il testo originale.")
        else:
            print(f"DEBUG: Usando {len(transformers_entities)} entità Transformer già filtrate")
            for entity in transformers_entities:
                label = entity.get('label', 'N/A')
                score = entity.get('score', 'N/A')
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Punteggio: {score}")
    lines.append("")

    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities


class ItalianNLPRecognizer:
    """
    Riconoscitore di entità ottimizzato per l'italiano in ambiente Colab.
    Usa una combinazione di Stanza (versione Python di Stanford CoreNLP)
    e altre librerie per l'italiano.
    """

    def __init__(self, use_gpu: bool = False):
        """
        Inizializza il riconoscitore per l'italiano.

        Args:
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        self.use_gpu = use_gpu
        self.is_available = False
        self.stanza_nlp = None
        self.initialized = False

        # Inizializza le risorse
        self._initialize_resources()

    def _initialize_resources(self):
        """Inizializza Stanza e altre risorse per l'italiano."""
        try:
            import stanza
            import simplemma
            import nltk
            from nltk.tokenize import word_tokenize

            # Scarica i modelli di Stanza per l'italiano
            stanza.download('it')

            # Inizializza il pipeline Stanza per l'italiano
            self.stanza_nlp = stanza.Pipeline(
                lang='it',
                processors='tokenize,mwt,pos,lemma,ner',
                use_gpu=self.use_gpu
            )

            # Scarica le risorse NLTK necessarie
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('averaged_perceptron_tagger')

            self.is_available = True
            self.initialized = True
            print("Riconoscitore NLP italiano inizializzato correttamente con Stanza.")

        except Exception as e:
            print(f"Errore nell'inizializzazione del riconoscitore italiano: {e}")
            print("Assicurati di aver eseguito le installazioni necessarie in Colab.")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo italiano utilizzando Stanza.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.stanza_nlp:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con Stanza
            doc = self.stanza_nlp(text)

            # Estrai le entità
            for sent in doc.sentences:
                for ent in sent.ents:
                    entity = {
                        'text': ent.text,
                        'label': ent.type,
                        'start_char': -1,  # Stanza non fornisce direttamente le posizioni dei caratteri
                        'end_char': -1,
                        'source': 'stanza'
                    }
                    entities.append(entity)

                # Aggiungi anche i nomi propri (POS tag == PROPN)
                for token in sent.tokens:
                    for word in token.words:
                        if word.upos == 'PROPN' and not any(e['text'] == word.text for e in entities):
                            entity = {
                                'text': word.text,
                                'label': 'PROPN',
                                'start_char': -1,
                                'end_char': -1,
                                'source': 'stanza_pos'
                            }
                            entities.append(entity)

            # Cerca anche entità geografiche italiane specifiche usando regole
            entities.extend(self._extract_italian_geo_entities(text))

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Stanza: {e}")

        return entities

    def _extract_italian_geo_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Estrae entità geografiche italiane usando regole.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità geografiche
        """
        import re
        entities = []

        # Regioni italiane
        regions = [
            "Abruzzo", "Basilicata", "Calabria", "Campania", "Emilia-Romagna",
            "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardia", "Marche",
            "Molise", "Piemonte", "Puglia", "Sardegna", "Sicilia", "Toscana",
            "Trentino-Alto Adige", "Umbria", "Valle d'Aosta", "Veneto"
        ]

        # Pattern per laghi, montagne, fiumi italiani
        geo_patterns = [
            r'\b(?:Lago|Monte|Fiume|Isola|Golfo|Capo|Valle|Parco Nazionale|Mar) [A-Z][a-zàèéìòù]+\b',
            r'\b(?:Alpi|Appennini|Dolomiti|Maremma|Laguna|Costa|Riviera) [A-Z][a-zàèéìòù]*\b'
        ]

        # Cerca regioni
        for region in regions:
            if re.search(r'\b' + re.escape(region) + r'\b', text):
                entity = {
                    'text': region,
                    'label': 'GPE',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        # Cerca pattern geografici
        for pattern in geo_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                entity = {
                    'text': match.group(0),
                    'label': 'LOC',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        return entities


# Alternative per modelli italiani NER
model_options = [
    "5had3/bert-base-italian-cased-ner",  # Un'alternativa affidabile
    "MilaNLProc/bert-italian-cased-ner",  # Altro modello italiano per NER
    "dslim/bert-base-NER"  # Modello generico che funziona bene
]


class TransformersNERRecognizer:
    def __init__(self,
                 model_name: str = None,
                 language: str = "it",
                 use_gpu: bool = False):
        """
        Inizializza il riconoscitore di entità con un modello Transformer.

        Args:
            model_name: Nome del modello pre-addestrato per NER
            language: Lingua del modello (default: italiano)
            use_gpu: Se utilizzare l'accelerazione GPU
        """
        self.language = language
        self.use_gpu = use_gpu

        # Dizionario di mapping per modelli in diverse lingue
        self.language_models = {
            "it": [
                "5had3/bert-base-italian-cased-ner",
                "MilaNLProc/bert-italian-cased-ner",
                "dslim/bert-base-NER"
            ],
            "en": ["dslim/bert-base-NER"],
            "de": ["RafLorem/bert-base-german-NER"],
            "fr": ["Jean-Baptiste/camembert-ner"]
        }

        # Lista di fallback globale (modelli che funzionano in molte lingue)
        self.fallback_models = [
            "dslim/bert-base-NER",               # Ottimo supporto multilingua
            "dbmdz/bert-large-cased-finetuned-conll03-english",  # Molto robusto
            "Babelscape/wikineural-multilingual-ner"  # Supporto multilingua avanzato
        ]

        # Se non specificato, scegli un modello predefinito
        if model_name is None:
            model_name = self._select_best_model()

        self.model_name = model_name
        self.is_available = False
        self.ner_pipeline = None

        # Inizializza il modello
        self._load_model()

    def _select_best_model(self) -> str:
        """
        Seleziona il miglior modello disponibile per la lingua.

        Returns:
            Nome del modello
        """
        print(f"DEBUG: Selezionando modello per lingua: {self.language}")

        # Prova prima i modelli specifici per la lingua
        models = self.language_models.get(self.language, self.language_models.get("en", []))

        for model in models:
            try:
                # Importa solo il tokenizer per verificare la disponibilità
                from transformers import AutoTokenizer
                print(f"DEBUG: Verifica disponibilità modello: {model}")

                # Verifica la disponibilità con caricamento offline
                tokenizer = AutoTokenizer.from_pretrained(model, local_files_only=False)
                print(f"DEBUG: Modello {model} disponibile e selezionato")
                return model
            except Exception as e:
                print(f"DEBUG: Modello {model} non disponibile: {str(e)}")
                continue

        # Se nessun modello specifico per lingua funziona, prova i fallback
        print("DEBUG: Nessun modello specifico per lingua disponibile, provo fallback globali")
        for fallback in self.fallback_models:
            try:
                from transformers import AutoTokenizer
                print(f"DEBUG: Provo fallback modello: {fallback}")
                tokenizer = AutoTokenizer.from_pretrained(fallback, local_files_only=False)
                print(f"DEBUG: Fallback {fallback} disponibile e selezionato")
                return fallback
            except Exception as e:
                print(f"DEBUG: Fallback {fallback} non disponibile: {str(e)}")
                continue

        # Ultimo tentativo: dslim/bert-base-NER (molto comune)
        print("DEBUG: Ultimo tentativo con dslim/bert-base-NER")
        return "dslim/bert-base-NER"

    def _load_model(self):
        """
        Carica il modello Transformer per il riconoscimento di entità.
        """
        try:
            # Assicurati che le dipendenze siano disponibili
            import torch
            from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

            print(f"DEBUG: Tentativo di caricare il modello: {self.model_name}")

            # Determina il device
            device = 0 if self.use_gpu and torch.cuda.is_available() else -1
            print(f"DEBUG: Usando device {'GPU' if device == 0 else 'CPU'}")

            # Prova a caricare con questo modello specifico
            try:
                # Carica tokenizer e modello
                tokenizer = AutoTokenizer.from_pretrained(self.model_name)
                model = AutoModelForTokenClassification.from_pretrained(self.model_name)

                # Crea pipeline NER
                self.ner_pipeline = pipeline(
                    "ner",
                    model=model,
                    tokenizer=tokenizer,
                    device=device
                )

                self.is_available = True
                print(f"DEBUG: Modello {self.model_name} caricato con successo!")

                # Test rapido per verificare il funzionamento
                test_text = "Mario Rossi vive a Roma."
                test_result = self.ner_pipeline(test_text)
                print(f"DEBUG: Test modello superato - entità trovate: {len(test_result)}")
                return

            except Exception as model_error:
                print(f"DEBUG: Errore nel caricamento del modello specifico: {str(model_error)}")

                # Se il modello specifico fallisce, prova i modelli di fallback dalla lista
                for fallback in self.fallback_models:
                    if fallback != self.model_name:  # Evita di riprovare lo stesso modello
                        try:
                            print(f"DEBUG: Provo modello di fallback: {fallback}")
                            tokenizer = AutoTokenizer.from_pretrained(fallback)
                            model = AutoModelForTokenClassification.from_pretrained(fallback)

                            self.ner_pipeline = pipeline(
                                "ner",
                                model=model,
                                tokenizer=tokenizer,
                                device=device
                            )

                            self.is_available = True
                            self.model_name = fallback  # Aggiorna il nome del modello
                            print(f"DEBUG: Modello fallback {fallback} caricato con successo!")
                            return

                        except Exception as fallback_error:
                            print(f"DEBUG: Fallback {fallback} fallito: {str(fallback_error)}")

                # Se tutti i fallback falliscono, solleva l'errore originale
                raise model_error

        except ImportError as e:
            print(f"DEBUG: Errore di importazione: {str(e)}")
            print("È necessario installare 'transformers' e 'torch':")
            print("pip install transformers torch")
            self.is_available = False

        except Exception as e:
            print(f"DEBUG: Errore generale nel caricamento del modello: {str(e)}")
            import traceback
            traceback.print_exc()
            self.is_available = False

    def recognize_entities(self, text: str) -> list:
        """
        Riconosce entità in un testo utilizzando un modello Transformer.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.ner_pipeline:
            print("DEBUG: Riconoscitore Transformer NER non disponibile.")
            return []

        entities = []

        try:
            print(f"DEBUG: Analisi testo con modello {self.model_name}")

            # Esegui NER con gestione di testi lunghi
            # Alcuni modelli hanno limiti sulla lunghezza del testo
            if len(text) > 500:
                # Dividi il testo in blocchi di 500 caratteri con sovrapposizione
                chunks = []
                chunk_size = 450  # Usa 450 per avere sovrapposizione
                for i in range(0, len(text), chunk_size):
                    # Assicurati che l'ultimo chunk non vada oltre la fine del testo
                    end = min(i + chunk_size + 50, len(text))  # +50 per sovrapposizione
                    chunks.append(text[i:end])

                print(f"DEBUG: Testo diviso in {len(chunks)} chunks per l'analisi")

                # Analizza ogni chunk separatamente
                all_results = []
                for i, chunk in enumerate(chunks):
                    print(f"DEBUG: Analisi chunk {i+1}/{len(chunks)}")
                    chunk_results = self.ner_pipeline(chunk)
                    # Aggiungi offset per i chunk successivi al primo
                    if i > 0:
                        offset = i * chunk_size
                        for result in chunk_results:
                            result['start'] += offset
                            result['end'] += offset
                    all_results.extend(chunk_results)

                ner_results = all_results
            else:
                # Per testi brevi, usa direttamente il pipeline
                ner_results = self.ner_pipeline(text)

            print(f"DEBUG: Trovati {len(ner_results)} token di entità")

            # Raggruppa entità consecutive con lo stesso tipo
            grouped_entities = []
            current_entity = None

            for result in ner_results:
                if result['entity'].startswith('B-'):
                    # Nuova entità
                    if current_entity:
                        grouped_entities.append(current_entity)

                    current_entity = {
                        'text': result['word'],
                        'label': result['entity'][2:],  # Rimuovi il prefisso "B-"
                        'start_char': result['start'],
                        'end_char': result['end'],
                        'score': result['score']
                    }

                elif result['entity'].startswith('I-') and current_entity:
                    # Continua l'entità precedente se il tipo corrisponde
                    if result['entity'][2:] == current_entity['label']:
                        # Gestione speciale per i token tokenizzati che iniziano con ##
                        if result['word'].startswith('##'):
                            current_entity['text'] += result['word'][2:]  # Rimuovi ##
                        else:
                            # Potrebbe essere necessario aggiungere uno spazio
                            space_needed = not (result['start'] == current_entity['end_char'])
                            if space_needed:
                                current_entity['text'] += ' ' + result['word']
                            else:
                                current_entity['text'] += result['word']

                        current_entity['end_char'] = result['end']
                        # Aggiorna lo score con la media o il massimo
                        current_entity['score'] = max(current_entity['score'], result['score'])

                elif result['entity'] != 'O':  # Altra etichetta (non "Other")
                    # Gestisci entità che non hanno markup B-/I-
                    if current_entity:
                        grouped_entities.append(current_entity)

                    # Crea una nuova entità
                    current_entity = {
                        'text': result['word'],
                        'label': result['entity'].replace('B-', '').replace('I-', ''),
                        'start_char': result['start'],
                        'end_char': result['end'],
                        'score': result['score']
                    }

                else:  # Token classificato come "O" (Other)
                    # Finalizza l'entità corrente, se esiste
                    if current_entity:
                        grouped_entities.append(current_entity)
                        current_entity = None

            # Aggiungi l'ultima entità in elaborazione
            if current_entity:
                grouped_entities.append(current_entity)

            print(f"DEBUG: Raggruppate in {len(grouped_entities)} entità")

            # Aggiungi source e crea lista finale
            for entity in grouped_entities:
                # Pulizia finale del testo (rimuovi spazi extra)
                entity['text'] = entity['text'].strip()
                # Aggiungi la fonte
                entity['source'] = 'transformers_ner'
                entities.append(entity)

            # Aggiungi post-processing alle entità
            print(f"DEBUG: Applicazione post-processing a {len(entities)} entità")
            entities = self.process_entities(entities, text)
            return entities

        except Exception as e:
            print(f"DEBUG: Errore nel riconoscimento entità: {str(e)}")
            import traceback
            traceback.print_exc()
            return []

    def set_language(self, language: str):
        """
        Cambia la lingua del riconoscitore.

        Args:
            language: Nuova lingua da impostare
        """
        if language != self.language:
            self.language = language
            self._load_model()

    def get_supported_languages(self) -> List[str]:
        """
        Restituisce le lingue supportate.

        Returns:
            Lista di codici lingua supportati
        """
        return list(self.language_models.keys())



# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }



# Estendi la classe EntityLinkerWithSpacy per incorporare il riconoscitore italiano
class EntityLinkerItalian(EntityLinkerWithSpacy):
    """
    Versione estesa di EntityLinker ottimizzata per l'italiano in ambiente Colab.
    Integra riconoscitori multipli: Wikidata, spaCy, Stanza NLP e Transformer NER.
    """

    def __init__(self, language: str = "it", use_gpu: bool = False):
        """
        Inizializza il linker di entità con supporto NLP per l'italiano.

        Args:
            language: Lingua preferita per i risultati (deve essere "it")
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        # Controlla che la lingua sia italiana
        if language != "it":
            print("Attenzione: questo riconoscitore è ottimizzato per l'italiano. Impostiamo language='it'.")
            language = "it"

        # Inizializza la classe base (con spaCy)
        super().__init__(language)

        # Aggiungi il riconoscitore italiano Stanza NLP
        self.italian_recognizer = ItalianNLPRecognizer(use_gpu=use_gpu)

        # Aggiungi il riconoscitore Transformer NER
        try:
            from transformers import AutoTokenizer, pipeline  # Verifica disponibilità di transformers
            import torch  # Verifica disponibilità di torch

            print("DEBUG: Moduli transformers e torch disponibili. Inizializzazione riconoscitore Transformer NER...")

            self.transformers_recognizer = TransformersNERRecognizer(
                model_name="5had3/bert-base-italian-cased-ner",
                language=language,
                use_gpu=use_gpu
            )

            # Verifica se il riconoscitore è stato inizializzato correttamente
            if not hasattr(self.transformers_recognizer, 'is_available') or not self.transformers_recognizer.is_available:
                print("DEBUG: Riconoscitore Transformer NER non disponibile dopo l'inizializzazione")
            else:
                print("DEBUG: Riconoscitore Transformer NER inizializzato con successo")

        except ImportError as e:
            print(f"DEBUG: ImportError - Transformer NER non disponibile. {e}")
            print("Installare 'transformers' e 'torch' con 'pip install transformers torch'")
            self.transformers_recognizer = None
        except Exception as e:
            print(f"DEBUG: Errore generico nell'inizializzazione del riconoscitore Transformer NER: {e}")
            import traceback
            traceback.print_exc()
            self.transformers_recognizer = None

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando Wikidata, spaCy, Stanza NLP e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Debug del riconoscitore Transformer
        print(f"DEBUG: Stato riconoscitore Transformer - Disponibile: {hasattr(self, 'transformers_recognizer')}")
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer:
            print(f"DEBUG: Riconoscitore Transformer - Is Available: {self.transformers_recognizer.is_available}")

        # Prima ottieni i risultati dalla classe base (Wikidata + spaCy)
        enriched_entities, stats = super().find_entities(text)
        existing_texts = {e.get('text', '').lower() for e in enriched_entities if 'text' in e}

        # Aggiungi le entità del riconoscitore italiano se disponibile
        italian_only_entities = self._get_italian_only_entities(text, existing_texts)
        enriched_entities.extend(italian_only_entities)

        # Aggiorna il set di testi esistenti dopo aver aggiunto le entità italiane
        for entity in italian_only_entities:
            if 'text' in entity:
                existing_texts.add(entity['text'].lower())

        # Aggiungi le entità dal riconoscitore Transformer NER
        transformers_only_entities = []
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer and self.transformers_recognizer.is_available:
            print("DEBUG: Tentativo di trovare entità Transformer NER")
            try:
                # Usa un metodo separato per ottenere le entità
                transformers_only_entities = self._get_transformers_only_entities(text, existing_texts)
                print(f"DEBUG: Entità Transformer NER trovate: {len(transformers_only_entities)}")

                # Aggiungi solo se ci sono entità
                if transformers_only_entities:
                    enriched_entities.extend(transformers_only_entities)
            except Exception as e:
                print(f"DEBUG: Errore nel trovare entità Transformer: {e}")
                import traceback
                traceback.print_exc()
        else:
            print("DEBUG: Riconoscitore Transformer NER non disponibile per find_entities")

        # Aggiorna le statistiche
        stats['italian_only_entities'] = len(italian_only_entities)
        stats['transformers_only_entities'] = len(transformers_only_entities)
        stats['total_entities'] = len(enriched_entities)

        return enriched_entities, stats

    def _get_italian_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore italiano e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo dal riconoscitore italiano
        """
        # Verifica che il riconoscitore italiano sia disponibile
        if not self.italian_recognizer.is_available:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore italiano
        italian_entities = self.italian_recognizer.recognize_entities(text)
        print(f"Riconoscitore italiano ha trovato {len(italian_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        italian_only = []
        for entity in italian_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore italiano: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                italian_type = self._map_italian_label_to_type(entity['label'])

                italian_entity = {
                    'text': entity_text,
                    'types': [italian_type],
                    'source': 'italian_nlp',
                    'label': entity['label']
                }

                italian_only.append(italian_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(italian_only)} sono uniche (non trovate da altre fonti)")
        return italian_only

    def _get_transformers_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
          """
          Ottiene entità riconosciute dal riconoscitore Transformer NER.
          Includendo sia entità nuove che quelle già trovate da altre fonti.
          """
          # Verifica che il riconoscitore Transformer sia disponibile
          if not self.transformers_recognizer or not self.transformers_recognizer.is_available:
              print("DEBUG: Riconoscitore Transformer NER non disponibile.")
              return []

          # Stampa informazioni sul riconoscitore
          print(f"DEBUG: Modello Transformer NER in uso: {self.transformers_recognizer.model_name}")

          try:
              # Ottieni tutte le entità dal riconoscitore Transformer NER
              transformers_entities = self.transformers_recognizer.recognize_entities(text)

              # Debug dettagliato delle entità
              print(f"DEBUG: Numero di entità Transformer NER trovate: {len(transformers_entities)}")

              # Filtra e mappa le entità - IMPORTANTE: non filtriamo per existing_texts
              # per assicurarci che tutte le entità Transformer siano incluse
              transformers_mapped = []
              for entity in transformers_entities:
                  if self._is_valid_entity(entity.get('text', '')):
                      # Aggiungi sempre il source 'transformers_ner'
                      mapped_entity = {
                          'text': entity.get('text', ''),
                          'label': entity.get('label', ''),
                          'types': [entity.get('label', 'N/A')],
                          'source': 'transformers_ner',
                          'score': entity.get('score', 0)
                      }
                      transformers_mapped.append(mapped_entity)

              print(f"DEBUG: Numero di entità Transformer NER mappate: {len(transformers_mapped)}")
              return transformers_mapped

          except Exception as e:
              print(f"DEBUG: Errore nell'ottenere entità Transformer: {e}")
              import traceback
              traceback.print_exc()
              return []

    def _map_italian_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore italiano a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        mapping = {
            'PERSON': 'Person',
            'PER': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'MISC': 'Miscellaneous',
            'PROPN': 'ProperNoun',
            'WORK_OF_ART': 'Artwork',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def _map_transformers_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore Transformer NER a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        # (simile a quella di altri riconoscitori)
        mapping = {
            'PER': 'Person',
            'LOC': 'Location',
            'ORG': 'Organization',
            'MISC': 'Miscellaneous',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'PERSON': 'Person',
            'ORGANIZATION': 'Organization',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estende il metodo della classe base per includere i risultati
        del riconoscitore italiano e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        # Ottieni i risultati di base
        uri_results = super().verify_uris_for_text(text)

        # Aggiungi una sezione per le entità del riconoscitore italiano senza URI
        italian_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'italian_nlp' and entity['text'] not in uri_results['results']:
                italian_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        # Aggiungi una sezione per le entità del riconoscitore Transformer NER senza URI
        transformers_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'transformers_ner' and entity['text'] not in uri_results['results']:
                transformers_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', []),
                    'score': entity.get('score', 0)
                }

        # Aggiungi le sezioni ai risultati
        uri_results['italian_results'] = italian_results
        uri_results['transformers_results'] = transformers_results

        return uri_results


# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

if __name__ == "__main__":
    # Importazioni necessarie
    import os
    import sys
    import traceback

    # Imposta il livello di log per transformers
    os.environ["TRANSFORMERS_VERBOSITY"] = "info"

    # Verifica le dipendenze
    try:
        import torch
        import transformers
        print(f"PyTorch versione: {torch.__version__}")
        print(f"Transformers versione: {transformers.__version__}")
        print(f"CUDA disponibile: {torch.cuda.is_available()}")
    except ImportError as e:
        print(f"ATTENZIONE: Libreria mancante - {e}")

    try:
        # Crea un'istanza del linker di entità migliorato con debug aggiuntivo
        print("Inizializzazione EntityLinkerItalian...")
        linker = EntityLinkerItalian(language="it", use_gpu=False)

        # Verifica disponibilità dei riconoscitori
        print(f"SpaCy riconoscitore disponibile: {linker.wikidata.spacy_recognizer.is_available}")
        print(f"Italian NLP riconoscitore disponibile: {linker.italian_recognizer.is_available}")
        if hasattr(linker, 'transformers_recognizer') and linker.transformers_recognizer:
            print(f"Transformer NER riconoscitore disponibile: {linker.transformers_recognizer.is_available}")
            print(f"Transformer NER modello: {linker.transformers_recognizer.model_name}")
        else:
            print("Transformer NER riconoscitore non disponibile")

        # Testo di esempio
        text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
               " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
               " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
               " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
               " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
               " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
               " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
               " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
               " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
               " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

        print(f"Analisi del testo:\n{text}\n")

        # Test esplicito del riconoscitore Transformer NER, se disponibile
        if hasattr(linker, 'transformers_recognizer') and linker.transformers_recognizer and linker.transformers_recognizer.is_available:
            print("\nTest diretto del riconoscitore Transformer NER:")
            test_sample = "Mario Rossi vive a Roma e lavora per l'Università La Sapienza."
            try:
                transformer_test_entities = linker.transformers_recognizer.recognize_entities(test_sample)
                print(f"Entità trovate dal riconoscitore Transformer in test: {len(transformer_test_entities)}")
                for entity in transformer_test_entities:
                    print(f"- '{entity['text']}' → Etichetta: {entity.get('label')}, Score: {entity.get('score')}")
            except Exception as e:
                print(f"Errore nel test diretto del riconoscitore Transformer: {e}")
                traceback.print_exc()

        # Trova entità
        print("\nRicerca entità nel testo principale...")
        entities, stats = linker.find_entities(text)

        # Verifica entità Transformer
        transformer_entities = [e for e in entities if e.get('source') == 'transformers_ner']
        print(f"\nEntità Transformer NER trovate: {len(transformer_entities)}")
        for entity in transformer_entities[:10]:  # Mostra solo le prime 10 per brevità
            print(f"- '{entity['text']}' → Etichetta: {entity.get('label')}, Score: {entity.get('score', 'N/A')}")

        print(f"\nTrovate {stats['total_entities']} entità totali:")
        for k, v in stats.items():
            print(f"- {k}: {v}")

        # Esporta le entità in un file
        print("\nEsportazione entità in file...")
        output_file_path = export_entities_to_txt(
            entities,
            stats,
            "entita_complete.txt",  # Nuovo nome file per distinguerlo dal precedente
            spacy_recognizer=linker.wikidata.spacy_recognizer,
            italian_recognizer=linker.italian_recognizer,
            transformers_recognizer=linker.transformers_recognizer,
            original_text=text
        )

        print(f"\nFile di output salvato in: {output_file_path}")

        # Verifica il file di output
        import os
        if os.path.exists(output_file_path):
            file_size = os.path.getsize(output_file_path)
            print(f"Verifica file: '{output_file_path}' esiste e ha dimensione {file_size} bytes")

            # Leggi il file per verificare il contenuto
            with open(output_file_path, 'r', encoding='utf-8') as f:
                content = f.read()

            # Cerca la sezione Transformer nel file
            if "TRANSFORMER NER ENTITIES:" in content:
                transformer_section = content.split("TRANSFORMER NER ENTITIES:")[1].split("\n\n")[0]
                transformer_lines = [line for line in transformer_section.split("\n") if line.strip() and "-----" not in line]
                print(f"\nSezione Transformer nel file di output contiene {len(transformer_lines)} righe")
                if len(transformer_lines) > 1:  # La prima riga è l'intestazione
                    print("Le prime 5 entità Transformer nel file:")
                    for line in transformer_lines[:6]:
                        print(f"  {line}")
                else:
                    print("ATTENZIONE: Nessuna entità Transformer trovata nel file di output")
            else:
                print("ATTENZIONE: Sezione Transformer non trovata nel file di output")
        else:
            print(f"ATTENZIONE: Il file '{output_file_path}' non è stato creato!")

    except Exception as e:
        print(f"Errore generale: {e}")
        traceback.print_exc()

In [5]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict, Any, Optional, Tuple, Set # Added missing Any type hint
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch


# Importazione di spaCy
import spacy
# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy
# Modifica alla funzione export_entities_to_txt per includere le entità del riconoscitore italiano
#def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, italian_recognizer=None, original_text=None):

def export_entities_to_txt(
    entities,
    stats,
    output_file="entita.txt",
    spacy_recognizer=None,
    italian_recognizer=None,
    transformers_recognizer=None,  # Nuovo parametro
    original_text=None
    ):
    # ... resto del codice invariato ...
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Include anche le entità riconosciute da spaCy e dal riconoscitore italiano.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy (opzionale)
        italian_recognizer: Riconoscitore italiano (opzionale)
        original_text: Testo originale da analizzare (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]
    spacy_entities = [e for e in entities if e.get('source') == 'spacy']
    italian_entities = [e for e in entities if e.get('source') in ['italian_nlp', 'stanza', 'stanza_pos', 'geo_rule']]
#    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']


    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in entities:  # Iterate through all entities
        if entity.get('source') in ['wikidata', 'wikidata+spacy']:  # Check if the entity is from Wikidata
            lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")  # Access Wikidata-specific attributes only if present
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")

    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_all_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_all_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback alle entità spaCy già filtrate
        if not spacy_entities:
            lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
        else:
            for entity in spacy_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore italiano
    lines.append("ITALIAN NLP ENTITIES:")
    lines.append("--------------------")

    # Se abbiamo un riconoscitore italiano e un testo originale, ottieni tutte le entità
    if italian_recognizer and original_text:
        try:
            italian_all_entities = italian_recognizer.recognize_entities(original_text)

            if not italian_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore italiano nel testo.")
            else:
                for entity in italian_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore italiano: {e}")
    else:
        # Fallback alle entità italiane già filtrate
        if not italian_entities:
            lines.append("Per mostrare tutte le entità del riconoscitore italiano, è necessario passare il riconoscitore e il testo originale.")
        else:
            for entity in italian_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")

    # Aggiungi questa sezione PRIMA delle ultime righe di scrittura del file
        # Sezione entità del riconoscitore Transformer NER
#        transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']
        """
        lines.append("\nTRANSFORMER NER ENTITIES:")
        lines.append("-------------------------")
        if transformers_entities:
            # Rimuovi duplicati mantenendo l'ordine originale
            unique_transformers_entities = []
            seen = set()
            for entity in transformers_entities:
                if entity['text'] not in seen:
                    unique_transformers_entities.append(entity)
                    seen.add(entity['text'])

            for entity in unique_transformers_entities:
                lines.append(f"- '{entity['text']}' → Etichetta: {entity.get('label', 'N/A')}, Punteggio: {entity.get('score', 'N/A')}")
        else:
            lines.append("Nessuna entità rilevata dal riconoscitore Transformer NER.")
        lines.append("")
        """

        # Aggiungi questa parte nella funzione export_entities_to_txt
        # Sezione entità del riconoscitore Transformer NER
        #transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']
        """

        lines.append("TRANSFORMER NER ENTITIES:")
        lines.append("-------------------------")

        if transformers_entities:
            for entity in transformers_entities:
                lines.append(f"- '{entity['text']}' → Etichetta: {entity.get('label', 'N/A')}, Punteggio: {entity.get('score', 'N/A')}")
        else:
            lines.append("Nessuna entità rilevata dal riconoscitore Transformer NER.")
        lines.append("")
        """

        # Sezione entità del riconoscitore Transformer NER
        transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']

        lines.append("TRANSFORMER NER ENTITIES:")
        lines.append("-------------------------")
        if transformers_entities:
            # Rimuovi duplicati mantenendo l'ordine originale
            unique_transformers_entities = []
            seen = set()
            for entity in transformers_entities:
                if entity['text'] not in seen:
                    unique_transformers_entities.append(entity)
                    seen.add(entity['text'])

            for entity in unique_transformers_entities:
                # Usa il primo tipo disponibile o 'N/A'
                label = entity.get('label', entity.get('types', ['N/A'])[0] if entity.get('types') else 'N/A')
                score = entity.get('score', 'N/A')
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Punteggio: {score}")
        else:
            lines.append("Nessuna entità rilevata dal riconoscitore Transformer NER.")
        lines.append("")



    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities


class ItalianNLPRecognizer:
    """
    Riconoscitore di entità ottimizzato per l'italiano in ambiente Colab.
    Usa una combinazione di Stanza (versione Python di Stanford CoreNLP)
    e altre librerie per l'italiano.
    """

    def __init__(self, use_gpu: bool = False):
        """
        Inizializza il riconoscitore per l'italiano.

        Args:
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        self.use_gpu = use_gpu
        self.is_available = False
        self.stanza_nlp = None
        self.initialized = False

        # Inizializza le risorse
        self._initialize_resources()

    def _initialize_resources(self):
        """Inizializza Stanza e altre risorse per l'italiano."""
        try:
            import stanza
            import simplemma
            import nltk
            from nltk.tokenize import word_tokenize

            # Scarica i modelli di Stanza per l'italiano
            stanza.download('it')

            # Inizializza il pipeline Stanza per l'italiano
            self.stanza_nlp = stanza.Pipeline(
                lang='it',
                processors='tokenize,mwt,pos,lemma,ner',
                use_gpu=self.use_gpu
            )

            # Scarica le risorse NLTK necessarie
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('averaged_perceptron_tagger')

            self.is_available = True
            self.initialized = True
            print("Riconoscitore NLP italiano inizializzato correttamente con Stanza.")

        except Exception as e:
            print(f"Errore nell'inizializzazione del riconoscitore italiano: {e}")
            print("Assicurati di aver eseguito le installazioni necessarie in Colab.")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo italiano utilizzando Stanza.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.stanza_nlp:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con Stanza
            doc = self.stanza_nlp(text)

            # Estrai le entità
            for sent in doc.sentences:
                for ent in sent.ents:
                    entity = {
                        'text': ent.text,
                        'label': ent.type,
                        'start_char': -1,  # Stanza non fornisce direttamente le posizioni dei caratteri
                        'end_char': -1,
                        'source': 'stanza'
                    }
                    entities.append(entity)

                # Aggiungi anche i nomi propri (POS tag == PROPN)
                for token in sent.tokens:
                    for word in token.words:
                        if word.upos == 'PROPN' and not any(e['text'] == word.text for e in entities):
                            entity = {
                                'text': word.text,
                                'label': 'PROPN',
                                'start_char': -1,
                                'end_char': -1,
                                'source': 'stanza_pos'
                            }
                            entities.append(entity)

            # Cerca anche entità geografiche italiane specifiche usando regole
            entities.extend(self._extract_italian_geo_entities(text))

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Stanza: {e}")

        return entities

    def _extract_italian_geo_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Estrae entità geografiche italiane usando regole.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità geografiche
        """
        import re
        entities = []

        # Regioni italiane
        regions = [
            "Abruzzo", "Basilicata", "Calabria", "Campania", "Emilia-Romagna",
            "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardia", "Marche",
            "Molise", "Piemonte", "Puglia", "Sardegna", "Sicilia", "Toscana",
            "Trentino-Alto Adige", "Umbria", "Valle d'Aosta", "Veneto"
        ]

        # Pattern per laghi, montagne, fiumi italiani
        geo_patterns = [
            r'\b(?:Lago|Monte|Fiume|Isola|Golfo|Capo|Valle|Parco Nazionale|Mar) [A-Z][a-zàèéìòù]+\b',
            r'\b(?:Alpi|Appennini|Dolomiti|Maremma|Laguna|Costa|Riviera) [A-Z][a-zàèéìòù]*\b'
        ]

        # Cerca regioni
        for region in regions:
            if re.search(r'\b' + re.escape(region) + r'\b', text):
                entity = {
                    'text': region,
                    'label': 'GPE',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        # Cerca pattern geografici
        for pattern in geo_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                entity = {
                    'text': match.group(0),
                    'label': 'LOC',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        return entities


# Alternative per modelli italiani NER
model_options = [
    "5had3/bert-base-italian-cased-ner",  # Un'alternativa affidabile
    "MilaNLProc/bert-italian-cased-ner",  # Altro modello italiano per NER
    "dslim/bert-base-NER"  # Modello generico che funziona bene
]


class TransformersNERRecognizer:
    def __init__(self,
                 model_name: str = None,
                 language: str = "it",
                 use_gpu: bool = False):
        """
        Inizializza il riconoscitore di entità con un modello Transformer.

        Args:
            model_name: Nome del modello pre-addestrato per NER
            language: Lingua del modello (default: italiano)
            use_gpu: Se utilizzare l'accelerazione GPU
        """
        self.language = language
        self.use_gpu = use_gpu

        # Dizionario di mapping per modelli in diverse lingue
        self.language_models = {
            "it": [
                "5had3/bert-base-italian-cased-ner",
                "MilaNLProc/bert-italian-cased-ner",
                "dslim/bert-base-NER"
            ],
            "en": ["dslim/bert-base-NER"],
            "de": ["RafLorem/bert-base-german-NER"],
            "fr": ["Jean-Baptiste/camembert-ner"]
        }

        # Se non specificato, scegli un modello predefinito
        if model_name is None:
            model_name = self._select_best_model()

        self.model_name = model_name
        self.is_available = False
        self.ner_pipeline = None

        # Inizializza il modello
        self._load_model()

    def _select_best_model(self) -> str:
        """
        Seleziona il miglior modello disponibile per la lingua.

        Returns:
            Nome del modello
        """
        models = self.language_models.get(self.language, self.language_models["en"])

        for model in models:
            try:
                # Prova a caricare il modello
                AutoTokenizer.from_pretrained(model)
                return model
            except Exception:
                continue

        # Fallback
        return "dslim/bert-base-NER"

    def _load_model(self):
        """
        Carica il modello Transformer per il riconoscimento di entità.
        """
        try:
            # Lista di modelli italiani alternativi
            alternative_models = [
                "5had3/bert-base-italian-cased-ner",
                "MilaNLProc/bert-italian-cased-ner",
                "Davlan/bert-base-multilingual-cased-ner-hrl"
            ]

            # Prova a caricare uno dei modelli
            for model_name in alternative_models:
                try:
                    # Determina il device
                    device = 0 if self.use_gpu and torch.cuda.is_available() else -1

                    # Carica tokenizer e modello
                    tokenizer = AutoTokenizer.from_pretrained(model_name)
                    model = AutoModelForTokenClassification.from_pretrained(model_name)

                    # Crea pipeline NER
                    self.ner_pipeline = pipeline(
                        "ner",
                        model=model,
                        tokenizer=tokenizer,
                        device=device
                    )

                    self.is_available = True
                    self.model_name = model_name
                    print(f"Modello Transformer NER caricato: {model_name}")
                    return

                except Exception as inner_e:
                    print(f"Tentativo fallito con modello {model_name}: {inner_e}")

            # Se nessun modello funziona
            raise Exception("Nessun modello NER disponibile")

        except Exception as e:
            print(f"Errore nel caricamento del modello Transformer NER: {e}")
            print("Assicurati di avere installato transformers e torch.")
            print("Prova a installare i modelli con:")
            print("pip install transformers torch")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando un modello Transformer.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.ner_pipeline:
            print("Riconoscitore Transformer NER non disponibile.")
            return []

        entities = []

        try:
            # Esegui NER
            ner_results = self.ner_pipeline(text)

            # Raggruppa entità consecutive con lo stesso tipo
            grouped_entities = []
            current_entity = None

            for result in ner_results:
                if result['entity'].startswith('B-'):
                    # Nuova entità
                    if current_entity:
                        grouped_entities.append(current_entity)

                    current_entity = {
                        'text': result['word'],
                        'label': result['entity'][2:],
                        'start_char': result['start'],
                        'end_char': result['end'],
                        'score': result['score']
                    }

                elif result['entity'].startswith('I-') and current_entity:
                    # Continua l'entità precedente
                    if result['entity'][2:] == current_entity['label']:
                        current_entity['text'] += result['word'].replace('##', '')
                        current_entity['end_char'] = result['end']
                        current_entity['score'] = max(current_entity['score'], result['score'])

                else:
                    # Caso di fallback
                    if current_entity:
                        grouped_entities.append(current_entity)
                        current_entity = None

            # Aggiungi ultima entità
            if current_entity:
                grouped_entities.append(current_entity)

            # Aggiungi source
            for entity in grouped_entities:
                entity['source'] = 'transformers_ner'
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Transformer NER: {e}")

        return entities

    def set_language(self, language: str):
        """
        Cambia la lingua del riconoscitore.

        Args:
            language: Nuova lingua da impostare
        """
        if language != self.language:
            self.language = language
            self._load_model()

    def get_supported_languages(self) -> List[str]:
        """
        Restituisce le lingue supportate.

        Returns:
            Lista di codici lingua supportati
        """
        return list(self.language_models.keys())



# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }



# Estendi la classe EntityLinkerWithSpacy per incorporare il riconoscitore italiano
class EntityLinkerItalian(EntityLinkerWithSpacy):
    """
    Versione estesa di EntityLinker ottimizzata per l'italiano in ambiente Colab.
    Integra riconoscitori multipli: Wikidata, spaCy, Stanza NLP e Transformer NER.
    """

    def __init__(self, language: str = "it", use_gpu: bool = False):
        """
        Inizializza il linker di entità con supporto NLP per l'italiano.

        Args:
            language: Lingua preferita per i risultati (deve essere "it")
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        # Controlla che la lingua sia italiana
        if language != "it":
            print("Attenzione: questo riconoscitore è ottimizzato per l'italiano. Impostiamo language='it'.")
            language = "it"

        # Inizializza la classe base (con spaCy)
        super().__init__(language)

        # Aggiungi il riconoscitore italiano Stanza NLP
        self.italian_recognizer = ItalianNLPRecognizer(use_gpu=use_gpu)

        # Aggiungi il riconoscitore Transformer NER
        try:
            from transformers import AutoTokenizer  # Verifica disponibilità
            self.transformers_recognizer = TransformersNERRecognizer(
                model_name="5had3/bert-base-italian-cased-ner",
                language=language,
                use_gpu=use_gpu
            )

            # Test di riconoscimento
            test_text = "Mario Rossi lavora per la Banca d'Italia a Roma."
            test_entities = self.transformers_recognizer.recognize_entities(test_text)
            print("DEBUG: Test riconoscitore Transformer NER:")
            for entity in test_entities:
                print(f"- {entity['text']} (Tipo: {entity.get('label', 'N/A')})")

        except ImportError:
            print("Transformer NER non disponibile. Installare 'transformers' e 'torch'.")
            self.transformers_recognizer = None

#    def __init__(self, language: str = "it", use_gpu: bool = False):
        # ... codice esistente ...

        # Test esplicito del riconoscitore Transformer NER
        try:
            from transformers import AutoTokenizer  # Verifica disponibilità
            self.transformers_recognizer = TransformersNERRecognizer(
                model_name="5had3/bert-base-italian-cased-ner",
                language=language,
                use_gpu=use_gpu
            )

            # Test di riconoscimento
            test_text = "Mario Rossi lavora per la Banca d'Italia a Roma."
            test_entities = self.transformers_recognizer.recognize_entities(test_text)
            print("DEBUG: Test riconoscitore Transformer NER:")
            for entity in test_entities:
                print(f"- {entity['text']} (Tipo: {entity.get('label', 'N/A')})")

        except ImportError:
            print("Transformer NER non disponibile. Installare 'transformers' e 'torch'.")
            self.transformers_recognizer = None

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando Wikidata, spaCy, Stanza NLP e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Debug del riconoscitore Transformer
        print(f"DEBUG: Stato riconoscitore Transformer - Disponibile: {hasattr(self, 'transformers_recognizer')}")
        if hasattr(self, 'transformers_recognizer'):
            print(f"DEBUG: Riconoscitore Transformer - Is Available: {self.transformers_recognizer.is_available}")

        # Prima ottieni i risultati dalla classe base (Wikidata + spaCy)
        enriched_entities, stats = super().find_entities(text)
        existing_texts = {e.get('text', '').lower() for e in enriched_entities if 'text' in e}

        # Aggiungi le entità del riconoscitore italiano se disponibile
        italian_only_entities = self._get_italian_only_entities(text, existing_texts)
        enriched_entities.extend(italian_only_entities)

        # Aggiungi le entità dal riconoscitore Transformer NER
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer and self.transformers_recognizer.is_available:
            print("DEBUG: Tentativo di trovare entità Transformer NER")
            transformers_only_entities = self._get_transformers_only_entities(text, existing_texts)
            print(f"DEBUG: Entità Transformer NER trovate: {len(transformers_only_entities)}")
            enriched_entities.extend(transformers_only_entities)
            stats['transformers_only_entities'] = len(transformers_only_entities)
        else:
            print("DEBUG: Riconoscitore Transformer NER non disponibile")

        # Aggiorna le statistiche
        stats['italian_only_entities'] = len(italian_only_entities)
        stats['total_entities'] = len(enriched_entities)

        return enriched_entities, stats

    def _get_italian_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore italiano e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo dal riconoscitore italiano
        """
        # Verifica che il riconoscitore italiano sia disponibile
        if not self.italian_recognizer.is_available:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore italiano
        italian_entities = self.italian_recognizer.recognize_entities(text)
        print(f"Riconoscitore italiano ha trovato {len(italian_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        italian_only = []
        for entity in italian_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore italiano: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                italian_type = self._map_italian_label_to_type(entity['label'])

                italian_entity = {
                    'text': entity_text,
                    'types': [italian_type],
                    'source': 'italian_nlp',
                    'label': entity['label']
                }

                italian_only.append(italian_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(italian_only)} sono uniche (non trovate da altre fonti)")
        return italian_only

    def _get_transformers_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore Transformer NER
        e non da altre fonti.
        """
        # Verifica che il riconoscitore Transformer sia disponibile
        if not self.transformers_recognizer or not self.transformers_recognizer.is_available:
            print("DEBUG: Riconoscitore Transformer NER non disponibile.")
            return []

        # Stampa informazioni sul riconoscitore
        print(f"DEBUG: Modello Transformer NER in uso: {self.transformers_recognizer.model_name}")

        # Ottieni tutte le entità dal riconoscitore Transformer NER
        transformers_entities = self.transformers_recognizer.recognize_entities(text)

        # Debug dettagliato delle entità
        print("DEBUG: Dettagli entità Transformer NER:")
        for entity in transformers_entities:
            print(f"- Testo: {entity.get('text')}")
            print(f"  Label: {entity.get('label')}")
            print(f"  Score: {entity.get('score')}")
            print(f"  Attributi disponibili: {list(entity.keys())}")

        # Filtra e mappa le entità
        transformers_only = []
        for entity in transformers_entities:
            # Aggiungi sempre il source 'transformers_ner'
            mapped_entity = {
                'text': entity.get('text', ''),
                'label': entity.get('label', ''),
                'types': [entity.get('label', 'N/A')],
                'source': 'transformers_ner',
                'score': entity.get('score', 0)
            }
            transformers_only.append(mapped_entity)

        print(f"DEBUG: Numero di entità Transformer NER mappate: {len(transformers_only)}")
        return transformers_only

    def _map_italian_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore italiano a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        mapping = {
            'PERSON': 'Person',
            'PER': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'MISC': 'Miscellaneous',
            'PROPN': 'ProperNoun',
            'WORK_OF_ART': 'Artwork',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def _map_transformers_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore Transformer NER a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        # (simile a quella di altri riconoscitori)
        mapping = {
            'PER': 'Person',
            'LOC': 'Location',
            'ORG': 'Organization',
            'MISC': 'Miscellaneous',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'PERSON': 'Person',
            'ORGANIZATION': 'Organization',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estende il metodo della classe base per includere i risultati
        del riconoscitore italiano e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        # Ottieni i risultati di base
        uri_results = super().verify_uris_for_text(text)

        # Aggiungi una sezione per le entità del riconoscitore italiano senza URI
        italian_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'italian_nlp' and entity['text'] not in uri_results['results']:
                italian_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        # Aggiungi una sezione per le entità del riconoscitore Transformer NER senza URI
        transformers_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'transformers_ner' and entity['text'] not in uri_results['results']:
                transformers_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', []),
                    'score': entity.get('score', 0)
                }

        # Aggiungi le sezioni ai risultati
        uri_results['italian_results'] = italian_results
        uri_results['transformers_results'] = transformers_results

        return uri_results


# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

if __name__ == "__main__":
    # Crea un'istanza del linker di entità migliorato
#    linker = EntityLinkerWithSpacy(language="it")
    linker = EntityLinkerItalian(language="it", use_gpu=False)


    # Testo di esempio esteso - usa lo stesso testo che stavi analizzando prima
    text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
           " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
           " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
           " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
           " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
           " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
           " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
           " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
           " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
           " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

    print(f"Analisi del testo:\n{text}\n")

    # Trova entità
    entities, stats = linker.find_entities(text)

    # Debug delle entità
    print("\nDEBUG: Entità trovate:")
    for entity in entities:
        print(f"- {entity.get('text')} → Fonte: {entity.get('source')} → Tipi: {entity.get('types')}")

    print(f"\nTrovate {stats['total_entities']} entità:")
    for entity in entities:
        source = entity.get('source', 'sconosciuta')
        uri = entity.get('dbpedia_uri', 'N/A')
        types = ', '.join(entity.get('types', ['Sconosciuto']))

        print(f"- '{entity['text']}' → {uri}")
        print(f"  Tipo: {types}")
        if 'wikidata_id' in entity:
            print(f"  Wikidata: {entity.get('wikidata_url', 'N/A')}")
        print(f"  Fonte: {source}")
        print("")

    # Verifica URI
    uri_results = linker.verify_uris_for_text(text)

    print("\nVerifica URI:")
    for word, info in uri_results['results'].items():
        print(f"- La parola '{word}' → URI: {info['uri']} esiste")

    # Mostra entità spaCy senza URI
    if 'spacy_results' in uri_results and uri_results['spacy_results']:
        print("\nEntità riconosciute solo da spaCy (senza URI):")
        for word, info in uri_results['spacy_results'].items():
            print(f"- '{word}' → Etichetta: {info.get('label', '')}, Tipi: {', '.join(info.get('types', []))}")

    # Mostra solo le entità con URI DBpedia
    print("\nEntità con URI DBpedia:")
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    for entity in entities_with_dbpedia:
        print(f"- '{entity['text']}' → {entity['dbpedia_uri']}")

    # Esporta le entità
    output_file_path = export_entities_to_txt(
    entities,
    stats,
    "entita.txt",
    spacy_recognizer=linker.wikidata.spacy_recognizer,
    italian_recognizer=linker.italian_recognizer,
    transformers_recognizer=linker.transformers_recognizer,  # Aggiungi questo
    original_text=text
    )

    print(f"\nFile di output salvato in: {output_file_path}")

    print("\nStatistiche:")
    for key, value in stats.items():
        print(f"- {key}: {value}")

    # Verifica aggiuntiva per il file
    import os
    if os.path.exists("entita.txt"):
        file_size = os.path.getsize("entita.txt")
        print(f"\nVerifica file: 'entita.txt' esiste e ha dimensione {file_size} bytes")
    else:
        print("\nATTENZIONE: Il file 'entita.txt' non è stato creato!")
        # Prova a identificare il problema
        try:
            with open("test_write_permission.txt", "w") as f:
                f.write("Test")
            print("La directory è scrivibile, il problema è altrove")
            os.remove("test_write_permission.txt")
        except Exception as e:
            print(f"Errore nell'accesso alla directory: {e}")
            print("Prova a specificare un percorso completo per il file di output")
            # Tenta con percorso nella home directory dell'utente
            home_dir = os.path.expanduser("~")
            output_path = os.path.join(home_dir, "entita.txt")
            try:
                export_entities_to_txt(entities, stats, output_path)
                print(f"File salvato con successo in: {output_path}")
            except Exception as e2:
                print(f"Errore anche con percorso alternativo: {e2}")

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


Modello spaCy it_core_news_lg caricato con successo.


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: it (Italian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/it/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| ner       | fbk               |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


...

config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

Modello Transformer NER caricato: Davlan/bert-base-multilingual-cased-ner-hrl
DEBUG: Test riconoscitore Transformer NER:
- MarioRossi (Tipo: PER)
- Bancad'Italia (Tipo: ORG)
- Roma (Tipo: LOC)
Tentativo fallito con modello 5had3/bert-base-italian-cased-ner: 5had3/bert-base-italian-cased-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Tentativo fallito con modello MilaNLProc/bert-italian-cased-ner: MilaNLProc/bert-italian-cased-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Modello Transformer NER caricato: Davlan/bert-base-multilin

In [3]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict, Any, Optional, Tuple, Set # Added missing Any type hint
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch


# Importazione di spaCy
import spacy
# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy
# Modifica alla funzione export_entities_to_txt per includere le entità del riconoscitore italiano
#def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, italian_recognizer=None, original_text=None):

def export_entities_to_txt(
    entities,
    stats,
    output_file="entita.txt",
    spacy_recognizer=None,
    italian_recognizer=None,
    transformers_recognizer=None,
    original_text=None
    ):
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Include anche le entità riconosciute da spaCy, dal riconoscitore italiano e dal riconoscitore Transformer NER.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy (opzionale)
        italian_recognizer: Riconoscitore italiano (opzionale)
        transformers_recognizer: Riconoscitore Transformer NER (opzionale)
        original_text: Testo originale da analizzare (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]
    spacy_entities = [e for e in entities if e.get('source') == 'spacy']
    italian_entities = [e for e in entities if e.get('source') in ['italian_nlp', 'stanza', 'stanza_pos', 'geo_rule']]
    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']  # Assicurati che questa riga non sia commentata

    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in entities:  # Iterate through all entities
        if entity.get('source') in ['wikidata', 'wikidata+spacy']:  # Check if the entity is from Wikidata
            lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")  # Access Wikidata-specific attributes only if present
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")

    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_all_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_all_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback alle entità spaCy già filtrate
        if not spacy_entities:
            lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
        else:
            for entity in spacy_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore italiano
    lines.append("ITALIAN NLP ENTITIES:")
    lines.append("--------------------")

    # Se abbiamo un riconoscitore italiano e un testo originale, ottieni tutte le entità
    if italian_recognizer and original_text:
        try:
            italian_all_entities = italian_recognizer.recognize_entities(original_text)

            if not italian_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore italiano nel testo.")
            else:
                for entity in italian_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore italiano: {e}")
    else:
        # Fallback alle entità italiane già filtrate
        if not italian_entities:
            lines.append("Per mostrare tutte le entità del riconoscitore italiano, è necessario passare il riconoscitore e il testo originale.")
        else:
            for entity in italian_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore Transformer NER
    lines.append("TRANSFORMER NER ENTITIES:")
    lines.append("-------------------------")

    # Se abbiamo un riconoscitore Transformer e un testo originale, ottieni tutte le entità
    if transformers_recognizer and original_text and hasattr(transformers_recognizer, 'recognize_entities'):
        try:
            print("DEBUG: Tentativo di ottenere entità dal riconoscitore Transformer")
            transformers_all_entities = transformers_recognizer.recognize_entities(original_text)

            if not transformers_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore Transformer NER.")
            else:
                print(f"DEBUG: Trovate {len(transformers_all_entities)} entità da Transformer")
                # Rimuovi duplicati mantenendo l'ordine originale
                unique_transformers_entities = []
                seen = set()
                for entity in transformers_all_entities:
                    if entity['text'] not in seen:
                        unique_transformers_entities.append(entity)
                        seen.add(entity['text'])

                for entity in unique_transformers_entities:
                    # Usa il primo tipo disponibile o 'N/A'
                    label = entity.get('label', entity.get('types', ['N/A'])[0] if entity.get('types') else 'N/A')
                    score = entity.get('score', 'N/A')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}, Punteggio: {score}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore Transformer: {e}")
            print(f"DEBUG: Errore nell'ottenere entità Transformer: {e}")
    else:
        # Fallback alle entità Transformer già filtrate nel metodo find_entities
        if not transformers_entities:
            if not transformers_recognizer:
                lines.append("Riconoscitore Transformer NER non disponibile.")
            elif not hasattr(transformers_recognizer, 'recognize_entities'):
                lines.append("Il riconoscitore Transformer NER non ha il metodo recognize_entities.")
            else:
                lines.append("Per mostrare tutte le entità del riconoscitore Transformer, è necessario passare il riconoscitore e il testo originale.")
        else:
            print(f"DEBUG: Usando {len(transformers_entities)} entità Transformer già filtrate")
            for entity in transformers_entities:
                label = entity.get('label', 'N/A')
                score = entity.get('score', 'N/A')
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Punteggio: {score}")
    lines.append("")

    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities


class ItalianNLPRecognizer:
    """
    Riconoscitore di entità ottimizzato per l'italiano in ambiente Colab.
    Usa una combinazione di Stanza (versione Python di Stanford CoreNLP)
    e altre librerie per l'italiano.
    """

    def __init__(self, use_gpu: bool = False):
        """
        Inizializza il riconoscitore per l'italiano.

        Args:
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        self.use_gpu = use_gpu
        self.is_available = False
        self.stanza_nlp = None
        self.initialized = False

        # Inizializza le risorse
        self._initialize_resources()

    def _initialize_resources(self):
        """Inizializza Stanza e altre risorse per l'italiano."""
        try:
            import stanza
            import simplemma
            import nltk
            from nltk.tokenize import word_tokenize

            # Scarica i modelli di Stanza per l'italiano
            stanza.download('it')

            # Inizializza il pipeline Stanza per l'italiano
            self.stanza_nlp = stanza.Pipeline(
                lang='it',
                processors='tokenize,mwt,pos,lemma,ner',
                use_gpu=self.use_gpu
            )

            # Scarica le risorse NLTK necessarie
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('averaged_perceptron_tagger')

            self.is_available = True
            self.initialized = True
            print("Riconoscitore NLP italiano inizializzato correttamente con Stanza.")

        except Exception as e:
            print(f"Errore nell'inizializzazione del riconoscitore italiano: {e}")
            print("Assicurati di aver eseguito le installazioni necessarie in Colab.")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo italiano utilizzando Stanza.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.stanza_nlp:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con Stanza
            doc = self.stanza_nlp(text)

            # Estrai le entità
            for sent in doc.sentences:
                for ent in sent.ents:
                    entity = {
                        'text': ent.text,
                        'label': ent.type,
                        'start_char': -1,  # Stanza non fornisce direttamente le posizioni dei caratteri
                        'end_char': -1,
                        'source': 'stanza'
                    }
                    entities.append(entity)

                # Aggiungi anche i nomi propri (POS tag == PROPN)
                for token in sent.tokens:
                    for word in token.words:
                        if word.upos == 'PROPN' and not any(e['text'] == word.text for e in entities):
                            entity = {
                                'text': word.text,
                                'label': 'PROPN',
                                'start_char': -1,
                                'end_char': -1,
                                'source': 'stanza_pos'
                            }
                            entities.append(entity)

            # Cerca anche entità geografiche italiane specifiche usando regole
            entities.extend(self._extract_italian_geo_entities(text))

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Stanza: {e}")

        return entities

    def _extract_italian_geo_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Estrae entità geografiche italiane usando regole.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità geografiche
        """
        import re
        entities = []

        # Regioni italiane
        regions = [
            "Abruzzo", "Basilicata", "Calabria", "Campania", "Emilia-Romagna",
            "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardia", "Marche",
            "Molise", "Piemonte", "Puglia", "Sardegna", "Sicilia", "Toscana",
            "Trentino-Alto Adige", "Umbria", "Valle d'Aosta", "Veneto"
        ]

        # Pattern per laghi, montagne, fiumi italiani
        geo_patterns = [
            r'\b(?:Lago|Monte|Fiume|Isola|Golfo|Capo|Valle|Parco Nazionale|Mar) [A-Z][a-zàèéìòù]+\b',
            r'\b(?:Alpi|Appennini|Dolomiti|Maremma|Laguna|Costa|Riviera) [A-Z][a-zàèéìòù]*\b'
        ]

        # Cerca regioni
        for region in regions:
            if re.search(r'\b' + re.escape(region) + r'\b', text):
                entity = {
                    'text': region,
                    'label': 'GPE',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        # Cerca pattern geografici
        for pattern in geo_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                entity = {
                    'text': match.group(0),
                    'label': 'LOC',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        return entities


# Alternative per modelli italiani NER
model_options = [
    "5had3/bert-base-italian-cased-ner",  # Un'alternativa affidabile
    "MilaNLProc/bert-italian-cased-ner",  # Altro modello italiano per NER
    "dslim/bert-base-NER"  # Modello generico che funziona bene
]


class TransformersNERRecognizer:
    def __init__(self,
                 model_name: str = None,
                 language: str = "it",
                 use_gpu: bool = False):
        """
        Inizializza il riconoscitore di entità con un modello Transformer.

        Args:
            model_name: Nome del modello pre-addestrato per NER
            language: Lingua del modello (default: italiano)
            use_gpu: Se utilizzare l'accelerazione GPU
        """
        self.language = language
        self.use_gpu = use_gpu

        # Dizionario di mapping per modelli in diverse lingue
        self.language_models = {
            "it": [
                "5had3/bert-base-italian-cased-ner",
                "MilaNLProc/bert-italian-cased-ner",
                "dslim/bert-base-NER"
            ],
            "en": ["dslim/bert-base-NER"],
            "de": ["RafLorem/bert-base-german-NER"],
            "fr": ["Jean-Baptiste/camembert-ner"]
        }

        # Lista di fallback globale (modelli che funzionano in molte lingue)
        self.fallback_models = [
            "dslim/bert-base-NER",               # Ottimo supporto multilingua
            "dbmdz/bert-large-cased-finetuned-conll03-english",  # Molto robusto
            "Babelscape/wikineural-multilingual-ner"  # Supporto multilingua avanzato
        ]

        # Se non specificato, scegli un modello predefinito
        if model_name is None:
            model_name = self._select_best_model()

        self.model_name = model_name
        self.is_available = False
        self.ner_pipeline = None

        # Inizializza il modello
        self._load_model()

    def _select_best_model(self) -> str:
        """
        Seleziona il miglior modello disponibile per la lingua.

        Returns:
            Nome del modello
        """
        print(f"DEBUG: Selezionando modello per lingua: {self.language}")

        # Prova prima i modelli specifici per la lingua
        models = self.language_models.get(self.language, self.language_models.get("en", []))

        for model in models:
            try:
                # Importa solo il tokenizer per verificare la disponibilità
                from transformers import AutoTokenizer
                print(f"DEBUG: Verifica disponibilità modello: {model}")

                # Verifica la disponibilità con caricamento offline
                tokenizer = AutoTokenizer.from_pretrained(model, local_files_only=False)
                print(f"DEBUG: Modello {model} disponibile e selezionato")
                return model
            except Exception as e:
                print(f"DEBUG: Modello {model} non disponibile: {str(e)}")
                continue

        # Se nessun modello specifico per lingua funziona, prova i fallback
        print("DEBUG: Nessun modello specifico per lingua disponibile, provo fallback globali")
        for fallback in self.fallback_models:
            try:
                from transformers import AutoTokenizer
                print(f"DEBUG: Provo fallback modello: {fallback}")
                tokenizer = AutoTokenizer.from_pretrained(fallback, local_files_only=False)
                print(f"DEBUG: Fallback {fallback} disponibile e selezionato")
                return fallback
            except Exception as e:
                print(f"DEBUG: Fallback {fallback} non disponibile: {str(e)}")
                continue

        # Ultimo tentativo: dslim/bert-base-NER (molto comune)
        print("DEBUG: Ultimo tentativo con dslim/bert-base-NER")
        return "dslim/bert-base-NER"

    def _load_model(self):
        """
        Carica il modello Transformer per il riconoscimento di entità.
        """
        try:
            # Assicurati che le dipendenze siano disponibili
            import torch
            from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

            print(f"DEBUG: Tentativo di caricare il modello: {self.model_name}")

            # Determina il device
            device = 0 if self.use_gpu and torch.cuda.is_available() else -1
            print(f"DEBUG: Usando device {'GPU' if device == 0 else 'CPU'}")

            # Prova a caricare con questo modello specifico
            try:
                # Carica tokenizer e modello
                tokenizer = AutoTokenizer.from_pretrained(self.model_name)
                model = AutoModelForTokenClassification.from_pretrained(self.model_name)

                # Crea pipeline NER
                self.ner_pipeline = pipeline(
                    "ner",
                    model=model,
                    tokenizer=tokenizer,
                    device=device
                )

                self.is_available = True
                print(f"DEBUG: Modello {self.model_name} caricato con successo!")

                # Test rapido per verificare il funzionamento
                test_text = "Mario Rossi vive a Roma."
                test_result = self.ner_pipeline(test_text)
                print(f"DEBUG: Test modello superato - entità trovate: {len(test_result)}")
                return

            except Exception as model_error:
                print(f"DEBUG: Errore nel caricamento del modello specifico: {str(model_error)}")

                # Se il modello specifico fallisce, prova i modelli di fallback dalla lista
                for fallback in self.fallback_models:
                    if fallback != self.model_name:  # Evita di riprovare lo stesso modello
                        try:
                            print(f"DEBUG: Provo modello di fallback: {fallback}")
                            tokenizer = AutoTokenizer.from_pretrained(fallback)
                            model = AutoModelForTokenClassification.from_pretrained(fallback)

                            self.ner_pipeline = pipeline(
                                "ner",
                                model=model,
                                tokenizer=tokenizer,
                                device=device
                            )

                            self.is_available = True
                            self.model_name = fallback  # Aggiorna il nome del modello
                            print(f"DEBUG: Modello fallback {fallback} caricato con successo!")
                            return

                        except Exception as fallback_error:
                            print(f"DEBUG: Fallback {fallback} fallito: {str(fallback_error)}")

                # Se tutti i fallback falliscono, solleva l'errore originale
                raise model_error

        except ImportError as e:
            print(f"DEBUG: Errore di importazione: {str(e)}")
            print("È necessario installare 'transformers' e 'torch':")
            print("pip install transformers torch")
            self.is_available = False

        except Exception as e:
            print(f"DEBUG: Errore generale nel caricamento del modello: {str(e)}")
            import traceback
            traceback.print_exc()
            self.is_available = False

    def recognize_entities(self, text: str) -> list:
        """
        Riconosce entità in un testo utilizzando un modello Transformer.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.ner_pipeline:
            print("DEBUG: Riconoscitore Transformer NER non disponibile.")
            return []

        entities = []

        try:
            print(f"DEBUG: Analisi testo con modello {self.model_name}")

            # Esegui NER con gestione di testi lunghi
            # Alcuni modelli hanno limiti sulla lunghezza del testo
            if len(text) > 500:
                # Dividi il testo in blocchi di 500 caratteri con sovrapposizione
                chunks = []
                chunk_size = 450  # Usa 450 per avere sovrapposizione
                for i in range(0, len(text), chunk_size):
                    # Assicurati che l'ultimo chunk non vada oltre la fine del testo
                    end = min(i + chunk_size + 50, len(text))  # +50 per sovrapposizione
                    chunks.append(text[i:end])

                print(f"DEBUG: Testo diviso in {len(chunks)} chunks per l'analisi")

                # Analizza ogni chunk separatamente
                all_results = []
                for i, chunk in enumerate(chunks):
                    print(f"DEBUG: Analisi chunk {i+1}/{len(chunks)}")
                    chunk_results = self.ner_pipeline(chunk)
                    # Aggiungi offset per i chunk successivi al primo
                    if i > 0:
                        offset = i * chunk_size
                        for result in chunk_results:
                            result['start'] += offset
                            result['end'] += offset
                    all_results.extend(chunk_results)

                ner_results = all_results
            else:
                # Per testi brevi, usa direttamente il pipeline
                ner_results = self.ner_pipeline(text)

            print(f"DEBUG: Trovati {len(ner_results)} token di entità")

            # Raggruppa entità consecutive con lo stesso tipo
            grouped_entities = []
            current_entity = None

            for result in ner_results:
                if result['entity'].startswith('B-'):
                    # Nuova entità
                    if current_entity:
                        grouped_entities.append(current_entity)

                    current_entity = {
                        'text': result['word'],
                        'label': result['entity'][2:],  # Rimuovi il prefisso "B-"
                        'start_char': result['start'],
                        'end_char': result['end'],
                        'score': result['score']
                    }

                elif result['entity'].startswith('I-') and current_entity:
                    # Continua l'entità precedente se il tipo corrisponde
                    if result['entity'][2:] == current_entity['label']:
                        # Gestione speciale per i token tokenizzati che iniziano con ##
                        if result['word'].startswith('##'):
                            current_entity['text'] += result['word'][2:]  # Rimuovi ##
                        else:
                            # Potrebbe essere necessario aggiungere uno spazio
                            space_needed = not (result['start'] == current_entity['end_char'])
                            if space_needed:
                                current_entity['text'] += ' ' + result['word']
                            else:
                                current_entity['text'] += result['word']

                        current_entity['end_char'] = result['end']
                        # Aggiorna lo score con la media o il massimo
                        current_entity['score'] = max(current_entity['score'], result['score'])

                elif result['entity'] != 'O':  # Altra etichetta (non "Other")
                    # Gestisci entità che non hanno markup B-/I-
                    if current_entity:
                        grouped_entities.append(current_entity)

                    # Crea una nuova entità
                    current_entity = {
                        'text': result['word'],
                        'label': result['entity'].replace('B-', '').replace('I-', ''),
                        'start_char': result['start'],
                        'end_char': result['end'],
                        'score': result['score']
                    }

                else:  # Token classificato come "O" (Other)
                    # Finalizza l'entità corrente, se esiste
                    if current_entity:
                        grouped_entities.append(current_entity)
                        current_entity = None

            # Aggiungi l'ultima entità in elaborazione
            if current_entity:
                grouped_entities.append(current_entity)

            print(f"DEBUG: Raggruppate in {len(grouped_entities)} entità")

            # Aggiungi source e crea lista finale
            for entity in grouped_entities:
                # Pulizia finale del testo (rimuovi spazi extra)
                entity['text'] = entity['text'].strip()
                # Aggiungi la fonte
                entity['source'] = 'transformers_ner'
                entities.append(entity)

            return entities

        except Exception as e:
            print(f"DEBUG: Errore nel riconoscimento entità: {str(e)}")
            import traceback
            traceback.print_exc()
            return []
    def set_language(self, language: str):
        """
        Cambia la lingua del riconoscitore.

        Args:
            language: Nuova lingua da impostare
        """
        if language != self.language:
            self.language = language
            self._load_model()

    def get_supported_languages(self) -> List[str]:
        """
        Restituisce le lingue supportate.

        Returns:
            Lista di codici lingua supportati
        """
        return list(self.language_models.keys())



# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }



# Estendi la classe EntityLinkerWithSpacy per incorporare il riconoscitore italiano
class EntityLinkerItalian(EntityLinkerWithSpacy):
    """
    Versione estesa di EntityLinker ottimizzata per l'italiano in ambiente Colab.
    Integra riconoscitori multipli: Wikidata, spaCy, Stanza NLP e Transformer NER.
    """

    def __init__(self, language: str = "it", use_gpu: bool = False):
        """
        Inizializza il linker di entità con supporto NLP per l'italiano.

        Args:
            language: Lingua preferita per i risultati (deve essere "it")
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        # Controlla che la lingua sia italiana
        if language != "it":
            print("Attenzione: questo riconoscitore è ottimizzato per l'italiano. Impostiamo language='it'.")
            language = "it"

        # Inizializza la classe base (con spaCy)
        super().__init__(language)

        # Aggiungi il riconoscitore italiano Stanza NLP
        self.italian_recognizer = ItalianNLPRecognizer(use_gpu=use_gpu)

        # Aggiungi il riconoscitore Transformer NER
        try:
            from transformers import AutoTokenizer, pipeline  # Verifica disponibilità di transformers
            import torch  # Verifica disponibilità di torch

            print("DEBUG: Moduli transformers e torch disponibili. Inizializzazione riconoscitore Transformer NER...")

            self.transformers_recognizer = TransformersNERRecognizer(
                model_name="5had3/bert-base-italian-cased-ner",
                language=language,
                use_gpu=use_gpu
            )

            # Verifica se il riconoscitore è stato inizializzato correttamente
            if not hasattr(self.transformers_recognizer, 'is_available') or not self.transformers_recognizer.is_available:
                print("DEBUG: Riconoscitore Transformer NER non disponibile dopo l'inizializzazione")
            else:
                print("DEBUG: Riconoscitore Transformer NER inizializzato con successo")

        except ImportError as e:
            print(f"DEBUG: ImportError - Transformer NER non disponibile. {e}")
            print("Installare 'transformers' e 'torch' con 'pip install transformers torch'")
            self.transformers_recognizer = None
        except Exception as e:
            print(f"DEBUG: Errore generico nell'inizializzazione del riconoscitore Transformer NER: {e}")
            import traceback
            traceback.print_exc()
            self.transformers_recognizer = None

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando Wikidata, spaCy, Stanza NLP e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Debug del riconoscitore Transformer
        print(f"DEBUG: Stato riconoscitore Transformer - Disponibile: {hasattr(self, 'transformers_recognizer')}")
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer:
            print(f"DEBUG: Riconoscitore Transformer - Is Available: {self.transformers_recognizer.is_available}")

        # Prima ottieni i risultati dalla classe base (Wikidata + spaCy)
        enriched_entities, stats = super().find_entities(text)
        existing_texts = {e.get('text', '').lower() for e in enriched_entities if 'text' in e}

        # Aggiungi le entità del riconoscitore italiano se disponibile
        italian_only_entities = self._get_italian_only_entities(text, existing_texts)
        enriched_entities.extend(italian_only_entities)

        # Aggiorna il set di testi esistenti dopo aver aggiunto le entità italiane
        for entity in italian_only_entities:
            if 'text' in entity:
                existing_texts.add(entity['text'].lower())

        # Aggiungi le entità dal riconoscitore Transformer NER
        transformers_only_entities = []
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer and self.transformers_recognizer.is_available:
            print("DEBUG: Tentativo di trovare entità Transformer NER")
            try:
                # Usa un metodo separato per ottenere le entità
                transformers_only_entities = self._get_transformers_only_entities(text, existing_texts)
                print(f"DEBUG: Entità Transformer NER trovate: {len(transformers_only_entities)}")

                # Aggiungi solo se ci sono entità
                if transformers_only_entities:
                    enriched_entities.extend(transformers_only_entities)
            except Exception as e:
                print(f"DEBUG: Errore nel trovare entità Transformer: {e}")
                import traceback
                traceback.print_exc()
        else:
            print("DEBUG: Riconoscitore Transformer NER non disponibile per find_entities")

        # Aggiorna le statistiche
        stats['italian_only_entities'] = len(italian_only_entities)
        stats['transformers_only_entities'] = len(transformers_only_entities)
        stats['total_entities'] = len(enriched_entities)

        return enriched_entities, stats

    def _get_italian_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore italiano e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo dal riconoscitore italiano
        """
        # Verifica che il riconoscitore italiano sia disponibile
        if not self.italian_recognizer.is_available:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore italiano
        italian_entities = self.italian_recognizer.recognize_entities(text)
        print(f"Riconoscitore italiano ha trovato {len(italian_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        italian_only = []
        for entity in italian_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore italiano: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                italian_type = self._map_italian_label_to_type(entity['label'])

                italian_entity = {
                    'text': entity_text,
                    'types': [italian_type],
                    'source': 'italian_nlp',
                    'label': entity['label']
                }

                italian_only.append(italian_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(italian_only)} sono uniche (non trovate da altre fonti)")
        return italian_only

    def _get_transformers_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
          """
          Ottiene entità riconosciute dal riconoscitore Transformer NER.
          Includendo sia entità nuove che quelle già trovate da altre fonti.
          """
          # Verifica che il riconoscitore Transformer sia disponibile
          if not self.transformers_recognizer or not self.transformers_recognizer.is_available:
              print("DEBUG: Riconoscitore Transformer NER non disponibile.")
              return []

          # Stampa informazioni sul riconoscitore
          print(f"DEBUG: Modello Transformer NER in uso: {self.transformers_recognizer.model_name}")

          try:
              # Ottieni tutte le entità dal riconoscitore Transformer NER
              transformers_entities = self.transformers_recognizer.recognize_entities(text)

              # Debug dettagliato delle entità
              print(f"DEBUG: Numero di entità Transformer NER trovate: {len(transformers_entities)}")

              # Filtra e mappa le entità - IMPORTANTE: non filtriamo per existing_texts
              # per assicurarci che tutte le entità Transformer siano incluse
              transformers_mapped = []
              for entity in transformers_entities:
                  if self._is_valid_entity(entity.get('text', '')):
                      # Aggiungi sempre il source 'transformers_ner'
                      mapped_entity = {
                          'text': entity.get('text', ''),
                          'label': entity.get('label', ''),
                          'types': [entity.get('label', 'N/A')],
                          'source': 'transformers_ner',
                          'score': entity.get('score', 0)
                      }
                      transformers_mapped.append(mapped_entity)

              print(f"DEBUG: Numero di entità Transformer NER mappate: {len(transformers_mapped)}")
              return transformers_mapped

          except Exception as e:
              print(f"DEBUG: Errore nell'ottenere entità Transformer: {e}")
              import traceback
              traceback.print_exc()
              return []

    def _map_italian_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore italiano a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        mapping = {
            'PERSON': 'Person',
            'PER': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'MISC': 'Miscellaneous',
            'PROPN': 'ProperNoun',
            'WORK_OF_ART': 'Artwork',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def _map_transformers_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore Transformer NER a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        # (simile a quella di altri riconoscitori)
        mapping = {
            'PER': 'Person',
            'LOC': 'Location',
            'ORG': 'Organization',
            'MISC': 'Miscellaneous',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'PERSON': 'Person',
            'ORGANIZATION': 'Organization',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estende il metodo della classe base per includere i risultati
        del riconoscitore italiano e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        # Ottieni i risultati di base
        uri_results = super().verify_uris_for_text(text)

        # Aggiungi una sezione per le entità del riconoscitore italiano senza URI
        italian_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'italian_nlp' and entity['text'] not in uri_results['results']:
                italian_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        # Aggiungi una sezione per le entità del riconoscitore Transformer NER senza URI
        transformers_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'transformers_ner' and entity['text'] not in uri_results['results']:
                transformers_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', []),
                    'score': entity.get('score', 0)
                }

        # Aggiungi le sezioni ai risultati
        uri_results['italian_results'] = italian_results
        uri_results['transformers_results'] = transformers_results

        return uri_results


# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

if __name__ == "__main__":
    # Importazioni necessarie
    import os
    import sys
    import traceback

    # Imposta il livello di log per transformers
    os.environ["TRANSFORMERS_VERBOSITY"] = "info"

    # Verifica le dipendenze
    try:
        import torch
        import transformers
        print(f"PyTorch versione: {torch.__version__}")
        print(f"Transformers versione: {transformers.__version__}")
        print(f"CUDA disponibile: {torch.cuda.is_available()}")
    except ImportError as e:
        print(f"ATTENZIONE: Libreria mancante - {e}")

    try:
        # Crea un'istanza del linker di entità migliorato con debug aggiuntivo
        print("Inizializzazione EntityLinkerItalian...")
        linker = EntityLinkerItalian(language="it", use_gpu=False)

        # Verifica disponibilità dei riconoscitori
        print(f"SpaCy riconoscitore disponibile: {linker.wikidata.spacy_recognizer.is_available}")
        print(f"Italian NLP riconoscitore disponibile: {linker.italian_recognizer.is_available}")
        if hasattr(linker, 'transformers_recognizer') and linker.transformers_recognizer:
            print(f"Transformer NER riconoscitore disponibile: {linker.transformers_recognizer.is_available}")
            print(f"Transformer NER modello: {linker.transformers_recognizer.model_name}")
        else:
            print("Transformer NER riconoscitore non disponibile")

        # Testo di esempio
        text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
               " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
               " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
               " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
               " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
               " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
               " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
               " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
               " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
               " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

        print(f"Analisi del testo:\n{text}\n")

        # Test esplicito del riconoscitore Transformer NER, se disponibile
        if hasattr(linker, 'transformers_recognizer') and linker.transformers_recognizer and linker.transformers_recognizer.is_available:
            print("\nTest diretto del riconoscitore Transformer NER:")
            test_sample = "Mario Rossi vive a Roma e lavora per l'Università La Sapienza."
            try:
                transformer_test_entities = linker.transformers_recognizer.recognize_entities(test_sample)
                print(f"Entità trovate dal riconoscitore Transformer in test: {len(transformer_test_entities)}")
                for entity in transformer_test_entities:
                    print(f"- '{entity['text']}' → Etichetta: {entity.get('label')}, Score: {entity.get('score')}")
            except Exception as e:
                print(f"Errore nel test diretto del riconoscitore Transformer: {e}")
                traceback.print_exc()

        # Trova entità
        print("\nRicerca entità nel testo principale...")
        entities, stats = linker.find_entities(text)

        # Verifica entità Transformer
        transformer_entities = [e for e in entities if e.get('source') == 'transformers_ner']
        print(f"\nEntità Transformer NER trovate: {len(transformer_entities)}")
        for entity in transformer_entities[:10]:  # Mostra solo le prime 10 per brevità
            print(f"- '{entity['text']}' → Etichetta: {entity.get('label')}, Score: {entity.get('score', 'N/A')}")

        print(f"\nTrovate {stats['total_entities']} entità totali:")
        for k, v in stats.items():
            print(f"- {k}: {v}")

        # Esporta le entità in un file
        print("\nEsportazione entità in file...")
        output_file_path = export_entities_to_txt(
            entities,
            stats,
            "entita_complete.txt",  # Nuovo nome file per distinguerlo dal precedente
            spacy_recognizer=linker.wikidata.spacy_recognizer,
            italian_recognizer=linker.italian_recognizer,
            transformers_recognizer=linker.transformers_recognizer,
            original_text=text
        )

        print(f"\nFile di output salvato in: {output_file_path}")

        # Verifica il file di output
        import os
        if os.path.exists(output_file_path):
            file_size = os.path.getsize(output_file_path)
            print(f"Verifica file: '{output_file_path}' esiste e ha dimensione {file_size} bytes")

            # Leggi il file per verificare il contenuto
            with open(output_file_path, 'r', encoding='utf-8') as f:
                content = f.read()

            # Cerca la sezione Transformer nel file
            if "TRANSFORMER NER ENTITIES:" in content:
                transformer_section = content.split("TRANSFORMER NER ENTITIES:")[1].split("\n\n")[0]
                transformer_lines = [line for line in transformer_section.split("\n") if line.strip() and "-----" not in line]
                print(f"\nSezione Transformer nel file di output contiene {len(transformer_lines)} righe")
                if len(transformer_lines) > 1:  # La prima riga è l'intestazione
                    print("Le prime 5 entità Transformer nel file:")
                    for line in transformer_lines[:6]:
                        print(f"  {line}")
                else:
                    print("ATTENZIONE: Nessuna entità Transformer trovata nel file di output")
            else:
                print("ATTENZIONE: Sezione Transformer non trovata nel file di output")
        else:
            print(f"ATTENZIONE: Il file '{output_file_path}' non è stato creato!")

    except Exception as e:
        print(f"Errore generale: {e}")
        traceback.print_exc()

PyTorch versione: 2.6.0+cu124
Transformers versione: 4.33.3
CUDA disponibile: True
Inizializzazione EntityLinkerItalian...
Modello spaCy it_core_news_lg caricato con successo.


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: it (Italian) ...
INFO:stanza:File exists: /root/stanza_resources/it/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| ner       | fbk               |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Riconoscitore NLP italiano inizializzato correttamente con Stanza.
DEBUG: Moduli transformers e torch disponibili. Inizializzazione riconoscitore Transformer NER...
DEBUG: Tentativo di caricare il modello: 5had3/bert-base-italian-cased-ner
DEBUG: Usando device CPU
DEBUG: Errore nel caricamento del modello specifico: 5had3/bert-base-italian-cased-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
DEBUG: Provo modello di fallback: dslim/bert-base-NER


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DEBUG: Modello fallback dslim/bert-base-NER caricato con successo!
DEBUG: Riconoscitore Transformer NER inizializzato con successo
SpaCy riconoscitore disponibile: True
Italian NLP riconoscitore disponibile: True
Transformer NER riconoscitore disponibile: True
Transformer NER modello: dslim/bert-base-NER
Analisi del testo:
L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico. Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento. Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio. Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export. Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa. Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo. La Ferrari e la Lamborghini son

codice corretto

In [11]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict, Any, Optional, Tuple, Set # Added missing Any type hint
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch


# Importazione di spaCy
import spacy
# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy
# Modifica alla funzione export_entities_to_txt per includere le entità del riconoscitore italiano
#def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, italian_recognizer=None, original_text=None):

def export_entities_to_txt(
    entities,
    stats,
    output_file="entita.txt",
    spacy_recognizer=None,
    italian_recognizer=None,
    transformers_recognizer=None,
    original_text=None
    ):
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Include anche le entità riconosciute da spaCy, dal riconoscitore italiano e dal riconoscitore Transformer NER.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy (opzionale)
        italian_recognizer: Riconoscitore italiano (opzionale)
        transformers_recognizer: Riconoscitore Transformer NER (opzionale)
        original_text: Testo originale da analizzare (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]
    spacy_entities = [e for e in entities if e.get('source') == 'spacy']
    italian_entities = [e for e in entities if e.get('source') in ['italian_nlp', 'stanza', 'stanza_pos', 'geo_rule']]
    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']  # Assicurati che questa riga non sia commentata

    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in entities:  # Iterate through all entities
        if entity.get('source') in ['wikidata', 'wikidata+spacy']:  # Check if the entity is from Wikidata
            lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")  # Access Wikidata-specific attributes only if present
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")

    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_all_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_all_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback alle entità spaCy già filtrate
        if not spacy_entities:
            lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
        else:
            for entity in spacy_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore italiano
    lines.append("ITALIAN NLP ENTITIES:")
    lines.append("--------------------")

    # Se abbiamo un riconoscitore italiano e un testo originale, ottieni tutte le entità
    if italian_recognizer and original_text:
        try:
            italian_all_entities = italian_recognizer.recognize_entities(original_text)

            if not italian_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore italiano nel testo.")
            else:
                for entity in italian_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore italiano: {e}")
    else:
        # Fallback alle entità italiane già filtrate
        if not italian_entities:
            lines.append("Per mostrare tutte le entità del riconoscitore italiano, è necessario passare il riconoscitore e il testo originale.")
        else:
            for entity in italian_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore Transformer NER
    lines.append("TRANSFORMER NER ENTITIES:")
    lines.append("-------------------------")

    # Se abbiamo un riconoscitore Transformer e un testo originale, ottieni tutte le entità
    if transformers_recognizer and original_text and hasattr(transformers_recognizer, 'recognize_entities'):
        try:
            print("DEBUG: Tentativo di ottenere entità dal riconoscitore Transformer")
            transformers_all_entities = transformers_recognizer.recognize_entities(original_text)

            if not transformers_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore Transformer NER.")
            else:
                print(f"DEBUG: Trovate {len(transformers_all_entities)} entità da Transformer")
                # Rimuovi duplicati mantenendo l'ordine originale
                unique_transformers_entities = []
                seen = set()
                for entity in transformers_all_entities:
                    if entity['text'] not in seen:
                        unique_transformers_entities.append(entity)
                        seen.add(entity['text'])

                for entity in unique_transformers_entities:
                    # Usa il primo tipo disponibile o 'N/A'
                    label = entity.get('label', entity.get('types', ['N/A'])[0] if entity.get('types') else 'N/A')
                    score = entity.get('score', 'N/A')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}, Punteggio: {score}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore Transformer: {e}")
            print(f"DEBUG: Errore nell'ottenere entità Transformer: {e}")
    else:
        # Fallback alle entità Transformer già filtrate nel metodo find_entities
        if not transformers_entities:
            if not transformers_recognizer:
                lines.append("Riconoscitore Transformer NER non disponibile.")
            elif not hasattr(transformers_recognizer, 'recognize_entities'):
                lines.append("Il riconoscitore Transformer NER non ha il metodo recognize_entities.")
            else:
                lines.append("Per mostrare tutte le entità del riconoscitore Transformer, è necessario passare il riconoscitore e il testo originale.")
        else:
            print(f"DEBUG: Usando {len(transformers_entities)} entità Transformer già filtrate")
            for entity in transformers_entities:
                label = entity.get('label', 'N/A')
                score = entity.get('score', 'N/A')
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Punteggio: {score}")
    lines.append("")

    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities


class ItalianNLPRecognizer:
    """
    Riconoscitore di entità ottimizzato per l'italiano in ambiente Colab.
    Usa una combinazione di Stanza (versione Python di Stanford CoreNLP)
    e altre librerie per l'italiano.
    """

    def __init__(self, use_gpu: bool = False):
        """
        Inizializza il riconoscitore per l'italiano.

        Args:
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        self.use_gpu = use_gpu
        self.is_available = False
        self.stanza_nlp = None
        self.initialized = False

        # Inizializza le risorse
        self._initialize_resources()

    def _initialize_resources(self):
        """Inizializza Stanza e altre risorse per l'italiano."""
        try:
            import stanza
            import simplemma
            import nltk
            from nltk.tokenize import word_tokenize

            # Scarica i modelli di Stanza per l'italiano
            stanza.download('it')

            # Inizializza il pipeline Stanza per l'italiano
            self.stanza_nlp = stanza.Pipeline(
                lang='it',
                processors='tokenize,mwt,pos,lemma,ner',
                use_gpu=self.use_gpu
            )

            # Scarica le risorse NLTK necessarie
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('averaged_perceptron_tagger')

            self.is_available = True
            self.initialized = True
            print("Riconoscitore NLP italiano inizializzato correttamente con Stanza.")

        except Exception as e:
            print(f"Errore nell'inizializzazione del riconoscitore italiano: {e}")
            print("Assicurati di aver eseguito le installazioni necessarie in Colab.")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo italiano utilizzando Stanza.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.stanza_nlp:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con Stanza
            doc = self.stanza_nlp(text)

            # Estrai le entità
            for sent in doc.sentences:
                for ent in sent.ents:
                    entity = {
                        'text': ent.text,
                        'label': ent.type,
                        'start_char': -1,  # Stanza non fornisce direttamente le posizioni dei caratteri
                        'end_char': -1,
                        'source': 'stanza'
                    }
                    entities.append(entity)

                # Aggiungi anche i nomi propri (POS tag == PROPN)
                for token in sent.tokens:
                    for word in token.words:
                        if word.upos == 'PROPN' and not any(e['text'] == word.text for e in entities):
                            entity = {
                                'text': word.text,
                                'label': 'PROPN',
                                'start_char': -1,
                                'end_char': -1,
                                'source': 'stanza_pos'
                            }
                            entities.append(entity)

            # Cerca anche entità geografiche italiane specifiche usando regole
            entities.extend(self._extract_italian_geo_entities(text))

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Stanza: {e}")

        return entities

    def _extract_italian_geo_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Estrae entità geografiche italiane usando regole.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità geografiche
        """
        import re
        entities = []

        # Regioni italiane
        regions = [
            "Abruzzo", "Basilicata", "Calabria", "Campania", "Emilia-Romagna",
            "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardia", "Marche",
            "Molise", "Piemonte", "Puglia", "Sardegna", "Sicilia", "Toscana",
            "Trentino-Alto Adige", "Umbria", "Valle d'Aosta", "Veneto"
        ]

        # Pattern per laghi, montagne, fiumi italiani
        geo_patterns = [
            r'\b(?:Lago|Monte|Fiume|Isola|Golfo|Capo|Valle|Parco Nazionale|Mar) [A-Z][a-zàèéìòù]+\b',
            r'\b(?:Alpi|Appennini|Dolomiti|Maremma|Laguna|Costa|Riviera) [A-Z][a-zàèéìòù]*\b'
        ]

        # Cerca regioni
        for region in regions:
            if re.search(r'\b' + re.escape(region) + r'\b', text):
                entity = {
                    'text': region,
                    'label': 'GPE',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        # Cerca pattern geografici
        for pattern in geo_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                entity = {
                    'text': match.group(0),
                    'label': 'LOC',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        return entities


# Alternative per modelli italiani NER
model_options = [
    "5had3/bert-base-italian-cased-ner",  # Un'alternativa affidabile
    "MilaNLProc/bert-italian-cased-ner",  # Altro modello italiano per NER
    "dslim/bert-base-NER"  # Modello generico che funziona bene
]


class TransformersNERRecognizer:
    def __init__(self,
                 model_name: str = None,
                 language: str = "it",
                 use_gpu: bool = False):
        """
        Inizializza il riconoscitore di entità con un modello Transformer.

        Args:
            model_name: Nome del modello pre-addestrato per NER
            language: Lingua del modello (default: italiano)
            use_gpu: Se utilizzare l'accelerazione GPU
        """
        self.language = language
        self.use_gpu = use_gpu

        # Dizionario di mapping per modelli in diverse lingue
        self.language_models = {
            "it": [
                "5had3/bert-base-italian-cased-ner",
                "MilaNLProc/bert-italian-cased-ner",
                "dslim/bert-base-NER"
            ],
            "en": ["dslim/bert-base-NER"],
            "de": ["RafLorem/bert-base-german-NER"],
            "fr": ["Jean-Baptiste/camembert-ner"]
        }

        # Lista di fallback globale (modelli che funzionano in molte lingue)
        self.fallback_models = [
            "dslim/bert-base-NER",               # Ottimo supporto multilingua
            "dbmdz/bert-large-cased-finetuned-conll03-english",  # Molto robusto
            "Babelscape/wikineural-multilingual-ner"  # Supporto multilingua avanzato
        ]

        # Se non specificato, scegli un modello predefinito
        if model_name is None:
            model_name = self._select_best_model()

        self.model_name = model_name
        self.is_available = False
        self.ner_pipeline = None

        # Inizializza il modello
        self._load_model()

    def _select_best_model(self) -> str:
        """
        Seleziona il miglior modello disponibile per la lingua.

        Returns:
            Nome del modello
        """
        print(f"DEBUG: Selezionando modello per lingua: {self.language}")

        # Prova prima i modelli specifici per la lingua
        models = self.language_models.get(self.language, self.language_models.get("en", []))

        for model in models:
            try:
                # Importa solo il tokenizer per verificare la disponibilità
                from transformers import AutoTokenizer
                print(f"DEBUG: Verifica disponibilità modello: {model}")

                # Verifica la disponibilità con caricamento offline
                tokenizer = AutoTokenizer.from_pretrained(model, local_files_only=False)
                print(f"DEBUG: Modello {model} disponibile e selezionato")
                return model
            except Exception as e:
                print(f"DEBUG: Modello {model} non disponibile: {str(e)}")
                continue

        # Se nessun modello specifico per lingua funziona, prova i fallback
        print("DEBUG: Nessun modello specifico per lingua disponibile, provo fallback globali")
        for fallback in self.fallback_models:
            try:
                from transformers import AutoTokenizer
                print(f"DEBUG: Provo fallback modello: {fallback}")
                tokenizer = AutoTokenizer.from_pretrained(fallback, local_files_only=False)
                print(f"DEBUG: Fallback {fallback} disponibile e selezionato")
                return fallback
            except Exception as e:
                print(f"DEBUG: Fallback {fallback} non disponibile: {str(e)}")
                continue

        # Ultimo tentativo: dslim/bert-base-NER (molto comune)
        print("DEBUG: Ultimo tentativo con dslim/bert-base-NER")
        return "dslim/bert-base-NER"

    def _load_model(self):
        """
        Carica il modello Transformer per il riconoscimento di entità.
        """
        try:
            # Assicurati che le dipendenze siano disponibili
            import torch
            from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

            print(f"DEBUG: Tentativo di caricare il modello: {self.model_name}")

            # Determina il device
            device = 0 if self.use_gpu and torch.cuda.is_available() else -1
            print(f"DEBUG: Usando device {'GPU' if device == 0 else 'CPU'}")

            # Prova a caricare con questo modello specifico
            try:
                # Carica tokenizer e modello
                tokenizer = AutoTokenizer.from_pretrained(self.model_name)
                model = AutoModelForTokenClassification.from_pretrained(self.model_name)

                # Crea pipeline NER
                self.ner_pipeline = pipeline(
                    "ner",
                    model=model,
                    tokenizer=tokenizer,
                    device=device
                )

                self.is_available = True
                print(f"DEBUG: Modello {self.model_name} caricato con successo!")

                # Test rapido per verificare il funzionamento
                test_text = "Mario Rossi vive a Roma."
                test_result = self.ner_pipeline(test_text)
                print(f"DEBUG: Test modello superato - entità trovate: {len(test_result)}")
                return

            except Exception as model_error:
                print(f"DEBUG: Errore nel caricamento del modello specifico: {str(model_error)}")

                # Se il modello specifico fallisce, prova i modelli di fallback dalla lista
                for fallback in self.fallback_models:
                    if fallback != self.model_name:  # Evita di riprovare lo stesso modello
                        try:
                            print(f"DEBUG: Provo modello di fallback: {fallback}")
                            tokenizer = AutoTokenizer.from_pretrained(fallback)
                            model = AutoModelForTokenClassification.from_pretrained(fallback)

                            self.ner_pipeline = pipeline(
                                "ner",
                                model=model,
                                tokenizer=tokenizer,
                                device=device
                            )

                            self.is_available = True
                            self.model_name = fallback  # Aggiorna il nome del modello
                            print(f"DEBUG: Modello fallback {fallback} caricato con successo!")
                            return

                        except Exception as fallback_error:
                            print(f"DEBUG: Fallback {fallback} fallito: {str(fallback_error)}")

                # Se tutti i fallback falliscono, solleva l'errore originale
                raise model_error

        except ImportError as e:
            print(f"DEBUG: Errore di importazione: {str(e)}")
            print("È necessario installare 'transformers' e 'torch':")
            print("pip install transformers torch")
            self.is_available = False

        except Exception as e:
            print(f"DEBUG: Errore generale nel caricamento del modello: {str(e)}")
            import traceback
            traceback.print_exc()
            self.is_available = False

    def recognize_entities(self, text: str) -> list:
        """
        Riconosce entità in un testo utilizzando un modello Transformer,
        con miglioramenti alla tokenizzazione per l'italiano.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.ner_pipeline:
            print("DEBUG: Riconoscitore Transformer NER non disponibile.")
            return []

        entities = []

        try:
            print(f"DEBUG: Analisi testo con modello {self.model_name}")

            # Strategia 1: Analisi parola per parola (per migliorare la tokenizzazione)
            # Dividere il testo in frasi per mantenere un po' di contesto
            import re
            sentences = re.split(r'[.!?]\s+', text)

            print(f"DEBUG: Testo diviso in {len(sentences)} frasi")
            all_ner_results = []

            for sentence_idx, sentence in enumerate(sentences):
                # Calcola l'offset di caratteri per questa frase nel testo originale
                sentence_offset = text.find(sentence)
                if sentence_offset == -1:  # Se non trova esattamente la frase (a causa di spazi, ecc.)
                    # Fai un'approssimazione basata sulle frasi precedenti
                    sentence_offset = sum(len(s) + 2 for s in sentences[:sentence_idx])  # +2 per punteggiatura e spazio

                print(f"DEBUG: Analisi frase {sentence_idx+1}/{len(sentences)}")

                # Divide la frase in insiemi di parole (per mantenere un po' di contesto)
                words = sentence.split()

                # Analizza la frase in piccoli gruppi di parole per mantenere contesto ma migliorare tokenizzazione
                window_size = 3  # Numero di parole da analizzare insieme
                for i in range(0, len(words), window_size):
                    word_group = ' '.join(words[i:i+window_size])

                    # Skip gruppi vuoti
                    if not word_group.strip():
                        continue

                    # Calcola offset all'interno della frase
                    word_group_offset = sentence.find(word_group)
                    if word_group_offset == -1:
                        # Approssimazione se non trova esattamente
                        word_group_offset = sum(len(w) + 1 for w in words[:i])

                    # Offset totale nel testo originale
                    total_offset = sentence_offset + word_group_offset

                    try:
                        # Analizza questo gruppo di parole
                        group_results = self.ner_pipeline(word_group)

                        # Aggiungi l'offset al risultato
                        for result in group_results:
                            result['start'] += total_offset
                            result['end'] += total_offset
                            # Verifica che il contenuto corrisponda al testo originale
                            expected_text = text[result['start']:result['end']]
                            if expected_text != result['word']:
                                print(f"DEBUG: Mismatch - tokenizer ha '{result['word']}' ma il testo ha '{expected_text}'")
                                # Correzione
                                result['word'] = expected_text

                        all_ner_results.extend(group_results)
                    except Exception as e:
                        print(f"DEBUG: Errore nell'analisi del gruppo di parole '{word_group}': {e}")

            print(f"DEBUG: Totale token di entità trovati: {len(all_ner_results)}")

            # Ordinamento per posizione per assicurare la sequenza corretta
            all_ner_results.sort(key=lambda x: x['start'])

            # Raggruppa entità consecutive con lo stesso tipo
            grouped_entities = []
            current_entity = None

            # Fix per gestire entità sovrapposte - usa una strategia differente
            i = 0
            while i < len(all_ner_results):
                result = all_ner_results[i]
                # Ignora token classificati come "O" (Other)
                if result['entity'] == 'O':
                    i += 1
                    continue

                # Estrai tipo di entità (rimuovi prefissi B- e I-)
                entity_type = result['entity']
                if entity_type.startswith('B-') or entity_type.startswith('I-'):
                    entity_type = entity_type[2:]

                # Inizia una nuova entità
                start_pos = result['start']
                end_pos = result['end']
                entity_text = result['word']
                entity_score = result['score']

                # Cerca token consecutivi dello stesso tipo
                j = i + 1
                while j < len(all_ner_results):
                    next_result = all_ner_results[j]
                    next_type = next_result['entity']
                    if next_type.startswith('B-') or next_type.startswith('I-'):
                        next_type = next_type[2:]

                    # Se è lo stesso tipo ed è adiacente o vicino
                    if next_type == entity_type and next_result['start'] <= end_pos + 3:  # +3 per gestire spazi
                        # Estendi l'entità
                        # Gestisci lo spazio tra token
                        if next_result['start'] > end_pos:
                            # C'è uno spazio tra i token
                            entity_text += ' ' + next_result['word']
                        else:
                            # Token adiacenti o sovrapposti
                            entity_text += next_result['word'][end_pos - next_result['start']:]

                        end_pos = next_result['end']
                        entity_score = max(entity_score, next_result['score'])
                        j += 1
                    else:
                        break

                # Crea l'entità completa
                full_entity = {
                    'text': entity_text,
                    'label': entity_type,
                    'start_char': start_pos,
                    'end_char': end_pos,
                    'score': entity_score,
                    'source': 'transformers_ner'
                }

                # Post-processing per correggere errori comuni nei modelli italiani
                full_entity = self._post_process_entity(full_entity, text)

                # Aggiungi l'entità se non vuota dopo post-processing
                if full_entity and full_entity['text'].strip():
                    grouped_entities.append(full_entity)

                # Avanza all'indice dopo l'ultimo token usato in questa entità
                i = j

            print(f"DEBUG: Raggruppate in {len(grouped_entities)} entità")
            return grouped_entities

        except Exception as e:
            print(f"DEBUG: Errore nel riconoscimento entità: {str(e)}")
            import traceback
            traceback.print_exc()
            return []



    def _post_process_entity(self, entity, original_text):
        """
        Applica correzioni post-processing alle entità riconosciute.

        Args:
            entity: Entità da correggere
            original_text: Testo originale per verifica

        Returns:
            Entità corretta o None se l'entità deve essere scartata
        """
        text = entity['text']

        # 1. Correzioni specifiche per errori di tokenizzazione italiani
        corrections = {
            "Romaoss": "Roma",
            "Milanoite": "Milano",
            "Europaolo": "Europa",
            "Barolog": "Barolo",
            "Fiorentina": "Firenze",
            "Sa Polite": "",  # Rimuovi completamente
            "Rinascimentolo": "Rinascimento",
            "Vesuviana": "Vesuvio",
            "Napoli Sa": "Napoli",
            "Polite Milano": "Milano",
            "Milanoite Milano": "Milano",
            "Mediterraneano": "Mediterraneo",
            "Lamborghinini": "Lamborghini"
        }

        # Applica correzioni esatte
        for wrong, correct in corrections.items():
            if wrong in text:
                text = text.replace(wrong, correct)

        # 2. Correggi tokenizzazioni errate conosciute (suffissi e pattern comuni)
        if "oss" in text:
            text = text.replace("oss", "")

        if "aolo" in text and "Europa" in text:
            text = "Europa"

        if "ite" in text and any(city in text for city in ["Milano", "Napoli", "Roma"]):
            for city in ["Milano", "Napoli", "Roma"]:
                if city in text:
                    text = city
                    break

        # 3. Rimuovi suffissi comuni errati
        suffixes = ["oss", "olog", "ologna", "oni", "olino", "aolo", "ite", "inte"]
        for suffix in suffixes:
            if text.endswith(suffix):
                text = text[:-len(suffix)]

        # 4. Gestisci entità con lettere ripetute erroneamente
        if any(c*3 in text for c in "abcdefghijklmnopqrstuvwxyz"):
            # Troppi caratteri ripetuti, probabilmente un errore
            return None

        # 5. Separa entità erroneamente unite
        # Esempio: "NapoliMilano" -> "Napoli"
        cities = ["Roma", "Milano", "Napoli", "Firenze", "Venezia", "Torino", "Bologna", "Palermo"]
        for i, city1 in enumerate(cities):
            for city2 in cities[i+1:]:
                if city1 in text and city2 in text and len(text) < len(city1) + len(city2) + 3:
                    # Probabilmente città unite erroneamente - prendi la prima
                    if text.find(city1) < text.find(city2):
                        text = city1
                    else:
                        text = city2
                    break

        # 6. Gestisci casi specifici di entità concatenate con articoli o preposizioni
        # Es: "LaFerrari" -> "Ferrari"
        articles_preps = ["La", "Il", "Lo", "Gli", "Le", "Di", "Del", "Della", "Dello", "Dei", "Degli", "Delle"]
        for article in articles_preps:
            if text.startswith(article) and len(article) + 1 < len(text):
                # Verifica se c'è una lettera maiuscola dopo l'articolo/preposizione
                if text[len(article)].isupper():
                    text = text[len(article):]  # Rimuovi l'articolo/preposizione

        # 7. Verifica che il testo risultante sia effettivamente nel testo originale
        if text and text not in original_text:
            # Prova a trovare la corrispondenza più vicina nel testo originale
            from difflib import get_close_matches
            words = original_text.split()
            matches = get_close_matches(text, words, n=1)
            if matches:
                text = matches[0]
            else:
                # Ancora non trovato, prova una corrispondenza parziale
                potential_matches = []
                for word in words:
                    if len(word) >= 4 and (text in word or word in text):
                        potential_matches.append(word)

                if potential_matches:
                    # Scegli la corrispondenza più lunga (probabilmente la più accurata)
                    text = max(potential_matches, key=len)

        # 8. Rimuovi testo residuo non significativo
        # Se dopo tutte le correzioni, l'entità è troppo breve o è un articolo/preposizione
        stop_words = ["il", "lo", "la", "i", "gli", "le", "un", "uno", "una", "di", "a", "da", "in", "con", "su", "per", "tra", "fra"]
        if text.lower() in stop_words or len(text) < 3:
            return None

        # 9. Pulisci spazi e aggiorna l'entità
        text = text.strip()
        entity['text'] = text

        # 10. Se il testo è diventato vuoto, scarta l'entità
        if not entity['text']:
            return None

        return entity

    def process_entities(self, entities, original_text):
        """
        Applica post-processing a tutte le entità riconosciute.

        Args:
            entities: Lista di entità da correggere
            original_text: Testo originale per verifica

        Returns:
            Lista di entità corrette
        """
        print(f"DEBUG: Inizio post-processing di {len(entities)} entità")
        corrected_entities = []

        for entity in entities:
            # Applica post-processing all'entità
            corrected = self._post_process_entity(entity, original_text)
            if corrected:
                # Verifica per evitare duplicati
                if not any(e['text'] == corrected['text'] and e['label'] == corrected['label'] for e in corrected_entities):
                    corrected_entities.append(corrected)

        print(f"DEBUG: Post-processing completato, {len(corrected_entities)} entità valide rimaste")
        return corrected_entities





    def set_language(self, language: str):
        """
        Cambia la lingua del riconoscitore.

        Args:
            language: Nuova lingua da impostare
        """
        if language != self.language:
            self.language = language
            self._load_model()

    def get_supported_languages(self) -> List[str]:
        """
        Restituisce le lingue supportate.

        Returns:
            Lista di codici lingua supportati
        """
        return list(self.language_models.keys())



# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }



# Estendi la classe EntityLinkerWithSpacy per incorporare il riconoscitore italiano
class EntityLinkerItalian(EntityLinkerWithSpacy):
    """
    Versione estesa di EntityLinker ottimizzata per l'italiano.
    Integra riconoscitori multipli: Wikidata, spaCy, Stanza NLP, Transformer NER,
    Zero-Shot Classifier e WordNet.
    """

    def __init__(self, language: str = "it", use_gpu: bool = False):
        """
        Inizializza il linker di entità con supporto NLP per l'italiano.

        Args:
            language: Lingua preferita per i risultati (deve essere "it")
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        # Controlla che la lingua sia italiana
        if language != "it":
            print("Attenzione: questo riconoscitore è ottimizzato per l'italiano. Impostiamo language='it'.")
            language = "it"

        # Inizializza la classe base (con spaCy)
        super().__init__(language)

        # Aggiungi il riconoscitore italiano Stanza NLP
        self.italian_recognizer = ItalianNLPRecognizer(use_gpu=use_gpu)

        # Aggiungi il riconoscitore Transformer NER
        try:
            from transformers import AutoTokenizer, pipeline  # Verifica disponibilità di transformers
            import torch  # Verifica disponibilità di torch

            print("DEBUG: Moduli transformers e torch disponibili. Inizializzazione riconoscitore Transformer NER...")

            self.transformers_recognizer = TransformersNERRecognizer(
                model_name="5had3/bert-base-italian-cased-ner",
                language=language,
                use_gpu=use_gpu
            )

            # Verifica se il riconoscitore è stato inizializzato correttamente
            if not hasattr(self.transformers_recognizer, 'is_available') or not self.transformers_recognizer.is_available:
                print("DEBUG: Riconoscitore Transformer NER non disponibile dopo l'inizializzazione")
            else:
                print("DEBUG: Riconoscitore Transformer NER inizializzato con successo")

            print("DEBUG: Inizializzazione classificatore zero-shot...")

            # Verifica che pipeline sia disponibile
            from transformers import pipeline
            print("DEBUG: Transformers pipeline disponibile")

            # Verifica che i modelli per zero-shot siano accessibili
            print(f"DEBUG: Tentativo di accesso a modello zero-shot: facebook/bart-large-mnli")

            self.zero_shot_classifier = ZeroShotClassifier(language=language, use_gpu=use_gpu)

            if hasattr(self.zero_shot_classifier, 'is_available'):
                print(f"DEBUG: Zero-shot classifier disponibilità: {self.zero_shot_classifier.is_available}")
                if not self.zero_shot_classifier.is_available:
                    print("DEBUG: Classificatore zero-shot non disponibile dopo l'inizializzazione")
                else:
                    print(f"DEBUG: Classificatore zero-shot inizializzato con successo: {self.zero_shot_classifier.model_name}")
            else:
                print("DEBUG: L'attributo is_available non esiste nel classificatore")

        except ImportError as e:
            print(f"DEBUG: ImportError - Classificatore zero-shot non disponibile. {e}")
            print("Installare 'transformers' e 'torch' con 'pip install transformers torch'")
            self.zero_shot_classifier = None
        except Exception as e:
          print(f"DEBUG: Errore generico nell'inizializzazione del classificatore zero-shot: {e}")
          import traceback
          traceback.print_exc()
          self.zero_shot_classifier = None

        except ImportError as e:
            print(f"DEBUG: ImportError - Transformer NER non disponibile. {e}")
            print("Installare 'transformers' e 'torch' con 'pip install transformers torch'")
            self.transformers_recognizer = None
        except Exception as e:
            print(f"DEBUG: Errore generico nell'inizializzazione del riconoscitore Transformer NER: {e}")
            import traceback
            traceback.print_exc()
            self.transformers_recognizer = None

        # Inizializza il classificatore zero-shot
        try:
            print("DEBUG: Inizializzazione classificatore zero-shot...")
            self.zero_shot_classifier = ZeroShotClassifier(language=language, use_gpu=use_gpu)

            if not self.zero_shot_classifier.is_available:
                print("DEBUG: Classificatore zero-shot non disponibile")
            else:
                print("DEBUG: Classificatore zero-shot inizializzato con successo")

        except ImportError as e:
            print(f"DEBUG: ImportError - Classificatore zero-shot non disponibile. {e}")
            print("Installare 'transformers' e 'torch' con 'pip install transformers torch'")
            self.zero_shot_classifier = None
        except Exception as e:
            print(f"DEBUG: Errore generico nell'inizializzazione del classificatore zero-shot: {e}")
            import traceback
            traceback.print_exc()
            self.zero_shot_classifier = None

        # Inizializza WordNet knowledge base
        try:
            print("DEBUG: Inizializzazione WordNet knowledge base...")
            self.wordnet_kb = WordNetKnowledgeBase(language="it")

            if not self.wordnet_kb.is_available:
                print("DEBUG: WordNet knowledge base non disponibile")
            else:
                print("DEBUG: WordNet knowledge base inizializzata con successo")

        except ImportError as e:
            print(f"DEBUG: ImportError - WordNet non disponibile. {e}")
            print("Installare 'nltk' con 'pip install nltk'")
            self.wordnet_kb = None
        except Exception as e:
            print(f"DEBUG: Errore generico nell'inizializzazione di WordNet: {e}")
            import traceback
            traceback.print_exc()
            self.wordnet_kb = None


        def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando tutti i riconoscitori disponibili
        e arricchisce con WordNet e zero-shot classification.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Debug dello stato dei riconoscitori
        print(f"DEBUG: Stato riconoscitori:")
        print(f"- SpaCy: {self.wikidata.spacy_recognizer.is_available}")
        print(f"- Italian NLP: {self.italian_recognizer.is_available}")
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer:
            print(f"- Transformer NER: {self.transformers_recognizer.is_available}")
        if hasattr(self, 'zero_shot_classifier') and self.zero_shot_classifier:
            print(f"- Zero-Shot: {self.zero_shot_classifier.is_available}")
        if hasattr(self, 'wordnet_kb') and self.wordnet_kb:
            print(f"- WordNet KB: {self.wordnet_kb.is_available}")

        # Prima ottieni i risultati dalla classe base (Wikidata + spaCy)
        enriched_entities, stats = super().find_entities(text)
        existing_texts = {e.get('text', '').lower() for e in enriched_entities if 'text' in e}

        # Aggiungi le entità del riconoscitore italiano
        italian_only_entities = self._get_italian_only_entities(text, existing_texts)
        enriched_entities.extend(italian_only_entities)

        # Aggiorna il set di testi esistenti
        for entity in italian_only_entities:
            if 'text' in entity:
                existing_texts.add(entity['text'].lower())

       # Aggiungi le entità dal riconoscitore Transformer NER
        transformers_entities = []
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer and self.transformers_recognizer.is_available:
            print("DEBUG: Riconoscimento entità con Transformer NER...")
            try:
                transformers_entities = self._get_transformers_only_entities(text, existing_texts)
                print(f"DEBUG: {len(transformers_entities)} entità trovate da Transformer NER")

                # Aggiungi solo se ci sono entità
                if transformers_entities:
                    enriched_entities.extend(transformers_entities)

                    # Aggiorna il set di testi esistenti
                    for entity in transformers_entities:
                        if 'text' in entity:
                            existing_texts.add(entity['text'].lower())
            except Exception as e:
                print(f"DEBUG: Errore nel riconoscimento entità con Transformer NER: {e}")
                import traceback
                traceback.print_exc()
        else:
            print("DEBUG: Riconoscitore Transformer NER non disponibile")

        # Raccogli statistiche base
        stats_base = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': stats.get('wikidata_entities', 0),
            'spacy_only_entities': stats.get('spacy_only_entities', 0),
            'italian_only_entities': len(italian_only_entities),
            'transformers_only_entities': len(transformers_entities),
            'local_entities': stats.get('local_entities', 0),
            'entities_with_dbpedia': stats.get('entities_with_dbpedia', 0),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        # Applica classificazione zero-shot se disponibile
        if hasattr(self, 'zero_shot_classifier') and self.zero_shot_classifier and self.zero_shot_classifier.is_available:
            print("DEBUG: Applicazione classificazione zero-shot...")
            try:
                # Classifica le entità che non hanno un tipo
                enriched_entities = self.zero_shot_classifier.classify_entities(enriched_entities, text)

                # Aggiorna statistiche
                stats_base['entities_with_zero_shot'] = sum(1 for e in enriched_entities if 'zero_shot_classification' in e)

                print(f"DEBUG: {stats_base['entities_with_zero_shot']} entità classificate con zero-shot")
            except Exception as e:
                print(f"DEBUG: Errore nella classificazione zero-shot: {e}")
                traceback.print_exc()

        # Applica arricchimento semantico con WordNet se disponibile
        if hasattr(self, 'wordnet_kb') and self.wordnet_kb and self.wordnet_kb.is_available:
            print("DEBUG: Applicazione arricchimento semantico con WordNet...")
            try:
                # Arricchisci le entità con WordNet
                enriched_entities = self.wordnet_kb.enrich_entities(enriched_entities)

                # Aggiorna statistiche
                stats_base['entities_with_wordnet'] = sum(1 for e in enriched_entities if 'semantic_info' in e)

                print(f"DEBUG: {stats_base['entities_with_wordnet']} entità arricchite con WordNet")
            except Exception as e:
                print(f"DEBUG: Errore nell'arricchimento semantico con WordNet: {e}")
                traceback.print_exc()

        return enriched_entities, stats_base


    def _get_italian_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore italiano e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo dal riconoscitore italiano
        """
        # Verifica che il riconoscitore italiano sia disponibile
        if not self.italian_recognizer.is_available:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore italiano
        italian_entities = self.italian_recognizer.recognize_entities(text)
        print(f"Riconoscitore italiano ha trovato {len(italian_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        italian_only = []
        for entity in italian_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore italiano: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                italian_type = self._map_italian_label_to_type(entity['label'])

                italian_entity = {
                    'text': entity_text,
                    'types': [italian_type],
                    'source': 'italian_nlp',
                    'label': entity['label']
                }

                italian_only.append(italian_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(italian_only)} sono uniche (non trovate da altre fonti)")
        return italian_only

    def _get_transformers_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
          """
          Ottiene entità riconosciute dal riconoscitore Transformer NER.
          Includendo sia entità nuove che quelle già trovate da altre fonti.
          """
          # Verifica che il riconoscitore Transformer sia disponibile
          if not self.transformers_recognizer or not self.transformers_recognizer.is_available:
              print("DEBUG: Riconoscitore Transformer NER non disponibile.")
              return []

          # Stampa informazioni sul riconoscitore
          print(f"DEBUG: Modello Transformer NER in uso: {self.transformers_recognizer.model_name}")

          try:
              # Ottieni tutte le entità dal riconoscitore Transformer NER
              transformers_entities = self.transformers_recognizer.recognize_entities(text)

              # Debug dettagliato delle entità
              print(f"DEBUG: Numero di entità Transformer NER trovate: {len(transformers_entities)}")

              # Filtra e mappa le entità - IMPORTANTE: non filtriamo per existing_texts
              # per assicurarci che tutte le entità Transformer siano incluse
              transformers_mapped = []
              for entity in transformers_entities:
                  if self._is_valid_entity(entity.get('text', '')):
                      # Aggiungi sempre il source 'transformers_ner'
                      mapped_entity = {
                          'text': entity.get('text', ''),
                          'label': entity.get('label', ''),
                          'types': [entity.get('label', 'N/A')],
                          'source': 'transformers_ner',
                          'score': entity.get('score', 0)
                      }
                      transformers_mapped.append(mapped_entity)

              print(f"DEBUG: Numero di entità Transformer NER mappate: {len(transformers_mapped)}")
              return transformers_mapped

          except Exception as e:
              print(f"DEBUG: Errore nell'ottenere entità Transformer: {e}")
              import traceback
              traceback.print_exc()
              return []

    def _map_italian_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore italiano a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        mapping = {
            'PERSON': 'Person',
            'PER': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'MISC': 'Miscellaneous',
            'PROPN': 'ProperNoun',
            'WORK_OF_ART': 'Artwork',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def _map_transformers_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore Transformer NER a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        # (simile a quella di altri riconoscitori)
        mapping = {
            'PER': 'Person',
            'LOC': 'Location',
            'ORG': 'Organization',
            'MISC': 'Miscellaneous',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'PERSON': 'Person',
            'ORGANIZATION': 'Organization',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estende il metodo della classe base per includere i risultati
        del riconoscitore italiano e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        # Ottieni i risultati di base
        uri_results = super().verify_uris_for_text(text)

        # Aggiungi una sezione per le entità del riconoscitore italiano senza URI
        italian_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'italian_nlp' and entity['text'] not in uri_results['results']:
                italian_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        # Aggiungi una sezione per le entità del riconoscitore Transformer NER senza URI
        transformers_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'transformers_ner' and entity['text'] not in uri_results['results']:
                transformers_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', []),
                    'score': entity.get('score', 0)
                }

        # Aggiungi le sezioni ai risultati
        uri_results['italian_results'] = italian_results
        uri_results['transformers_results'] = transformers_results

        return uri_results


# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

if __name__ == "__main__":
    # Importazioni necessarie
    import os
    import sys
    import traceback

    # Imposta il livello di log per transformers
    os.environ["TRANSFORMERS_VERBOSITY"] = "info"

    # Verifica le dipendenze
    try:
        import torch
        import transformers
        print(f"PyTorch versione: {torch.__version__}")
        print(f"Transformers versione: {transformers.__version__}")
        print(f"CUDA disponibile: {torch.cuda.is_available()}")
    except ImportError as e:
        print(f"ATTENZIONE: Libreria mancante - {e}")

    try:
        # Crea un'istanza del linker di entità migliorato con debug aggiuntivo
        print("Inizializzazione EntityLinkerItalian...")
        linker = EntityLinkerItalian(language="it", use_gpu=False)

        # Verifica disponibilità dei riconoscitori
        print(f"SpaCy riconoscitore disponibile: {linker.wikidata.spacy_recognizer.is_available}")
        print(f"Italian NLP riconoscitore disponibile: {linker.italian_recognizer.is_available}")
        if hasattr(linker, 'transformers_recognizer') and linker.transformers_recognizer:
            print(f"Transformer NER riconoscitore disponibile: {linker.transformers_recognizer.is_available}")
            print(f"Transformer NER modello: {linker.transformers_recognizer.model_name}")
        else:
            print("Transformer NER riconoscitore non disponibile")

        # Testo di esempio
        text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
               " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
               " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
               " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
               " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
               " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
               " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
               " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
               " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
               " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

        print(f"Analisi del testo:\n{text}\n")

        # Test esplicito del riconoscitore Transformer NER, se disponibile
        if hasattr(linker, 'transformers_recognizer') and linker.transformers_recognizer and linker.transformers_recognizer.is_available:
            print("\nTest diretto del riconoscitore Transformer NER:")
            test_sample = "Mario Rossi vive a Roma e lavora per l'Università La Sapienza."
            try:
                transformer_test_entities = linker.transformers_recognizer.recognize_entities(test_sample)
                print(f"Entità trovate dal riconoscitore Transformer in test: {len(transformer_test_entities)}")
                for entity in transformer_test_entities:
                    print(f"- '{entity['text']}' → Etichetta: {entity.get('label')}, Score: {entity.get('score')}")
            except Exception as e:
                print(f"Errore nel test diretto del riconoscitore Transformer: {e}")
                traceback.print_exc()

        # Trova entità
        print("\nRicerca entità nel testo principale...")
        entities, stats = linker.find_entities(text)

        # Verifica entità Transformer
        transformer_entities = [e for e in entities if e.get('source') == 'transformers_ner']
        print(f"\nEntità Transformer NER trovate: {len(transformer_entities)}")
        for entity in transformer_entities[:10]:  # Mostra solo le prime 10 per brevità
            print(f"- '{entity['text']}' → Etichetta: {entity.get('label')}, Score: {entity.get('score', 'N/A')}")

        print(f"\nTrovate {stats['total_entities']} entità totali:")
        for k, v in stats.items():
            print(f"- {k}: {v}")

        # Esporta le entità in un file
        print("\nEsportazione entità in file...")
        output_file_path = export_entities_to_txt(
            entities,
            stats,
            "entita_complete.txt",  # Nuovo nome file per distinguerlo dal precedente
            spacy_recognizer=linker.wikidata.spacy_recognizer,
            italian_recognizer=linker.italian_recognizer,
            transformers_recognizer=linker.transformers_recognizer,
            original_text=text
        )

        print(f"\nFile di output salvato in: {output_file_path}")

        # Verifica il file di output
        import os
        if os.path.exists(output_file_path):
            file_size = os.path.getsize(output_file_path)
            print(f"Verifica file: '{output_file_path}' esiste e ha dimensione {file_size} bytes")

            # Leggi il file per verificare il contenuto
            with open(output_file_path, 'r', encoding='utf-8') as f:
                content = f.read()

            # Cerca la sezione Transformer nel file
            if "TRANSFORMER NER ENTITIES:" in content:
                transformer_section = content.split("TRANSFORMER NER ENTITIES:")[1].split("\n\n")[0]
                transformer_lines = [line for line in transformer_section.split("\n") if line.strip() and "-----" not in line]
                print(f"\nSezione Transformer nel file di output contiene {len(transformer_lines)} righe")
                if len(transformer_lines) > 1:  # La prima riga è l'intestazione
                    print("Le prime 5 entità Transformer nel file:")
                    for line in transformer_lines[:6]:
                        print(f"  {line}")
                else:
                    print("ATTENZIONE: Nessuna entità Transformer trovata nel file di output")
            else:
                print("ATTENZIONE: Sezione Transformer non trovata nel file di output")
        else:
            print(f"ATTENZIONE: Il file '{output_file_path}' non è stato creato!")

    except Exception as e:
        print(f"Errore generale: {e}")
        traceback.print_exc()

IndentationError: expected an indented block after function definition on line 1958 (<ipython-input-11-4e12beceb7d2>, line 1959)

variazione ultima

In [12]:
import requests
import json
import time
import re
from typing import List, Dict, Any, Optional, Tuple, Set
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict, Any, Optional, Tuple, Set # Added missing Any type hint
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Open Multilingual WordNet

# Importazione di spaCy
import spacy
# Modifica alla funzione export_entities_to_txt per assicurare la corretta inclusione delle entità spaCy
# Modifica alla funzione export_entities_to_txt per includere le entità del riconoscitore italiano
#def export_entities_to_txt(entities, stats, output_file="entita_trovate.txt", spacy_recognizer=None, italian_recognizer=None, original_text=None):

def export_entities_to_txt(
    entities,
    stats,
    output_file="entita.txt",
    spacy_recognizer=None,
    italian_recognizer=None,
    transformers_recognizer=None,
    zero_shot_classifier=None,  # Nuovo parametro
    wordnet_kb=None,            # Nuovo parametro
    original_text=None
    ):
    """
    Esporta le entità trovate in un file di testo semplice con le categorie richieste.
    Include anche le entità riconosciute da tutti i riconoscitori e gli arricchitori.

    Args:
        entities: Lista di entità trovate
        stats: Statistiche sulle entità
        output_file: Nome del file di output
        spacy_recognizer: Riconoscitore spaCy (opzionale)
        italian_recognizer: Riconoscitore italiano (opzionale)
        transformers_recognizer: Riconoscitore Transformer NER (opzionale)
        zero_shot_classifier: Classificatore zero-shot (opzionale)
        wordnet_kb: Knowledge base WordNet (opzionale)
        original_text: Testo originale da analizzare (opzionale)

    Returns:
        Percorso completo del file creato
    """
    import os

    # Filtra le entità per categoria
    wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
    entities_with_dbpedia = [e for e in entities if 'dbpedia_uri' in e and e['dbpedia_uri']]
    entities_with_types = [e for e in entities if 'types' in e and e['types']]
    spacy_entities = [e for e in entities if e.get('source') == 'spacy']
    italian_entities = [e for e in entities if e.get('source') in ['italian_nlp', 'stanza', 'stanza_pos', 'geo_rule']]
    transformers_entities = [e for e in entities if e.get('source') == 'transformers_ner']
    zero_shot_entities = [e for e in entities if 'zero_shot_classification' in e]
    wordnet_entities = [e for e in entities if 'semantic_info' in e]

    # Prepara i contenuti
    lines = []

    # Intestazione
    lines.append("ELENCO ENTITÀ TROVATE NEL TESTO")
    lines.append("============================\n")

    # Entità Wikidata
    lines.append("WIKIDATA ENTITIES:")
    lines.append("------------------")
    for entity in entities:
        if entity.get('source') in ['wikidata', 'wikidata+spacy']:
            lines.append(f"- '{entity['text']}' → {entity.get('wikidata_id', 'N/A')} (URL: {entity.get('wikidata_url', 'N/A')})")
    lines.append("")

    # Entità con URI DBpedia
    lines.append("ENTITIES WITH DBPEDIA:")
    lines.append("----------------------")
    for entity in entities_with_dbpedia:
        lines.append(f"- '{entity['text']}' → {entity['dbpedia_uri']}")
    lines.append("")

    # Entità con tipi definiti
    lines.append("ENTITIES WITH TYPES:")
    lines.append("-------------------")
    for entity in entities_with_types:
        types_str = ", ".join(entity.get('types', []))
        lines.append(f"- '{entity['text']}' → Tipi: {types_str}")
    lines.append("")

    # Sezione entità spaCy
    lines.append("SPACY ENTITIES:")
    lines.append("--------------")

    # Se abbiamo un riconoscitore spaCy e un testo originale, ottieni tutte le entità spaCy
    if spacy_recognizer and original_text:
        try:
            spacy_all_entities = spacy_recognizer.recognize_entities(original_text)

            if not spacy_all_entities:
                lines.append("Nessuna entità rilevata da spaCy nel testo.")
            else:
                for entity in spacy_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi spaCy: {e}")
    else:
        # Fallback alle entità spaCy già filtrate
        if not spacy_entities:
            lines.append("Per mostrare tutte le entità spaCy, è necessario passare il riconoscitore spaCy e il testo originale.")
        else:
            for entity in spacy_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore italiano
    lines.append("ITALIAN NLP ENTITIES:")
    lines.append("--------------------")

    # Se abbiamo un riconoscitore italiano e un testo originale, ottieni tutte le entità
    if italian_recognizer and original_text:
        try:
            italian_all_entities = italian_recognizer.recognize_entities(original_text)

            if not italian_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore italiano nel testo.")
            else:
                for entity in italian_all_entities:
                    label = entity.get('label', '')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore italiano: {e}")
    else:
        # Fallback alle entità italiane già filtrate
        if not italian_entities:
            lines.append("Per mostrare tutte le entità del riconoscitore italiano, è necessario passare il riconoscitore e il testo originale.")
        else:
            for entity in italian_entities:
                label = entity.get('label', '')
                types_str = ", ".join(entity.get('types', []))
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Tipi: {types_str}")
    lines.append("")

    # Sezione entità del riconoscitore Transformer NER
    lines.append("TRANSFORMER NER ENTITIES:")
    lines.append("-------------------------")

    # Se abbiamo un riconoscitore Transformer e un testo originale, ottieni tutte le entità
    if transformers_recognizer and original_text and hasattr(transformers_recognizer, 'recognize_entities'):
        try:
            print("DEBUG: Tentativo di ottenere entità dal riconoscitore Transformer")
            transformers_all_entities = transformers_recognizer.recognize_entities(original_text)

            if not transformers_all_entities:
                lines.append("Nessuna entità rilevata dal riconoscitore Transformer NER.")
            else:
                print(f"DEBUG: Trovate {len(transformers_all_entities)} entità da Transformer")
                # Rimuovi duplicati mantenendo l'ordine originale
                unique_transformers_entities = []
                seen = set()
                for entity in transformers_all_entities:
                    if entity['text'] not in seen:
                        unique_transformers_entities.append(entity)
                        seen.add(entity['text'])

                for entity in unique_transformers_entities:
                    # Usa il primo tipo disponibile o 'N/A'
                    label = entity.get('label', entity.get('types', ['N/A'])[0] if entity.get('types') else 'N/A')
                    score = entity.get('score', 'N/A')
                    lines.append(f"- '{entity['text']}' → Etichetta: {label}, Punteggio: {score}")
        except Exception as e:
            lines.append(f"Errore nell'analisi del riconoscitore Transformer: {e}")
            print(f"DEBUG: Errore nell'ottenere entità Transformer: {e}")
    else:
        # Fallback alle entità Transformer già filtrate nel metodo find_entities
        if not transformers_entities:
            if not transformers_recognizer:
                lines.append("Riconoscitore Transformer NER non disponibile.")
            elif not hasattr(transformers_recognizer, 'recognize_entities'):
                lines.append("Il riconoscitore Transformer NER non ha il metodo recognize_entities.")
            else:
                lines.append("Per mostrare tutte le entità del riconoscitore Transformer, è necessario passare il riconoscitore e il testo originale.")
        else:
            print(f"DEBUG: Usando {len(transformers_entities)} entità Transformer già filtrate")
            for entity in transformers_entities:
                label = entity.get('label', 'N/A')
                score = entity.get('score', 'N/A')
                lines.append(f"- '{entity['text']}' → Etichetta: {label}, Punteggio: {score}")
    lines.append("")

    # NUOVA SEZIONE: Entità classificate con Zero-Shot
    lines.append("ZERO-SHOT CLASSIFIED ENTITIES:")
    lines.append("------------------------------")
    if zero_shot_entities:
        for entity in zero_shot_entities:
            classification = entity.get('zero_shot_classification', {})
            label = classification.get('label', 'N/A')
            score = classification.get('score', 'N/A')
            lines.append(f"- '{entity['text']}' → Classe: {label}, Confidenza: {score:.4f}")

            # Mostra anche altri punteggi se disponibili
            scores = classification.get('scores', {})
            if scores and len(scores) > 1:
                lines.append(f"  Altri punteggi:")
                for score_label, score_value in scores.items():
                    if score_label != label:  # Salta l'etichetta principale
                        lines.append(f"  - {score_label}: {score_value:.4f}")
    else:
        lines.append("Nessuna entità classificata con Zero-Shot.")
    lines.append("")

    # NUOVA SEZIONE: Entità arricchite con WordNet
    lines.append("WORDNET SEMANTIC ENTITIES:")
    lines.append("-------------------------")
    if wordnet_entities:
        for entity in wordnet_entities:
            semantic_info = entity.get('semantic_info', {})
            lines.append(f"- '{entity['text']}':")

            # Definizioni
            definitions = semantic_info.get('definitions', [])
            if definitions:
                lines.append(f"  Definizioni:")
                for i, definition in enumerate(definitions[:2]):  # Mostra solo le prime 2 definizioni
                    lines.append(f"  {i+1}. {definition}")

            # Sinonimi
            synonyms = semantic_info.get('synonyms', [])
            if synonyms:
                lines.append(f"  Sinonimi: {', '.join(synonyms[:5])}")  # Mostra solo i primi 5 sinonimi

            # Iperonimi (concetti più generali)
            hypernyms = semantic_info.get('hypernyms', [])
            if hypernyms:
                lines.append(f"  Concetti generali:")
                for i, hypernym in enumerate(hypernyms[:2]):  # Mostra solo i primi 2 iperonimi
                    lines.append(f"  - {hypernym.get('name', 'N/A')}")
    else:
        lines.append("Nessuna entità arricchita con WordNet.")
    lines.append("")

    # Scrivi il file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))

    # Ottieni e restituisci il percorso completo
    output_path = os.path.abspath(output_file)
    print(f"File salvato in: {output_path}")

    return output_path


class SpacyRecognizer:
    """
    Riconoscitore di entità basato su spaCy.
    Riconosce entità in testo italiano e altre lingue supportate.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il riconoscitore spaCy.

        Args:
            language: Lingua del testo da analizzare (it, en, ecc.)
        """
        self.language = language
        self.model = None
        self.is_available = False

        # Mappa codici lingua a modelli spaCy
        self.language_models = {
            "it": "it_core_news_lg",
            "en": "en_core_web_lg",
            "fr": "fr_core_news_lg",
            "de": "de_core_news_lg",
            "es": "es_core_news_lg"
        }

        # Inizializza il modello appropriato
        self._load_model()

    def _load_model(self):
        """Carica il modello spaCy per la lingua specificata."""
        try:
            # Ottieni il nome del modello per la lingua specificata
            model_name = self.language_models.get(self.language)

            if not model_name:
                print(f"Lingua {self.language} non supportata da spaCy. Utilizzo del modello inglese.")
                model_name = "en_core_web_sm"

            # Carica il modello
            self.model = spacy.load(model_name)
            self.is_available = True
            print(f"Modello spaCy {model_name} caricato con successo.")
        except Exception as e:
            print(f"Errore nel caricamento del modello spaCy: {e}")
            print("Assicurati di aver installato spaCy e i modelli linguistici con:")
            print(f"python -m spacy download {self.language_models.get(self.language, 'en_core_web_sm')}")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo utilizzando spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.model:
            print("SpacyRecognizer non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con spaCy
            doc = self.model(text)

            # Estrai le entità
            for ent in doc.ents:
                entity = {
                    'text': ent.text,
                    'label': ent.label_,
                    'start_char': ent.start_char,
                    'end_char': ent.end_char,
                    'source': 'spacy'
                }
                entities.append(entity)

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con spaCy: {e}")

        return entities


class ItalianNLPRecognizer:
    """
    Riconoscitore di entità ottimizzato per l'italiano in ambiente Colab.
    Usa una combinazione di Stanza (versione Python di Stanford CoreNLP)
    e altre librerie per l'italiano.
    """

    def __init__(self, use_gpu: bool = False):
        """
        Inizializza il riconoscitore per l'italiano.

        Args:
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        self.use_gpu = use_gpu
        self.is_available = False
        self.stanza_nlp = None
        self.initialized = False

        # Inizializza le risorse
        self._initialize_resources()

    def _initialize_resources(self):
        """Inizializza Stanza e altre risorse per l'italiano."""
        try:
            import stanza
            import simplemma
            import nltk
            from nltk.tokenize import word_tokenize

            # Scarica i modelli di Stanza per l'italiano
            stanza.download('it')

            # Inizializza il pipeline Stanza per l'italiano
            self.stanza_nlp = stanza.Pipeline(
                lang='it',
                processors='tokenize,mwt,pos,lemma,ner',
                use_gpu=self.use_gpu
            )

            # Scarica le risorse NLTK necessarie
            nltk.download('punkt')
            nltk.download('stopwords')
            nltk.download('averaged_perceptron_tagger')

            self.is_available = True
            self.initialized = True
            print("Riconoscitore NLP italiano inizializzato correttamente con Stanza.")

        except Exception as e:
            print(f"Errore nell'inizializzazione del riconoscitore italiano: {e}")
            print("Assicurati di aver eseguito le installazioni necessarie in Colab.")
            self.is_available = False

    def recognize_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Riconosce entità in un testo italiano utilizzando Stanza.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.stanza_nlp:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        entities = []

        try:
            # Analizza il testo con Stanza
            doc = self.stanza_nlp(text)

            # Estrai le entità
            for sent in doc.sentences:
                for ent in sent.ents:
                    entity = {
                        'text': ent.text,
                        'label': ent.type,
                        'start_char': -1,  # Stanza non fornisce direttamente le posizioni dei caratteri
                        'end_char': -1,
                        'source': 'stanza'
                    }
                    entities.append(entity)

                # Aggiungi anche i nomi propri (POS tag == PROPN)
                for token in sent.tokens:
                    for word in token.words:
                        if word.upos == 'PROPN' and not any(e['text'] == word.text for e in entities):
                            entity = {
                                'text': word.text,
                                'label': 'PROPN',
                                'start_char': -1,
                                'end_char': -1,
                                'source': 'stanza_pos'
                            }
                            entities.append(entity)

            # Cerca anche entità geografiche italiane specifiche usando regole
            entities.extend(self._extract_italian_geo_entities(text))

        except Exception as e:
            print(f"Errore nel riconoscimento delle entità con Stanza: {e}")

        return entities

    def _extract_italian_geo_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Estrae entità geografiche italiane usando regole.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità geografiche
        """
        import re
        entities = []

        # Regioni italiane
        regions = [
            "Abruzzo", "Basilicata", "Calabria", "Campania", "Emilia-Romagna",
            "Friuli Venezia Giulia", "Lazio", "Liguria", "Lombardia", "Marche",
            "Molise", "Piemonte", "Puglia", "Sardegna", "Sicilia", "Toscana",
            "Trentino-Alto Adige", "Umbria", "Valle d'Aosta", "Veneto"
        ]

        # Pattern per laghi, montagne, fiumi italiani
        geo_patterns = [
            r'\b(?:Lago|Monte|Fiume|Isola|Golfo|Capo|Valle|Parco Nazionale|Mar) [A-Z][a-zàèéìòù]+\b',
            r'\b(?:Alpi|Appennini|Dolomiti|Maremma|Laguna|Costa|Riviera) [A-Z][a-zàèéìòù]*\b'
        ]

        # Cerca regioni
        for region in regions:
            if re.search(r'\b' + re.escape(region) + r'\b', text):
                entity = {
                    'text': region,
                    'label': 'GPE',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        # Cerca pattern geografici
        for pattern in geo_patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                entity = {
                    'text': match.group(0),
                    'label': 'LOC',
                    'start_char': -1,
                    'end_char': -1,
                    'source': 'geo_rule'
                }
                entities.append(entity)

        return entities


# Alternative per modelli italiani NER
model_options = [
    "5had3/bert-base-italian-cased-ner",  # Un'alternativa affidabile
    "MilaNLProc/bert-italian-cased-ner",  # Altro modello italiano per NER
    "dslim/bert-base-NER"  # Modello generico che funziona bene
]


class TransformersNERRecognizer:
    def __init__(self,
                 model_name: str = None,
                 language: str = "it",
                 use_gpu: bool = False):
        """
        Inizializza il riconoscitore di entità con un modello Transformer.

        Args:
            model_name: Nome del modello pre-addestrato per NER
            language: Lingua del modello (default: italiano)
            use_gpu: Se utilizzare l'accelerazione GPU
        """
        self.language = language
        self.use_gpu = use_gpu

        # Dizionario di mapping per modelli in diverse lingue
        self.language_models = {
            "it": [


                "dbmdz/bert-base-italian-xxl-cased",        # Modello italiano grande
                "Babelscape/wikineural-multilingual-ner",   # Ottimo per NER multilingua
                "Davlan/xlm-roberta-base-ner-hrl",          # Buon supporto per italiano
                "MilaNLProc/bert-italian-cased-ner",        # Altra alternativa italiana
                "5had3/bert-base-italian-cased-ner",        # Modello che dava problemi - spostato in fondo
                "dslim/bert-base-NER"

            ],
            "en": ["dslim/bert-base-NER"],
            "de": ["RafLorem/bert-base-german-NER"],
            "fr": ["Jean-Baptiste/camembert-ner"]
        }

        # Lista di fallback globale (modelli che funzionano in molte lingue)
        self.fallback_models = [
            "dslim/bert-base-NER",               # Ottimo supporto multilingua
            "dbmdz/bert-large-cased-finetuned-conll03-english",  # Molto robusto
            "Babelscape/wikineural-multilingual-ner"  # Supporto multilingua avanzato
        ]

        # Se non specificato, scegli un modello predefinito
        if model_name is None:
            model_name = self._select_best_model()

        self.model_name = model_name
        self.is_available = False
        self.ner_pipeline = None

        # Inizializza il modello
        self._load_model()

    def _select_best_model(self) -> str:
        """
        Seleziona il miglior modello disponibile per la lingua.

        Returns:
            Nome del modello
        """
        print(f"DEBUG: Selezionando modello per lingua: {self.language}")

        # Prova prima i modelli specifici per la lingua
        models = self.language_models.get(self.language, self.language_models.get("en", []))
        print(f"DEBUG: Proverò questi modelli nell'ordine: {models}")

        for model in models:
            try:
                # Importa solo il tokenizer per verificare la disponibilità
                from transformers import AutoTokenizer
                print(f"DEBUG: Verifica disponibilità modello: {model}")

                # Verifica la disponibilità con caricamento offline
                tokenizer = AutoTokenizer.from_pretrained(model, local_files_only=False)
                print(f"DEBUG: Modello {model} disponibile e selezionato")
                return model
            except Exception as e:
                print(f"DEBUG: Modello {model} non disponibile: {str(e)}")
                continue

        # Se nessun modello specifico per lingua funziona, prova i fallback
        print("DEBUG: Nessun modello specifico per lingua disponibile, provo fallback globali")
        for fallback in self.fallback_models:
            try:
                from transformers import AutoTokenizer
                print(f"DEBUG: Provo fallback modello: {fallback}")
                tokenizer = AutoTokenizer.from_pretrained(fallback, local_files_only=False)
                print(f"DEBUG: Fallback {fallback} disponibile e selezionato")
                return fallback
            except Exception as e:
                print(f"DEBUG: Fallback {fallback} non disponibile: {str(e)}")
                continue

        # Ultimo tentativo: dslim/bert-base-NER (molto comune)
        print("DEBUG: Ultimo tentativo con dslim/bert-base-NER")
        return "dslim/bert-base-NER"

    def _load_model(self):
        """
        Carica il modello Transformer per il riconoscimento di entità.
        """
        try:
            # Assicurati che le dipendenze siano disponibili
            import torch
            from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

            print(f"DEBUG: Tentativo di caricare il modello: {self.model_name}")

            # Determina il device
            device = 0 if self.use_gpu and torch.cuda.is_available() else -1
            print(f"DEBUG: Usando device {'GPU' if device == 0 else 'CPU'}")

            # Prova a caricare con questo modello specifico
            try:
                # Carica tokenizer e modello
                tokenizer = AutoTokenizer.from_pretrained(self.model_name)
                model = AutoModelForTokenClassification.from_pretrained(self.model_name)

                # Crea pipeline NER
                self.ner_pipeline = pipeline(
                    "ner",
                    model=model,
                    tokenizer=tokenizer,
                    device=device
                )

                self.is_available = True
                print(f"DEBUG: Modello {self.model_name} caricato con successo!")

                # Test rapido per verificare il funzionamento
                test_text = "Mario Rossi vive a Roma."
                test_result = self.ner_pipeline(test_text)
                print(f"DEBUG: Test modello superato - entità trovate: {len(test_result)}")
                return

            except Exception as model_error:
                print(f"DEBUG: Errore nel caricamento del modello specifico: {str(model_error)}")

                # Se il modello specifico fallisce, prova i modelli di fallback dalla lista
                for fallback in self.fallback_models:
                    if fallback != self.model_name:  # Evita di riprovare lo stesso modello
                        try:
                            print(f"DEBUG: Provo modello di fallback: {fallback}")
                            tokenizer = AutoTokenizer.from_pretrained(fallback)
                            model = AutoModelForTokenClassification.from_pretrained(fallback)

                            self.ner_pipeline = pipeline(
                                "ner",
                                model=model,
                                tokenizer=tokenizer,
                                device=device
                            )

                            self.is_available = True
                            self.model_name = fallback  # Aggiorna il nome del modello
                            print(f"DEBUG: Modello fallback {fallback} caricato con successo!")
                            return

                        except Exception as fallback_error:
                            print(f"DEBUG: Fallback {fallback} fallito: {str(fallback_error)}")

                # Se tutti i fallback falliscono, solleva l'errore originale
                raise model_error

        except ImportError as e:
            print(f"DEBUG: Errore di importazione: {str(e)}")
            print("È necessario installare 'transformers' e 'torch':")
            print("pip install transformers torch")
            self.is_available = False

        except Exception as e:
            print(f"DEBUG: Errore generale nel caricamento del modello: {str(e)}")
            import traceback
            traceback.print_exc()
            self.is_available = False

    def recognize_entities(self, text: str) -> list:
        """
        Riconosce entità in un testo utilizzando un modello Transformer,
        con miglioramenti alla tokenizzazione per l'italiano.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità riconosciute
        """
        if not self.is_available or not self.ner_pipeline:
            print("DEBUG: Riconoscitore Transformer NER non disponibile.")
            return []

        entities = []

        try:
            print(f"DEBUG: Analisi testo con modello {self.model_name}")

            # Strategia 1: Analisi parola per parola (per migliorare la tokenizzazione)
            # Dividere il testo in frasi per mantenere un po' di contesto
            import re
            sentences = re.split(r'[.!?]\s+', text)

            print(f"DEBUG: Testo diviso in {len(sentences)} frasi")
            all_ner_results = []

            for sentence_idx, sentence in enumerate(sentences):
                # Calcola l'offset di caratteri per questa frase nel testo originale
                sentence_offset = text.find(sentence)
                if sentence_offset == -1:  # Se non trova esattamente la frase (a causa di spazi, ecc.)
                    # Fai un'approssimazione basata sulle frasi precedenti
                    sentence_offset = sum(len(s) + 2 for s in sentences[:sentence_idx])  # +2 per punteggiatura e spazio

                print(f"DEBUG: Analisi frase {sentence_idx+1}/{len(sentences)}")

                # Divide la frase in insiemi di parole (per mantenere un po' di contesto)
                words = sentence.split()

                # Analizza la frase in piccoli gruppi di parole per mantenere contesto ma migliorare tokenizzazione
                window_size = 3  # Numero di parole da analizzare insieme
                for i in range(0, len(words), window_size):
                    word_group = ' '.join(words[i:i+window_size])

                    # Skip gruppi vuoti
                    if not word_group.strip():
                        continue

                    # Calcola offset all'interno della frase
                    word_group_offset = sentence.find(word_group)
                    if word_group_offset == -1:
                        # Approssimazione se non trova esattamente
                        word_group_offset = sum(len(w) + 1 for w in words[:i])

                    # Offset totale nel testo originale
                    total_offset = sentence_offset + word_group_offset

                    try:
                        # Analizza questo gruppo di parole
                        group_results = self.ner_pipeline(word_group)

                        # Aggiungi l'offset al risultato
                        for result in group_results:
                            result['start'] += total_offset
                            result['end'] += total_offset
                            # Verifica che il contenuto corrisponda al testo originale
                            expected_text = text[result['start']:result['end']]
                            if expected_text != result['word']:
                                print(f"DEBUG: Mismatch - tokenizer ha '{result['word']}' ma il testo ha '{expected_text}'")
                                # Correzione
                                result['word'] = expected_text

                        all_ner_results.extend(group_results)
                    except Exception as e:
                        print(f"DEBUG: Errore nell'analisi del gruppo di parole '{word_group}': {e}")

            print(f"DEBUG: Totale token di entità trovati: {len(all_ner_results)}")

            # Ordinamento per posizione per assicurare la sequenza corretta
            all_ner_results.sort(key=lambda x: x['start'])

            # Raggruppa entità consecutive con lo stesso tipo
            grouped_entities = []
            current_entity = None

            # Fix per gestire entità sovrapposte - usa una strategia differente
            i = 0
            while i < len(all_ner_results):
                result = all_ner_results[i]
                # Ignora token classificati come "O" (Other)
                if result['entity'] == 'O':
                    i += 1
                    continue

                # Estrai tipo di entità (rimuovi prefissi B- e I-)
                entity_type = result['entity']
                if entity_type.startswith('B-') or entity_type.startswith('I-'):
                    entity_type = entity_type[2:]

                # Inizia una nuova entità
                start_pos = result['start']
                end_pos = result['end']
                entity_text = result['word']
                entity_score = result['score']

                # Cerca token consecutivi dello stesso tipo
                j = i + 1
                while j < len(all_ner_results):
                    next_result = all_ner_results[j]
                    next_type = next_result['entity']
                    if next_type.startswith('B-') or next_type.startswith('I-'):
                        next_type = next_type[2:]

                    # Se è lo stesso tipo ed è adiacente o vicino
                    if next_type == entity_type and next_result['start'] <= end_pos + 3:  # +3 per gestire spazi
                        # Estendi l'entità
                        # Gestisci lo spazio tra token
                        if next_result['start'] > end_pos:
                            # C'è uno spazio tra i token
                            entity_text += ' ' + next_result['word']
                        else:
                            # Token adiacenti o sovrapposti
                            entity_text += next_result['word'][end_pos - next_result['start']:]

                        end_pos = next_result['end']
                        entity_score = max(entity_score, next_result['score'])
                        j += 1
                    else:
                        break

                # Crea l'entità completa
                full_entity = {
                    'text': entity_text,
                    'label': entity_type,
                    'start_char': start_pos,
                    'end_char': end_pos,
                    'score': entity_score,
                    'source': 'transformers_ner'
                }

                # Post-processing per correggere errori comuni nei modelli italiani
                full_entity = self._post_process_entity(full_entity, text)

                # Aggiungi l'entità se non vuota dopo post-processing
                if full_entity and full_entity['text'].strip():
                    grouped_entities.append(full_entity)

                # Avanza all'indice dopo l'ultimo token usato in questa entità
                i = j

            print(f"DEBUG: Raggruppate in {len(grouped_entities)} entità")
            return grouped_entities

        except Exception as e:
            print(f"DEBUG: Errore nel riconoscimento entità: {str(e)}")
            import traceback
            traceback.print_exc()
            return []



    def _post_process_entity(self, entity, original_text):
        """
        Applica correzioni post-processing alle entità riconosciute.

        Args:
            entity: Entità da correggere
            original_text: Testo originale per verifica

        Returns:
            Entità corretta o None se l'entità deve essere scartata
        """
        text = entity['text']

        # 1. Correzioni specifiche per errori di tokenizzazione italiani
        corrections = {
            "Romaoss": "Roma",
            "Milanoite": "Milano",
            "Europaolo": "Europa",
            "Barolog": "Barolo",
            "Fiorentina": "Firenze",
            "Sa Polite": "",  # Rimuovi completamente
            "Rinascimentolo": "Rinascimento",
            "Vesuviana": "Vesuvio",
            "Napoli Sa": "Napoli",
            "Polite Milano": "Milano",
            "Milanoite Milano": "Milano",
            "Mediterraneano": "Mediterraneo",
            "Lamborghinini": "Lamborghini"
        }

        # Applica correzioni esatte
        for wrong, correct in corrections.items():
            if wrong in text:
                text = text.replace(wrong, correct)

        # 2. Correggi tokenizzazioni errate conosciute (suffissi e pattern comuni)
        if "oss" in text:
            text = text.replace("oss", "")

        if "aolo" in text and "Europa" in text:
            text = "Europa"

        if "ite" in text and any(city in text for city in ["Milano", "Napoli", "Roma"]):
            for city in ["Milano", "Napoli", "Roma"]:
                if city in text:
                    text = city
                    break

        # 3. Rimuovi suffissi comuni errati
        suffixes = ["oss", "olog", "ologna", "oni", "olino", "aolo", "ite", "inte"]
        for suffix in suffixes:
            if text.endswith(suffix):
                text = text[:-len(suffix)]

        # 4. Gestisci entità con lettere ripetute erroneamente
        if any(c*3 in text for c in "abcdefghijklmnopqrstuvwxyz"):
            # Troppi caratteri ripetuti, probabilmente un errore
            return None

        # 5. Separa entità erroneamente unite
        # Esempio: "NapoliMilano" -> "Napoli"
        cities = ["Roma", "Milano", "Napoli", "Firenze", "Venezia", "Torino", "Bologna", "Palermo"]
        for i, city1 in enumerate(cities):
            for city2 in cities[i+1:]:
                if city1 in text and city2 in text and len(text) < len(city1) + len(city2) + 3:
                    # Probabilmente città unite erroneamente - prendi la prima
                    if text.find(city1) < text.find(city2):
                        text = city1
                    else:
                        text = city2
                    break

        # 6. Gestisci casi specifici di entità concatenate con articoli o preposizioni
        # Es: "LaFerrari" -> "Ferrari"
        articles_preps = ["La", "Il", "Lo", "Gli", "Le", "Di", "Del", "Della", "Dello", "Dei", "Degli", "Delle"]
        for article in articles_preps:
            if text.startswith(article) and len(article) + 1 < len(text):
                # Verifica se c'è una lettera maiuscola dopo l'articolo/preposizione
                if text[len(article)].isupper():
                    text = text[len(article):]  # Rimuovi l'articolo/preposizione

        # 7. Verifica che il testo risultante sia effettivamente nel testo originale
        if text and text not in original_text:
            # Prova a trovare la corrispondenza più vicina nel testo originale
            from difflib import get_close_matches
            words = original_text.split()
            matches = get_close_matches(text, words, n=1)
            if matches:
                text = matches[0]
            else:
                # Ancora non trovato, prova una corrispondenza parziale
                potential_matches = []
                for word in words:
                    if len(word) >= 4 and (text in word or word in text):
                        potential_matches.append(word)

                if potential_matches:
                    # Scegli la corrispondenza più lunga (probabilmente la più accurata)
                    text = max(potential_matches, key=len)

        # 8. Rimuovi testo residuo non significativo
        # Se dopo tutte le correzioni, l'entità è troppo breve o è un articolo/preposizione
        stop_words = ["il", "lo", "la", "i", "gli", "le", "un", "uno", "una", "di", "a", "da", "in", "con", "su", "per", "tra", "fra"]
        if text.lower() in stop_words or len(text) < 3:
            return None

        # 9. Pulisci spazi e aggiorna l'entità
        text = text.strip()
        entity['text'] = text

        # 10. Se il testo è diventato vuoto, scarta l'entità
        if not entity['text']:
            return None

        return entity

    def process_entities(self, entities, original_text):
        """
        Applica post-processing a tutte le entità riconosciute.

        Args:
            entities: Lista di entità da correggere
            original_text: Testo originale per verifica

        Returns:
            Lista di entità corrette
        """
        print(f"DEBUG: Inizio post-processing di {len(entities)} entità")
        corrected_entities = []

        for entity in entities:
            # Applica post-processing all'entità
            corrected = self._post_process_entity(entity, original_text)
            if corrected:
                # Verifica per evitare duplicati
                if not any(e['text'] == corrected['text'] and e['label'] == corrected['label'] for e in corrected_entities):
                    corrected_entities.append(corrected)

        print(f"DEBUG: Post-processing completato, {len(corrected_entities)} entità valide rimaste")
        return corrected_entities





    def set_language(self, language: str):
        """
        Cambia la lingua del riconoscitore.

        Args:
            language: Nuova lingua da impostare
        """
        if language != self.language:
            self.language = language
            self._load_model()

    def get_supported_languages(self) -> List[str]:
        """
        Restituisce le lingue supportate.

        Returns:
            Lista di codici lingua supportati
        """
        return list(self.language_models.keys())



# Aggiungi questo prima della definizione di WikidataConnectorWithSpacy
class WikidataConnector:
    """
    Classe per l'integrazione con Wikidata per il riconoscimento e l'arricchimento di entità.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore a Wikidata.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        self.language = language
        self.delay = delay
        self.cache = {}  # Cache per risultati

        # Italiano stop words
        self.stop_words = {
            "a", "al", "alla", "allo", "ai", "agli", "alle", "con", "col", "coi", "da", "dal", "dallo",
            "dalla", "dai", "dagli", "dalle", "di", "del", "dello", "della", "dei", "degli", "delle",
            "in", "nel", "nello", "nella", "nei", "negli", "nelle", "su", "sul", "sullo", "sulla",
            "sui", "sugli", "sulle", "per", "tra", "fra", "il", "lo", "la", "i", "gli", "le", "un",
            "uno", "una", "un'", "che", "chi", "cui", "è", "sono", "sei", "siamo", "siete", "ha",
            "ho", "hai", "abbiamo", "avete", "hanno", "e", "o", "ma", "se", "anche", "però", "come",
            "dove", "quando", "mentre", "ed", "od", "né", "più", "meno", "molto", "poco", "tanto",
            "ogni", "questo", "questa", "questi", "queste", "quello", "quella", "quelli", "quelle"
        }

    def search_entity(self, term: str) -> List[Dict[str, Any]]:
        """
        Cerca entità su Wikidata in base a un termine.

        Args:
            term: Il termine da cercare

        Returns:
            Lista di entità trovate
        """
        # Controlla cache
        cache_key = f"search_{self.language}_{term}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per la ricerca su Wikidata
        params = {
            'action': 'wbsearchentities',
            'search': term,
            'language': self.language,
            'format': 'json',
            'limit': 5  # Limita a 5 risultati per efficienza
        }

        try:
            # Effettua ricerca
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            results = data.get('search', [])
            self.cache[cache_key] = results

            # Rispetta i rate limit
            time.sleep(self.delay)

            return results

        except Exception as e:
            print(f"Errore nella ricerca Wikidata: {e}")
            return []

    def get_entity_by_id(self, entity_id: str) -> Optional[Dict[str, Any]]:
        """
        Ottiene i dettagli completi di un'entità Wikidata tramite ID.

        Args:
            entity_id: L'ID dell'entità Wikidata (es. Q220)

        Returns:
            Dizionario con i dettagli dell'entità o None in caso di errore
        """
        # Controlla cache
        cache_key = f"entity_{entity_id}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Parametri per l'API Wikidata
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'languages': self.language,
            'format': 'json'
        }

        try:
            # Effettua richiesta
            response = requests.get(
                'https://www.wikidata.org/w/api.php',
                params=params,
                timeout=10
            )
            response.raise_for_status()
            data = response.json()

            if 'entities' in data and entity_id in data['entities']:
                entity = data['entities'][entity_id]
                self.cache[cache_key] = entity

                # Rispetta i rate limit
                time.sleep(self.delay)

                return entity

            return None

        except Exception as e:
            print(f"Errore nel recupero dell'entità Wikidata {entity_id}: {e}")
            return None

    def get_entity_types(self, entity_id: str) -> List[Dict[str, str]]:
        """
        Ottiene i tipi di un'entità Wikidata tramite la proprietà 'instance of' (P31).

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            Lista di tipi dell'entità con ID e label
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity or 'claims' not in entity:
            return []

        # P31 è la proprietà "instance of" in Wikidata
        if 'P31' not in entity['claims']:
            return []

        types = []
        for claim in entity['claims']['P31']:
            if 'mainsnak' in claim and 'datavalue' in claim['mainsnak']:
                datavalue = claim['mainsnak']['datavalue']
                if datavalue['type'] == 'wikibase-entityid':
                    type_id = datavalue['value']['id']

                    # Ottieni l'etichetta del tipo
                    type_entity = self.get_entity_by_id(type_id)
                    type_label = None

                    if type_entity and 'labels' in type_entity:
                        if self.language in type_entity['labels']:
                            type_label = type_entity['labels'][self.language]['value']
                        elif 'en' in type_entity['labels']:
                            # Fallback all'inglese
                            type_label = type_entity['labels']['en']['value']

                    types.append({
                        'id': type_id,
                        'label': type_label or type_id
                    })

        return types

    def get_equivalent_dbpedia_uri(self, entity_id: str) -> Optional[str]:
        """
        Trova l'URI DBpedia equivalente per un'entità Wikidata.

        Args:
            entity_id: L'ID dell'entità Wikidata

        Returns:
            URI DBpedia equivalente o None se non trovato
        """
        entity = self.get_entity_by_id(entity_id)
        if not entity:
            return None

        # Cerca l'equivalente Wikipedia
        if 'sitelinks' in entity:
            # Prima cerca nella lingua specificata
            wiki_key = f"{self.language}wiki"
            if wiki_key in entity['sitelinks']:
                title = entity['sitelinks'][wiki_key]['title']
                # Converti spazi in underscore e codifica per URI
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

            # Fallback all'inglese
            elif 'enwiki' in entity['sitelinks']:
                title = entity['sitelinks']['enwiki']['title']
                title = title.replace(' ', '_')
                return f"http://dbpedia.org/resource/{title}"

        return None

    def extract_candidates(self, text: str) -> List[str]:
        """
        Estrae candidati entità da un testo, ripulendoli dalle stop words.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        candidates = []

        # 1. Estrai frasi multi-parola con iniziali maiuscole (nomi propri complessi)
        multi_word_entities = re.findall(r'\b[A-Z][a-zA-Z]*(?: [A-Z][a-zA-Z]*)+\b', text)
        candidates.extend(multi_word_entities)

        # 2. Estrai sequenze "nome e nome" con iniziali maiuscole (es. "Dante Alighieri")
        name_with_surname = re.findall(r'\b[A-Z][a-zA-Z]* (?:di |da |de |del |della |degli |dei |van |von |)[A-Z][a-zA-Z]*\b', text)
        candidates.extend(name_with_surname)

        # 3. Estrai singole parole con iniziale maiuscola (nomi propri)
        proper_nouns = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', text)

        # Filtra nomi propri per rimuovere quelli all'inizio di frase e parole comuni
        for noun in proper_nouns:
            # Esclude parole che iniziano frasi
            pattern = r'(?:\. |^)' + re.escape(noun)
            if not re.search(pattern, text):
                candidates.append(noun)

        # 4. Coppie di sostantivi che potrebbero essere entità
        # Es. "Politecnico di Milano", "Università di Bologna"
        institution_patterns = [
            r'\b(?:Università|Politecnico|Accademia|Istituto|Teatro|Museo|Galleria) (?:di|del|della|degli|dei) [A-Z][a-zA-Z]+\b',
            r'\b[A-Z][a-zA-Z]+ (?:University|College|Institute|Museum|Gallery)\b'
        ]

        for pattern in institution_patterns:
            institutions = re.findall(pattern, text)
            candidates.extend(institutions)

        # 5. Nomi di luoghi geografici
        geo_patterns = [
            r'\b(?:Mar|Monte|Lago|Fiume|Golfo|Isola|Isole|Monti|Catena|Arcipelago|Oceano) [A-Z][a-zA-Z]+\b'
        ]

        for pattern in geo_patterns:
            geo_entities = re.findall(pattern, text)
            candidates.extend(geo_entities)

        # Rimuovi duplicati mantenendo l'ordine
        seen = set()
        filtered_candidates = []

        for candidate in candidates:
            if candidate.lower() not in seen:
                # Verifica che il candidato non sia composto solo da stop words
                words = candidate.lower().split()
                if any(word not in self.stop_words for word in words):
                    filtered_candidates.append(candidate)
                    seen.add(candidate.lower())

        return filtered_candidates

    def analyze_text(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Estrai candidati entità
        candidates = self.extract_candidates(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }




# Estendi la classe WikidataConnector esistente per integrare spaCy
class WikidataConnectorWithSpacy(WikidataConnector):
    """
    Versione estesa di WikidataConnector che integra spaCy
    per un riconoscimento delle entità più accurato.
    """

    def __init__(self, language: str = "it", delay: float = 0.3):
        """
        Inizializza il connettore Wikidata con integrazione spaCy.

        Args:
            language: Lingua preferita per i risultati (it, en, fr, de, es, ecc.)
            delay: Ritardo tra le richieste per evitare limiti di rate
        """
        super().__init__(language, delay)
        self.spacy_recognizer = SpacyRecognizer(language)

    def extract_candidates_with_spacy(self, text: str) -> List[str]:
        """
        Estrae candidati entità usando spaCy e regole euristiche.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di candidati entità
        """
        # Ottieni entità da spaCy
        spacy_entities = self.spacy_recognizer.recognize_entities(text)

        # Estrai testi delle entità
        spacy_texts = [entity['text'] for entity in spacy_entities]

        # Combina con candidati estratti dal metodo originale
        rule_based_candidates = self.extract_candidates(text)

        # Unisci i candidati eliminando duplicati, mantenendo l'ordine
        all_candidates = []
        seen = set()

        # Prima aggiungi entità spaCy (hanno priorità)
        for candidate in spacy_texts:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        # Poi aggiungi i candidati basati su regole
        for candidate in rule_based_candidates:
            if candidate.lower() not in seen:
                all_candidates.append(candidate)
                seen.add(candidate.lower())

        return all_candidates

    def analyze_text_with_spacy(self, text: str) -> Dict[str, Any]:
        """
        Analizza un testo ed estrae entità usando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con le entità trovate e metadati
        """
        # Se spaCy non è disponibile, usa il metodo originale
        if not self.spacy_recognizer.is_available:
            return self.analyze_text(text)

        # Estrai candidati combinando spaCy e regole
        candidates = self.extract_candidates_with_spacy(text)

        entities = []

        # Cerca ogni candidato su Wikidata
        for candidate in candidates:
            search_results = self.search_entity(candidate)
            if search_results:
                # Prendi il primo risultato (più rilevante)
                top_result = search_results[0]
                entity_id = top_result['id']

                # Ottieni tipi
                types = self.get_entity_types(entity_id)
                type_labels = [t['label'] for t in types if 'label' in t]

                # Ottieni URI DBpedia equivalente
                dbpedia_uri = self.get_equivalent_dbpedia_uri(entity_id)

                entity = {
                    'text': candidate,
                    'wikidata_id': entity_id,
                    'wikidata_url': f"https://www.wikidata.org/wiki/{entity_id}",
                    'dbpedia_uri': dbpedia_uri,
                    'label': top_result.get('label', candidate),
                    'description': top_result.get('description', ''),
                    'types': type_labels,
                    'score': top_result.get('score', 0),
                    'source': 'wikidata+spacy'
                }

                entities.append(entity)

        return {
            'text_length': len(text),
            'entities_count': len(entities),
            'entities': entities
        }



class EntityLinker:
    """
    Classe per collegare entità tra diversi sistemi (DBpedia, Wikidata, testo).
    Combina diverse strategie per massimizzare la copertura.
    """

    def __init__(self, language: str = "it"):
        """
        Inizializza il linker di entità.

        Args:
            language: Lingua preferita per i risultati
        """
        self.language = language
        self.wikidata = WikidataConnector(language=language)

        # Dizionario ampliato di entità comuni italiane
        self.local_entities = {
            # Paesi e continenti
            "italia": {"uri": "http://dbpedia.org/resource/Italy", "type": "Country", "wikidata_id": "Q38"},
            "europa": {"uri": "http://dbpedia.org/resource/Europe", "type": "Continent", "wikidata_id": "Q46"},
            "stati uniti": {"uri": "http://dbpedia.org/resource/United_States", "type": "Country", "wikidata_id": "Q30"},
            "vaticano": {"uri": "http://dbpedia.org/resource/Vatican_City", "type": "Country", "wikidata_id": "Q237"},

            # Città italiane
            "roma": {"uri": "http://dbpedia.org/resource/Rome", "type": "City", "wikidata_id": "Q220"},
            "milano": {"uri": "http://dbpedia.org/resource/Milan", "type": "City", "wikidata_id": "Q490"},
            "napoli": {"uri": "http://dbpedia.org/resource/Naples", "type": "City", "wikidata_id": "Q2634"},
            "firenze": {"uri": "http://dbpedia.org/resource/Florence", "type": "City", "wikidata_id": "Q2044"},
            "venezia": {"uri": "http://dbpedia.org/resource/Venice", "type": "City", "wikidata_id": "Q641"},
            "torino": {"uri": "http://dbpedia.org/resource/Turin", "type": "City", "wikidata_id": "Q495"},
            "bologna": {"uri": "http://dbpedia.org/resource/Bologna", "type": "City", "wikidata_id": "Q1891"},

            # Monumenti e luoghi
            "colosseo": {"uri": "http://dbpedia.org/resource/Colosseum", "type": "Monument", "wikidata_id": "Q10285"},
            "vesuvio": {"uri": "http://dbpedia.org/resource/Mount_Vesuvius", "type": "Volcano", "wikidata_id": "Q2292"},
            "mediterraneo": {"uri": "http://dbpedia.org/resource/Mediterranean_Sea", "type": "Sea", "wikidata_id": "Q4918"},

            # Università
            "sapienza": {"uri": "http://dbpedia.org/resource/Sapienza_University_of_Rome", "type": "University", "wikidata_id": "Q213485"},
            "politecnico di milano": {"uri": "http://dbpedia.org/resource/Polytechnic_University_of_Milan", "type": "University", "wikidata_id": "Q1151817"},

            # Prodotti e marchi
            "ferrari": {"uri": "http://dbpedia.org/resource/Ferrari", "type": "Company", "wikidata_id": "Q8402"},
            "lamborghini": {"uri": "http://dbpedia.org/resource/Lamborghini", "type": "Company", "wikidata_id": "Q26378"},
            "chianti": {"uri": "http://dbpedia.org/resource/Chianti", "type": "Wine", "wikidata_id": "Q662785"},
            "barolo": {"uri": "http://dbpedia.org/resource/Barolo", "type": "Wine", "wikidata_id": "Q812394"},
            "pizza": {"uri": "http://dbpedia.org/resource/Pizza", "type": "Food", "wikidata_id": "Q177"},

            # Personaggi storici
            "dante alighieri": {"uri": "http://dbpedia.org/resource/Dante_Alighieri", "type": "Person", "wikidata_id": "Q1067"},
            "leonardo da vinci": {"uri": "http://dbpedia.org/resource/Leonardo_da_Vinci", "type": "Person", "wikidata_id": "Q762"},
            "galileo galilei": {"uri": "http://dbpedia.org/resource/Galileo_Galilei", "type": "Person", "wikidata_id": "Q307"}
        }

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando molteplici strategie.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Inizia con Wikidata
        wikidata_results = self.wikidata.analyze_text(text)
        entities = wikidata_results['entities']

        # Usa anche il metodo locale per avere una copertura completa
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _is_valid_entity(self, text: str) -> bool:
        """
        Verifica se un testo rappresenta un'entità valida (non una stopword o articolo).

        Args:
            text: Il testo da verificare

        Returns:
            True se l'entità è valida, False altrimenti
        """
        if not text:
            return False

        text_lower = text.lower()

        # Stopwords più comuni
        stopwords = self.wikidata.stop_words

        # Articoli in italiano
        articles = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una"}

        # Congiunzioni e preposizioni
        conjunctions = {"e", "o", "ma", "se", "mentre", "per", "con", "su", "in", "da", "di", "a"}

        # Verifica se il testo è composto solo da stopwords, articoli o congiunzioni
        words = text_lower.split()

        # Se è una singola parola e fa parte delle stopwords
        if len(words) == 1 and (words[0] in stopwords or words[0] in articles or words[0] in conjunctions):
            return False

        # Se tutte le parole sono stopwords
        if all(word in stopwords or word in articles or word in conjunctions for word in words):
            return False

        # Se è una parola troppo corta (meno di 3 caratteri)
        if len(text) < 3:
            return False

        return True

    def _find_local_entities(self, text: str) -> List[Dict[str, Any]]:
        """
        Trova entità usando il dizionario locale.

        Args:
            text: Il testo da analizzare

        Returns:
            Lista di entità trovate
        """
        text_lower = text.lower()
        entities = []

        # Ordina le chiavi del dizionario per lunghezza (decrescente)
        # Questo assicura che "politecnico di milano" venga trovato prima di "milano"
        sorted_keys = sorted(self.local_entities.keys(), key=len, reverse=True)

        for entity_name in sorted_keys:
            if entity_name in text_lower:
                info = self.local_entities[entity_name]

                # Trova la posizione dell'entità nel testo
                start = text_lower.find(entity_name)

                # Controlla se l'entità è parte di un'altra parola
                if start > 0 and text_lower[start-1].isalnum():
                    continue  # Skip se l'entità è parte di una parola più lunga

                if start + len(entity_name) < len(text_lower) and text_lower[start + len(entity_name)].isalnum():
                    continue  # Skip se l'entità è seguita da altri caratteri alfanumerici

                # Estrai il testo originale con la capitalizzazione originale
                original_text = text[start:start+len(entity_name)]

                entity = {
                    'text': original_text,
                    'dbpedia_uri': info['uri'],
                    'types': [info['type']],
                    'wikidata_id': info.get('wikidata_id'),
                    'wikidata_url': f"https://www.wikidata.org/wiki/{info.get('wikidata_id')}" if info.get('wikidata_id') else None,
                    'score': 1.0,
                    'source': 'local'
                }
                entities.append(entity)

        return entities

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'stats': stats,
            'text': text
        }




# Estendi la classe EntityLinker per utilizzare il WikidataConnector migliorato
class EntityLinkerWithSpacy(EntityLinker):
    """
    Versione estesa di EntityLinker che utilizza il WikidataConnector con integrazione spaCy.
    """

    def __init__(self, language: str = "it"):
        # Prima chiamiamo il costruttore base per inizializzare il dizionario locale
        super().__init__(language)

        # Poi sostituiamo l'oggetto wikidata con la versione estesa
        self.wikidata = WikidataConnectorWithSpacy(language=language)

    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando spaCy e Wikidata.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Usa il metodo migliorato che integra spaCy
        wikidata_results = self.wikidata.analyze_text_with_spacy(text)
        entities = wikidata_results['entities']

        # Il resto del metodo è identico alla versione originale
        local_entities = self._find_local_entities(text)

        # Combina i risultati, evitando duplicati
        existing_ids = {e.get('wikidata_id') for e in entities if 'wikidata_id' in e}
        existing_texts = {e.get('text').lower() for e in entities if 'text' in e}

        for local_entity in local_entities:
            wikidata_id = local_entity.get('wikidata_id')
            text_lower = local_entity.get('text', '').lower()

            if (wikidata_id and wikidata_id not in existing_ids) or (text_lower and text_lower not in existing_texts):
                entities.append(local_entity)

        # Rimuovi le entità che sono stopwords o articoli
        entities = [e for e in entities if self._is_valid_entity(e.get('text', ''))]

        # Arricchisci le entità con dati aggiuntivi se necessario
        enriched_entities = []
        for entity in entities:
            # Assicurati che abbiamo un URI DBpedia
            if 'dbpedia_uri' not in entity or not entity['dbpedia_uri']:
                wikidata_id = entity.get('wikidata_id')
                if wikidata_id:
                    dbpedia_uri = self.wikidata.get_equivalent_dbpedia_uri(wikidata_id)
                    if dbpedia_uri:
                        entity['dbpedia_uri'] = dbpedia_uri

            enriched_entities.append(entity)

        # Aggiungi le entità spaCy non trovate in Wikidata
        spacy_only_entities = self._get_spacy_only_entities(text, existing_texts)
        enriched_entities.extend(spacy_only_entities)

        # Raccogli statistiche
        stats = {
            'total_entities': len(enriched_entities),
            'wikidata_entities': len(wikidata_results['entities']),
            'spacy_only_entities': len(spacy_only_entities),
            'local_entities': sum(1 for e in enriched_entities if e.get('source') == 'local'),
            'entities_with_dbpedia': sum(1 for e in enriched_entities if 'dbpedia_uri' in e and e['dbpedia_uri']),
            'entities_with_types': sum(1 for e in enriched_entities if 'types' in e and e['types'])
        }

        return enriched_entities, stats

    def _get_spacy_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
            """
            Ottiene entità riconosciute solo da spaCy e non da Wikidata o dal dizionario locale.

            Args:
                text: Il testo da analizzare
                existing_texts: Set di testi di entità già trovate

            Returns:
                Lista di entità riconosciute solo da spaCy
            """
            # CORREZIONE: Verifica che il riconoscitore spaCy sia disponibile
            if not hasattr(self.wikidata, 'spacy_recognizer') or not self.wikidata.spacy_recognizer.is_available:
                print("Riconoscitore spaCy non disponibile.")
                return []

            # Ottieni tutte le entità da spaCy
            spacy_entities = self.wikidata.spacy_recognizer.recognize_entities(text)

            # CORREZIONE: Debug per verificare quante entità spaCy sono state trovate
            print(f"spaCy ha trovato {len(spacy_entities)} entità nel testo")

            # Lista per entità trovate solo da spaCy
            spacy_only = []

            # CORREZIONE: Itera su tutte le entità spaCy
            for entity in spacy_entities:
                entity_text = entity['text']
                entity_lower = entity_text.lower()

                # Verifica se l'entità è già stata trovata da altre fonti
                if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                    # CORREZIONE: Debug per ogni entità spaCy considerata unica
                    print(f"Entità unica da spaCy: {entity_text} ({entity['label']})")

                    # Converti l'etichetta spaCy in un tipo più generico
                    spacy_type = self._map_spacy_label_to_type(entity['label'])

                    # Crea un'entità formattata
                    spacy_entity = {
                        'text': entity_text,
                        'types': [spacy_type],
                        'source': 'spacy',
                        'label': entity['label']
                    }

                    # Aggiungi alla lista e al set di testi esistenti
                    spacy_only.append(spacy_entity)
                    existing_texts.add(entity_lower)

            # CORREZIONE: Debug per verificare quante entità spaCy uniche sono state trovate
            print(f"Di cui {len(spacy_only)} sono uniche (non trovate da Wikidata o dal dizionario locale)")

            return spacy_only

    def _map_spacy_label_to_type(self, spacy_label: str) -> str:
        """
        Mappa le etichette spaCy a tipi più generici.

        Args:
            spacy_label: L'etichetta spaCy

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette spaCy a tipi più generali
        # Questa mappatura dipende dal modello spaCy utilizzato
        mapping = {
            'PER': 'Person',
            'PERSON': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'FAC': 'Facility',
            'FACILITY': 'Facility',
            'PRODUCT': 'Product',
            'EVENT': 'Event',
            'WORK_OF_ART': 'Artwork',
            'LAW': 'Law',
            'LANGUAGE': 'Language',
            'DATE': 'Date',
            'TIME': 'Time',
            'PERCENT': 'Percent',
            'MONEY': 'Money',
            'QUANTITY': 'Quantity',
            'ORDINAL': 'Ordinal',
            'CARDINAL': 'Cardinal'
        }

        return mapping.get(spacy_label, spacy_label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estensione del metodo originale per includere i risultati di spaCy.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        entities, stats = self.find_entities(text)

        # Organizza i risultati per parola/frase
        results = {}
        for entity in entities:
            text_key = entity['text']

            # Ottieni l'URI disponibile (preferibilmente DBpedia)
            uri = entity.get('dbpedia_uri')

            if text_key and uri:
                results[text_key] = {
                    'uri': uri,
                    'exists': True,  # Assumiamo che esista perché l'abbiamo trovato
                    'wikidata_id': entity.get('wikidata_id'),
                    'types': entity.get('types', [])
                }

        # Aggiungi una nuova sezione per le entità spaCy senza URI
        spacy_results = {}
        for entity in entities:
            if entity.get('source') == 'spacy' and entity['text'] not in results:
                spacy_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        return {
            'results': results,
            'spacy_results': spacy_results,
            'stats': stats,
            'text': text
        }



# Estendi la classe EntityLinkerWithSpacy per incorporare il riconoscitore italiano
class EntityLinkerItalian(EntityLinkerWithSpacy):
    """
    Versione estesa di EntityLinker ottimizzata per l'italiano in ambiente Colab.
    Integra riconoscitori multipli: Wikidata, spaCy, Stanza NLP e Transformer NER.
    """

    def __init__(self, language: str = "it", use_gpu: bool = False):
        """
        Inizializza il linker di entità con supporto NLP per l'italiano.

        Args:
            language: Lingua preferita per i risultati (deve essere "it")
            use_gpu: Se True, utilizza GPU per l'accelerazione (se disponibile in Colab)
        """
        # Controlla che la lingua sia italiana
        if language != "it":
            print("Attenzione: questo riconoscitore è ottimizzato per l'italiano. Impostiamo language='it'.")
            language = "it"

        # Inizializza la classe base (con spaCy)
        super().__init__(language)

        # Aggiungi il riconoscitore italiano Stanza NLP
        self.italian_recognizer = ItalianNLPRecognizer(use_gpu=use_gpu)

        # Aggiungi il riconoscitore Transformer NER
        try:
            from transformers import AutoTokenizer, pipeline  # Verifica disponibilità di transformers
            import torch  # Verifica disponibilità di torch

            print("DEBUG: Moduli transformers e torch disponibili. Inizializzazione riconoscitore Transformer NER...")

            self.transformers_recognizer = TransformersNERRecognizer(
                #model_name="5had3/bert-base-italian-cased-ner",
                model_name=None,
                language=language,
                use_gpu=use_gpu
            )

            # Verifica se il riconoscitore è stato inizializzato correttamente
            if not hasattr(self.transformers_recognizer, 'is_available') or not self.transformers_recognizer.is_available:
                print("DEBUG: Riconoscitore Transformer NER non disponibile dopo l'inizializzazione")
            else:
                print("DEBUG: Riconoscitore Transformer NER inizializzato con successo")


            print("DEBUG: Inizializzazione WordNet knowledge base...")

        except ImportError as e:
            print(f"DEBUG: ImportError - Transformer NER non disponibile. {e}")
            print("Installare 'transformers' e 'torch' con 'pip install transformers torch'")
            self.transformers_recognizer = None
        except Exception as e:
            print(f"DEBUG: Errore generico nell'inizializzazione del riconoscitore Transformer NER: {e}")
            import traceback
            traceback.print_exc()
            self.transformers_recognizer = None

    # Aggiungi il classificatore Zero-Shot in un blocco try/except separato
        try:
            print("DEBUG: Inizializzazione Zero-Shot Classifier...")
            self.zero_shot_classifier = ZeroShotClassifier(
                language=language,
                use_gpu=use_gpu
            )

            # Verifica se il classificatore è stato inizializzato correttamente
            if not hasattr(self.zero_shot_classifier, 'is_available') or not self.zero_shot_classifier.is_available:
                print("DEBUG: Zero-Shot Classifier non disponibile dopo l'inizializzazione")
            else:
                print("DEBUG: Zero-Shot Classifier inizializzato con successo")

        except ImportError as e:
            print(f"DEBUG: ImportError - Zero-Shot Classifier non disponibile. {e}")
            print("Installare 'transformers' e 'torch' con 'pip install transformers torch'")
            self.zero_shot_classifier = None
        except Exception as e:
            print(f"DEBUG: Errore generico nell'inizializzazione del Zero-Shot Classifier: {e}")
            import traceback
            traceback.print_exc()
            self.zero_shot_classifier = None

        # Dentro il __init__ della classe EntityLinkerItalian
        try:
            print("DEBUG: Inizializzazione WordNet knowledge base...")

            # Verifica che nltk e wordnet siano disponibili
            import nltk
            from nltk.corpus import wordnet as wn
            print("DEBUG: NLTK e WordNet importati con successo")

            # Verifica che i dati di wordnet siano stati scaricati
            try:
                nltk.data.find('corpora/wordnet')
                print("DEBUG: WordNet corpus trovato")
            except LookupError:
                print("DEBUG: WordNet corpus non trovato, tentativo di download...")
                nltk.download('wordnet')

            try:
                nltk.data.find('corpora/omw-1.4')
                print("DEBUG: Open Multilingual WordNet trovato")
            except LookupError:
                print("DEBUG: Open Multilingual WordNet non trovato, tentativo di download...")
                nltk.download('omw-1.4')

            # Test con una parola italiana
            test_word = "casa"
            synsets = wn.synsets(test_word, lang='ita')
            print(f"DEBUG: Test WordNet in italiano per 'casa': {len(synsets)} synsets trovati")

            self.wordnet_kb = WordNetKnowledgeBase(language="it")

            if hasattr(self.wordnet_kb, 'is_available'):
                print(f"DEBUG: WordNet KB disponibilità: {self.wordnet_kb.is_available}")
            else:
                print("DEBUG: L'attributo is_available non esiste in WordNet KB")

        except ImportError as e:
            print(f"DEBUG: ImportError - WordNet non disponibile. {e}")
            print("Installare 'nltk' con 'pip install nltk'")
            self.wordnet_kb = None
        except Exception as e:
            print(f"DEBUG: Errore generico nell'inizializzazione di WordNet: {e}")
            import traceback
            traceback.print_exc()
            self.wordnet_kb = None


    def find_entities(self, text: str) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
        """
        Trova entità nel testo utilizzando Wikidata, spaCy, Stanza NLP e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Tuple con lista di entità e statistiche
        """
        # Debug del riconoscitore Transformer
        print(f"DEBUG: Stato riconoscitore Transformer - Disponibile: {hasattr(self, 'transformers_recognizer')}")
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer:
            print(f"DEBUG: Riconoscitore Transformer - Is Available: {self.transformers_recognizer.is_available}")

        # Prima ottieni i risultati dalla classe base (Wikidata + spaCy)
        enriched_entities, stats = super().find_entities(text)
        existing_texts = {e.get('text', '').lower() for e in enriched_entities if 'text' in e}

        # Aggiungi le entità del riconoscitore italiano se disponibile
        italian_only_entities = self._get_italian_only_entities(text, existing_texts)
        enriched_entities.extend(italian_only_entities)

        # Aggiorna il set di testi esistenti dopo aver aggiunto le entità italiane
        for entity in italian_only_entities:
            if 'text' in entity:
                existing_texts.add(entity['text'].lower())

        # Aggiungi le entità dal riconoscitore Transformer NER
        transformers_only_entities = []
        if hasattr(self, 'transformers_recognizer') and self.transformers_recognizer and self.transformers_recognizer.is_available:
            print("DEBUG: Tentativo di trovare entità Transformer NER")
            try:
                # Usa un metodo separato per ottenere le entità
                transformers_only_entities = self._get_transformers_only_entities(text, existing_texts)
                print(f"DEBUG: Entità Transformer NER trovate: {len(transformers_only_entities)}")

                # Aggiungi solo se ci sono entità
                if transformers_only_entities:
                    enriched_entities.extend(transformers_only_entities)
            except Exception as e:
                print(f"DEBUG: Errore nel trovare entità Transformer: {e}")
                import traceback
                traceback.print_exc()
        else:
            print("DEBUG: Riconoscitore Transformer NER non disponibile per find_entities")


        # Dopo aver aggiunto le entità da Transformer NER
        # Applica classificazione zero-shot se disponibile
        if hasattr(self, 'zero_shot_classifier') and self.zero_shot_classifier and self.zero_shot_classifier.is_available:
            print("DEBUG: Applicazione classificazione Zero-Shot alle entità")
            try:
                # Classifica le entità
                enriched_entities = self.zero_shot_classifier.classify_entities(enriched_entities, original_text=text)
                print(f"DEBUG: Classificazione Zero-Shot completata")
            except Exception as e:
                print(f"DEBUG: Errore nella classificazione Zero-Shot: {e}")
                import traceback
                traceback.print_exc()


         # Arricchisci con WordNet se disponibile
        if hasattr(self, 'wordnet_kb') and self.wordnet_kb and self.wordnet_kb.is_available:
            print("DEBUG: Arricchimento entità con WordNet")
            try:
                # Arricchisci le entità con informazioni semantiche
                enriched_entities = self.wordnet_kb.enrich_entities(enriched_entities)
                print(f"DEBUG: Arricchimento WordNet completato")
            except Exception as e:
                print(f"DEBUG: Errore nell'arricchimento WordNet: {e}")
                import traceback
                traceback.print_exc()

        # Aggiorna le statistiche
        stats['italian_only_entities'] = len(italian_only_entities)
        stats['transformers_only_entities'] = len(transformers_only_entities)
        stats['total_entities'] = len(enriched_entities)

        return enriched_entities, stats

    def _get_italian_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
        """
        Ottiene entità riconosciute solo dal riconoscitore italiano e non da altre fonti.

        Args:
            text: Il testo da analizzare
            existing_texts: Set di testi di entità già trovate

        Returns:
            Lista di entità riconosciute solo dal riconoscitore italiano
        """
        # Verifica che il riconoscitore italiano sia disponibile
        if not self.italian_recognizer.is_available:
            print("Riconoscitore NLP italiano non disponibile.")
            return []

        # Ottieni tutte le entità dal riconoscitore italiano
        italian_entities = self.italian_recognizer.recognize_entities(text)
        print(f"Riconoscitore italiano ha trovato {len(italian_entities)} entità nel testo")

        # Filtra per entità uniche non trovate da altre fonti
        italian_only = []
        for entity in italian_entities:
            entity_text = entity['text']
            entity_lower = entity_text.lower()

            if entity_lower not in existing_texts and self._is_valid_entity(entity_text):
                print(f"Entità unica dal riconoscitore italiano: {entity_text} ({entity['label']})")

                # Converti l'etichetta in un tipo più generico
                italian_type = self._map_italian_label_to_type(entity['label'])

                italian_entity = {
                    'text': entity_text,
                    'types': [italian_type],
                    'source': 'italian_nlp',
                    'label': entity['label']
                }

                italian_only.append(italian_entity)
                existing_texts.add(entity_lower)

        print(f"Di cui {len(italian_only)} sono uniche (non trovate da altre fonti)")
        return italian_only

    def _get_transformers_only_entities(self, text: str, existing_texts: Set[str]) -> List[Dict[str, Any]]:
          """
          Ottiene entità riconosciute dal riconoscitore Transformer NER.
          Includendo sia entità nuove che quelle già trovate da altre fonti.
          """
          # Verifica che il riconoscitore Transformer sia disponibile
          if not self.transformers_recognizer or not self.transformers_recognizer.is_available:
              print("DEBUG: Riconoscitore Transformer NER non disponibile.")
              return []

          # Stampa informazioni sul riconoscitore
          print(f"DEBUG: Modello Transformer NER in uso: {self.transformers_recognizer.model_name}")

          try:
              # Ottieni tutte le entità dal riconoscitore Transformer NER
              transformers_entities = self.transformers_recognizer.recognize_entities(text)

              # Debug dettagliato delle entità
              print(f"DEBUG: Numero di entità Transformer NER trovate: {len(transformers_entities)}")

              # Filtra e mappa le entità - IMPORTANTE: non filtriamo per existing_texts
              # per assicurarci che tutte le entità Transformer siano incluse
              transformers_mapped = []
              for entity in transformers_entities:
                  if self._is_valid_entity(entity.get('text', '')):
                      # Aggiungi sempre il source 'transformers_ner'
                      mapped_entity = {
                          'text': entity.get('text', ''),
                          'label': entity.get('label', ''),
                          'types': [entity.get('label', 'N/A')],
                          'source': 'transformers_ner',
                          'score': entity.get('score', 0)
                      }
                      transformers_mapped.append(mapped_entity)

              print(f"DEBUG: Numero di entità Transformer NER mappate: {len(transformers_mapped)}")
              return transformers_mapped

          except Exception as e:
              print(f"DEBUG: Errore nell'ottenere entità Transformer: {e}")
              import traceback
              traceback.print_exc()
              return []

    def _map_italian_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore italiano a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        mapping = {
            'PERSON': 'Person',
            'PER': 'Person',
            'LOC': 'Location',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'ORG': 'Organization',
            'ORGANIZATION': 'Organization',
            'MISC': 'Miscellaneous',
            'PROPN': 'ProperNoun',
            'WORK_OF_ART': 'Artwork',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def _map_transformers_label_to_type(self, label: str) -> str:
        """
        Mappa le etichette del riconoscitore Transformer NER a tipi più generici.

        Args:
            label: L'etichetta originale

        Returns:
            Tipo generico
        """
        # Mappatura delle etichette a tipi più generali
        # (simile a quella di altri riconoscitori)
        mapping = {
            'PER': 'Person',
            'LOC': 'Location',
            'ORG': 'Organization',
            'MISC': 'Miscellaneous',
            'GPE': 'Location',
            'LOCATION': 'Location',
            'PERSON': 'Person',
            'ORGANIZATION': 'Organization',
            'EVENT': 'Event',
            'DATE': 'Date',
            'TIME': 'Time',
            'MONEY': 'Money',
            'PERCENT': 'Percent',
            'QUANTITY': 'Quantity'
        }

        return mapping.get(label, label)

    def verify_uris_for_text(self, text: str) -> Dict[str, Any]:
        """
        Verifica l'esistenza di URI per tutte le entità in un testo.
        Estende il metodo della classe base per includere i risultati
        del riconoscitore italiano e Transformer NER.

        Args:
            text: Il testo da analizzare

        Returns:
            Dizionario con risultati e statistiche
        """
        # Ottieni i risultati di base
        uri_results = super().verify_uris_for_text(text)

        # Aggiungi una sezione per le entità del riconoscitore italiano senza URI
        italian_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'italian_nlp' and entity['text'] not in uri_results['results']:
                italian_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', [])
                }

        # Aggiungi una sezione per le entità del riconoscitore Transformer NER senza URI
        transformers_results = {}
        for entity in uri_results.get('entities', []):
            if entity.get('source') == 'transformers_ner' and entity['text'] not in uri_results['results']:
                transformers_results[entity['text']] = {
                    'label': entity.get('label', ''),
                    'types': entity.get('types', []),
                    'score': entity.get('score', 0)
                }

        # Aggiungi le sezioni ai risultati
        uri_results['italian_results'] = italian_results
        uri_results['transformers_results'] = transformers_results

        return uri_results


class ZeroShotClassifier:
    """
    Classificatore zero-shot per il riconoscimento di entità senza addestramento specifico.
    Utilizza modelli transformer pre-addestrati per classificare entità in categorie arbitrarie.
    """

    def __init__(self, model_name: str = "facebook/bart-large-mnli", language: str = "it", use_gpu: bool = False):
        """
        Inizializza il classificatore zero-shot.

        Args:
            model_name: Nome del modello per la classificazione zero-shot
            language: Lingua del testo da analizzare
            use_gpu: Se utilizzare GPU per l'accelerazione
        """
        self.model_name = model_name
        self.language = language
        self.use_gpu = use_gpu
        self.is_available = False
        self.classifier = None

        # Elenco modelli alternativi multilingua per zero-shot
        self.alternative_models = [
            "facebook/bart-large-mnli",              # Inglese
            "joeddav/xlm-roberta-large-xnli",        # Multilingua, ottimo per italiano
            "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7", # Ottimo modello multilingua
            "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli", # Alta performance
            "cross-encoder/nli-deberta-v3-large",    # Inglese, alta qualità
        ]

        # Categorie predefinite per le entità in italiano
        self.default_labels = {
            'it': {
                'persona': "Questa entità è una persona, un individuo o un personaggio?",
                'luogo': "Questa entità è un luogo, una località o una posizione geografica?",
                'organizzazione': "Questa entità è un'organizzazione, un'azienda, un gruppo o un'istituzione?",
                'evento': "Questa entità è un evento, una manifestazione o un avvenimento storico?",
                'opera': "Questa entità è un'opera d'arte, un libro, una canzone o un film?",
                'prodotto': "Questa entità è un prodotto, una marca o un oggetto commerciale?",
                'concetto': "Questa entità è un concetto astratto, un'idea o una teoria?"
            },
            'en': {
                'person': "This entity is a person, individual, or character?",
                'location': "This entity is a place, location, or geographical area?",
                'organization': "This entity is an organization, company, group, or institution?",
                'event': "This entity is an event, happening, or historical occurrence?",
                'work': "This entity is a work of art, book, song, or movie?",
                'product': "This entity is a product, brand, or commercial item?",
                'concept': "This entity is an abstract concept, idea, or theory?"
            }
        }

        # Inizializza il modello
        self._load_model()

    def _load_model(self):
        """
        Carica il modello per la classificazione zero-shot.
        """
        try:
            from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
            import torch

            print(f"DEBUG: Tentativo di caricare il classificatore zero-shot: {self.model_name}")

            # Determina il device
            device = 0 if self.use_gpu and torch.cuda.is_available() else -1

            # Prova a caricare il modello principale
            try:
                self.classifier = pipeline(
                    "zero-shot-classification",
                    model=self.model_name,
                    device=device
                )
                self.is_available = True
                print(f"DEBUG: Classificatore zero-shot caricato con successo: {self.model_name}")

                # Test rapido
                test_result = self.classifier(
                    "Roma è la capitale d'Italia",
                    candidate_labels=["città", "paese", "monumenti"],
                )
                print(f"DEBUG: Test zero-shot completato con successo")
                return

            except Exception as model_error:
                print(f"DEBUG: Errore nel caricamento del modello principale: {str(model_error)}")

                # Prova modelli alternativi
                for alt_model in self.alternative_models:
                    if alt_model != self.model_name:  # Evita di provare lo stesso modello
                        try:
                            print(f"DEBUG: Tentativo con modello alternativo: {alt_model}")
                            self.classifier = pipeline(
                                "zero-shot-classification",
                                model=alt_model,
                                device=device
                            )
                            self.model_name = alt_model
                            self.is_available = True
                            print(f"DEBUG: Modello alternativo caricato con successo: {alt_model}")
                            return
                        except Exception as alt_error:
                            print(f"DEBUG: Errore con modello alternativo {alt_model}: {str(alt_error)}")

                raise model_error

        except ImportError as e:
            print(f"DEBUG: Errore di importazione per zero-shot-classification: {str(e)}")
            print("Installare transformers e torch: pip install transformers torch")
            self.is_available = False

        except Exception as e:
            print(f"DEBUG: Errore generale nel caricamento del classificatore zero-shot: {str(e)}")
            import traceback
            traceback.print_exc()
            self.is_available = False

    def classify_entity(self, entity_text: str, context: str = None, custom_labels: List[str] = None):
        """
        Classifica un'entità con il modello zero-shot.

        Args:
            entity_text: Il testo dell'entità da classificare
            context: Contesto opzionale in cui appare l'entità
            custom_labels: Etichette personalizzate da utilizzare (se None, usa le predefinite)

        Returns:
            Dizionario con etichette e punteggi di confidenza
        """
        if not self.is_available or not self.classifier:
            print("DEBUG: Classificatore zero-shot non disponibile")
            return {'label': 'unknown', 'scores': {}}

        try:
            # Usa le etichette predefinite se non sono specificate etichette personalizzate
            if custom_labels is None:
                # Scegli le etichette basate sulla lingua
                lang_key = 'en' if self.language not in self.default_labels else self.language
                labels = list(self.default_labels[lang_key].values())
            else:
                labels = custom_labels

            # Prepara il testo per la classificazione
            if context:
                # Se c'è un contesto, aggiungiamo una frase che colloca l'entità nel contesto
                text_to_classify = f"Nel contesto di: '{context}', l'entità '{entity_text}' è di che tipo?"
            else:
                # Altrimenti classifichiamo direttamente l'entità
                text_to_classify = entity_text

            # Esegui la classificazione
            result = self.classifier(
                text_to_classify,
                candidate_labels=labels,
                hypothesis_template="{}",  # Template vuoto (già incluso nelle etichette)
                multi_label=False
            )

            # Estrai l'etichetta migliore e il punteggio
            best_label = result['labels'][0]
            best_score = result['scores'][0]

            # Se stiamo usando le etichette predefinite, mappa l'ipotesi all'etichetta reale
            if custom_labels is None:
                lang_key = 'en' if self.language not in self.default_labels else self.language
                # Inverte il dizionario per ottenere l'etichetta dalla domanda
                label_map = {v: k for k, v in self.default_labels[lang_key].items()}
                best_label = label_map.get(best_label, best_label)

            # Crea un dizionario di punteggi per ogni etichetta
            scores = {}
            for idx, label in enumerate(result['labels']):
                if custom_labels is None:
                    lang_key = 'en' if self.language not in self.default_labels else self.language
                    label_map = {v: k for k, v in self.default_labels[lang_key].items()}
                    mapped_label = label_map.get(label, label)
                    scores[mapped_label] = result['scores'][idx]
                else:
                    scores[label] = result['scores'][idx]

            return {
                'label': best_label,
                'score': best_score,
                'scores': scores
            }

        except Exception as e:
            print(f"DEBUG: Errore nella classificazione zero-shot: {str(e)}")
            import traceback
            traceback.print_exc()
            return {'label': 'error', 'score': 0.0, 'scores': {}}

    def classify_entities(self, entities, original_text=None):
        """
        Classifica un elenco di entità.

        Args:
            entities: Lista di entità da classificare
            original_text: Testo originale per contesto

        Returns:
            Lista di entità arricchite con classificazione
        """
        if not self.is_available:
            return entities

        print(f"DEBUG: Classificazione zero-shot di {len(entities)} entità")

        enriched_entities = []
        for entity in entities:
            # Salta entità che hanno già tipi definiti
            if entity.get('types') and len(entity.get('types', [])) > 0:
                enriched_entities.append(entity)
                continue

            # Estrai un contesto intorno all'entità se disponibile
            context = None
            if original_text and 'text' in entity:
                entity_text = entity['text']
                if entity_text in original_text:
                    # Trova la posizione dell'entità nel testo
                    start_pos = original_text.find(entity_text)
                    # Estrai fino a 100 caratteri prima e dopo come contesto
                    context_start = max(0, start_pos - 100)
                    context_end = min(len(original_text), start_pos + len(entity_text) + 100)
                    context = original_text[context_start:context_end]

            # Classifica l'entità
            classification = self.classify_entity(entity['text'], context)

            # Aggiungi i risultati all'entità
            if 'types' not in entity or not entity['types']:
                entity['types'] = [classification['label']]
            if 'score' not in entity:
                entity['score'] = classification['score']

            # Aggiungi attributo di classificazione zero-shot
            entity['zero_shot_classification'] = {
                'label': classification['label'],
                'score': classification['score'],
                'scores': classification['scores']
            }

            enriched_entities.append(entity)

        print(f"DEBUG: Classificazione zero-shot completata")
        return enriched_entities


class WordNetKnowledgeBase:
    """
    Classe per l'arricchimento semantico di entità usando WordNet.
    Fornisce definizioni, sinonimi, iponimi e iperonimi.
    """

    def __init__(self, language: str = "ita"):
        """
        Inizializza la knowledge base con WordNet.

        Args:
            language: Lingua per WordNet (ita per italiano, eng per inglese)
        """
        self.language = language
        self.is_available = False

        # Mappa lingue a codici WordNet
        self.language_map = {
            "it": "ita",
            "en": "eng",
            "fr": "fra",
            "de": "deu",
            "es": "spa"
        }

        # Inizializza WordNet
        self._initialize_wordnet()

    def _initialize_wordnet(self):
        """
        Inizializza la risorsa WordNet e verifica disponibilità.
        """
        try:
            from nltk.corpus import wordnet as wn
            import nltk

            # Assicurati che le risorse NLTK necessarie siano scaricate
            try:
                nltk.data.find('corpora/wordnet')
            except LookupError:
                print("DEBUG: Scaricamento di WordNet...")
                nltk.download('wordnet')

            try:
                nltk.data.find('corpora/omw-1.4')
            except LookupError:
                print("DEBUG: Scaricamento di Open Multilingual WordNet...")
                nltk.download('omw-1.4')

            # Verifica che la lingua sia supportata
            mapped_lang = self.language_map.get(self.language, self.language)

            # Test rapido per verificare il funzionamento di WordNet
            test_word = "casa" if mapped_lang == "ita" else "house"
            synsets = wn.synsets(test_word, lang=mapped_lang)

            if synsets:
                self.is_available = True
                print(f"DEBUG: WordNet inizializzato con successo. Lingua: {mapped_lang}")
                print(f"DEBUG: Esempio per '{test_word}': {len(synsets)} synsets trovati")
            else:
                print(f"DEBUG: WordNet non ha trovato synsets per '{test_word}' in {mapped_lang}")
                print("DEBUG: Verifica che la lingua sia supportata e installata correttamente")
                # Prova con l'inglese come fallback
                synsets = wn.synsets('house', lang='eng')
                if synsets:
                    self.language = "en"
                    print("DEBUG: Utilizzo dell'inglese come lingua di fallback")
                    self.is_available = True
                else:
                    raise Exception("WordNet non funziona neanche con la lingua inglese")

        except ImportError as e:
            print(f"DEBUG: Errore di importazione NLTK/WordNet: {str(e)}")
            print("Installare NLTK: pip install nltk")
            self.is_available = False

        except Exception as e:
            print(f"DEBUG: Errore nell'inizializzazione di WordNet: {str(e)}")
            import traceback
            traceback.print_exc()
            self.is_available = False

    def get_synsets(self, word: str):
        """
        Ottiene i synset di WordNet per una parola.

        Args:
            word: Parola da cercare

        Returns:
            Lista di synset trovati
        """
        if not self.is_available:
            return []

        try:
            from nltk.corpus import wordnet as wn

            # Usa il codice lingua mappato
            mapped_lang = self.language_map.get(self.language, self.language)

            # Cerca i synset nella lingua specificata
            synsets = wn.synsets(word, lang=mapped_lang)

            # Se non trova nulla e la lingua non è inglese, prova anche in inglese
            if not synsets and mapped_lang != 'eng':
                eng_synsets = wn.synsets(word, lang='eng')
                if eng_synsets:
                    print(f"DEBUG: Usando synset inglesi per '{word}' (non trovati in {mapped_lang})")
                    return eng_synsets

            return synsets

        except Exception as e:
            print(f"DEBUG: Errore nel recupero dei synset per '{word}': {str(e)}")
            return []

    def get_entity_semantic_info(self, entity_text: str):
        """
        Estrae informazioni semantiche su un'entità da WordNet.

        Args:
            entity_text: Testo dell'entità

        Returns:
            Dizionario con informazioni semantiche (definizioni, sinonimi, iperonimi, iponimi)
        """
        if not self.is_available:
            return {}

        try:
            # Normalizza il testo dell'entità (minuscolo per la ricerca in WordNet)
            normalized_text = entity_text.lower()

            # Per entità multi-parola, prova diverse strategie
            words = normalized_text.split()

            # Lista di synset da considerare
            all_synsets = []

            # 1. Prova l'entità completa
            synsets = self.get_synsets(normalized_text)
            all_synsets.extend(synsets)

            # 2. Per entità multi-parola, prova anche le singole parole
            if len(words) > 1 and not synsets:
                # Prova l'ultima parola (spesso la più significativa in molte lingue)
                last_word_synsets = self.get_synsets(words[-1])
                if last_word_synsets:
                    all_synsets.extend(last_word_synsets)

                # Prova anche la prima parola se non è un articolo/preposizione
                articles_preps = {"il", "lo", "la", "i", "gli", "le", "un", "uno", "una",
                                  "the", "a", "an", "le", "la", "les", "der", "die", "das"}
                if words[0].lower() not in articles_preps:
                    first_word_synsets = self.get_synsets(words[0])
                    if first_word_synsets:
                        all_synsets.extend(first_word_synsets)

            # Se non abbiamo trovato synset, restituisci un dizionario vuoto
            if not all_synsets:
                return {}

            # Prendi solo i primi N synset per evitare info irrilevanti
            top_synsets = all_synsets[:3]

            # Estrai informazioni semantiche dai synset
            definitions = []
            synonyms = set()
            hypernyms = []
            hyponyms = []

            for synset in top_synsets:
                # Definizione
                definitions.append(synset.definition())

                # Sinonimi (lemmi)
                for lemma in synset.lemmas():
                    synonyms.add(lemma.name().replace('_', ' '))

                # Iperonimi (concetti più generali)
                for hypernym in synset.hypernyms():
                    hypernyms.append({
                        'name': hypernym.name().split('.')[0].replace('_', ' '),
                        'definition': hypernym.definition()
                    })

                # Iponimi (concetti più specifici)
                for hyponym in synset.hyponyms():
                    hyponyms.append({
                        'name': hyponym.name().split('.')[0].replace('_', ' '),
                        'definition': hyponym.definition()
                    })

            # Crea il dizionario di informazioni semantiche
            semantic_info = {
                'definitions': definitions,
                'synonyms': list(synonyms),
                'hypernyms': hypernyms[:5],  # Limita a 5 per evitare troppe info
                'hyponyms': hyponyms[:5]     # Limita a 5 per evitare troppe info
            }

            return semantic_info

        except Exception as e:
            print(f"DEBUG: Errore nell'estrazione di informazioni semantiche per '{entity_text}': {str(e)}")
            return {}

    def enrich_entity(self, entity):
        """
        Arricchisce un'entità con informazioni semantiche da WordNet.

        Args:
            entity: Dizionario dell'entità da arricchire

        Returns:
            Entità arricchita con informazioni semantiche
        """
        if not self.is_available or 'text' not in entity:
            return entity

        entity_text = entity['text']
        semantic_info = self.get_entity_semantic_info(entity_text)

        if semantic_info:
            entity['semantic_info'] = semantic_info

            # Se l'entità non ha tipi, prova a inferirli dagli iperonimi
            if ('types' not in entity or not entity['types']) and semantic_info.get('hypernyms'):
                inferred_types = []
                for hypernym in semantic_info['hypernyms']:
                    hypernym_name = hypernym['name']
                    # Mappa alcuni iperonimi comuni a categorie di entità
                    if hypernym_name in ['person', 'persona', 'human', 'umano']:
                        inferred_types.append('Person')
                    elif hypernym_name in ['location', 'place', 'luogo', 'località']:
                        inferred_types.append('Location')
                    elif hypernym_name in ['organization', 'organizzazione', 'institution', 'istituzione']:
                        inferred_types.append('Organization')
                    elif hypernym_name in ['event', 'evento']:
                        inferred_types.append('Event')
                    elif hypernym_name in ['artifact', 'artefatto', 'product', 'prodotto']:
                        inferred_types.append('Product')

                if inferred_types:
                    entity['types'] = inferred_types
                    entity['types_source'] = 'wordnet_hypernyms'

        return entity

    def enrich_entities(self, entities):
        """
        Arricchisce una lista di entità con informazioni semantiche.

        Args:
            entities: Lista di entità da arricchire

        Returns:
            Lista di entità arricchite
        """
        if not self.is_available:
            return entities

        print(f"DEBUG: Arricchimento semantico di {len(entities)} entità con WordNet")

        enriched_entities = []
        for entity in entities:
            enriched_entity = self.enrich_entity(entity)
            enriched_entities.append(enriched_entity)

        print(f"DEBUG: Arricchimento semantico completato")
        return enriched_entities





# Aggiungi questo codice alla fine dello script o modifica la sezione principale:

# Aggiorna il codice principale per utilizzare le nuove funzionalità

if __name__ == "__main__":
    # Importazioni necessarie
    import os
    import sys
    import traceback

    # Imposta il livello di log per transformers
    os.environ["TRANSFORMERS_VERBOSITY"] = "info"

    # Verifica le dipendenze
    try:
        import torch
        import transformers
        print(f"PyTorch versione: {torch.__version__}")
        print(f"Transformers versione: {transformers.__version__}")
        print(f"CUDA disponibile: {torch.cuda.is_available()}")
    except ImportError as e:
        print(f"ATTENZIONE: Libreria mancante - {e}")

    try:
        # Crea un'istanza del linker di entità migliorato con debug aggiuntivo
        print("Inizializzazione EntityLinkerItalian avanzato...")
        linker = EntityLinkerItalian(language="it", use_gpu=False)

        # Verifica disponibilità dei riconoscitori
        print("\nDisponibilità riconoscitori:")
        print(f"- SpaCy: {linker.wikidata.spacy_recognizer.is_available}")
        print(f"- Italian NLP: {linker.italian_recognizer.is_available}")
        if hasattr(linker, 'transformers_recognizer') and linker.transformers_recognizer:
            print(f"- Transformer NER: {linker.transformers_recognizer.is_available}")
            if linker.transformers_recognizer.is_available:
                print(f"  Modello: {linker.transformers_recognizer.model_name}")
        else:
            print("- Transformer NER: non disponibile")

        if hasattr(linker, 'zero_shot_classifier') and linker.zero_shot_classifier:
            print(f"- Zero-Shot Classifier: {linker.zero_shot_classifier.is_available}")
            if linker.zero_shot_classifier.is_available:
                print(f"  Modello: {linker.zero_shot_classifier.model_name}")
        else:
            print("- Zero-Shot Classifier: non disponibile")

        if hasattr(linker, 'wordnet_kb') and linker.wordnet_kb:
            print(f"- WordNet KB: {linker.wordnet_kb.is_available}")
            if linker.wordnet_kb.is_available:
                print(f"  Lingua: {linker.wordnet_kb.language}")
        else:
            print("- WordNet KB: non disponibile")

        # Testo di esempio
        text = "L'Italia è uno dei paesi più visitati al mondo, con una storia millenaria e un patrimonio culturale unico." \
               " Roma, la capitale, ospita il Colosseo e il Vaticano, mentre Firenze è considerata la culla del Rinascimento." \
               " Milano è un centro globale della moda e dell'economia, e Napoli è famosa per la sua pizza e il Vesuvio." \
               " Il settore turistico è una delle principali fonti di reddito, insieme all'industria manifatturiera e all'export." \
               " Le università italiane, come la Sapienza e il Politecnico di Milano, sono tra le più antiche d'Europa." \
               " Il paese è noto anche per la sua tradizione enogastronomica, con vini rinomati come il Chianti e il Barolo." \
               " La Ferrari e la Lamborghini sono simboli del design automobilistico italiano apprezzati in tutto il mondo." \
               " Il Mediterraneo ha sempre avuto un ruolo strategico nella storia e nell'economia dell'Italia." \
               " Dante Alighieri, Leonardo da Vinci e Galileo Galilei sono tra i più grandi esponenti della cultura italiana." \
               " Negli ultimi anni, l'Italia ha investito molto nella digitalizzazione e nell'innovazione tecnologica."

        print(f"\nAnalisi del testo di esempio...")

        # Trova entità
        print("\nRicerca entità nel testo principale...")
        entities, stats = linker.find_entities(text)

        # Stampa statistiche
        print(f"\nStatistiche riconoscimento entità:")
        for stat_name, stat_value in stats.items():
            print(f"- {stat_name}: {stat_value}")

        # Stampa esempi di entità per tipo di riconoscitore
        print("\nEsempi di entità per tipo di riconoscitore:")

        # Mostra entità Wikidata
        wikidata_entities = [e for e in entities if e.get('source') in ['wikidata', 'wikidata+spacy']]
        if wikidata_entities:
            print(f"\nEntità Wikidata ({len(wikidata_entities)}):")
            for entity in wikidata_entities[:3]:  # Mostra solo le prime 3
                print(f"- '{entity['text']}' → ID: {entity.get('wikidata_id')}")
                if 'types' in entity and entity['types']:
                    print(f"  Tipi: {', '.join(entity['types'][:3])}")

        # Mostra entità Transformer NER
        transformer_entities = [e for e in entities if e.get('source') == 'transformers_ner']
        if transformer_entities:
            print(f"\nEntità Transformer NER ({len(transformer_entities)}):")
            for entity in transformer_entities[:3]:  # Mostra solo le prime 3
                print(f"- '{entity['text']}' → Etichetta: {entity.get('label')}, Score: {entity.get('score', 'N/A')}")

        # Mostra entità con classificazione zero-shot
        zero_shot_entities = [e for e in entities if 'zero_shot_classification' in e]
        if zero_shot_entities:
            print(f"\nEntità con classificazione Zero-Shot ({len(zero_shot_entities)}):")
            for entity in zero_shot_entities[:3]:  # Mostra solo le prime 3
                classification = entity.get('zero_shot_classification', {})
                print(f"- '{entity['text']}' → Classe: {classification.get('label')}, Confidenza: {classification.get('score', 0):.4f}")

        # Mostra entità con info semantiche WordNet
        wordnet_entities = [e for e in entities if 'semantic_info' in e]
        if wordnet_entities:
            print(f"\nEntità con informazioni semantiche WordNet ({len(wordnet_entities)}):")
            for entity in wordnet_entities[:3]:  # Mostra solo le prime 3
                semantic_info = entity.get('semantic_info', {})
                definitions = semantic_info.get('definitions', [])
                print(f"- '{entity['text']}'")
                if definitions:
                    print(f"  Definizione: {definitions[0][:100]}...")
                synonyms = semantic_info.get('synonyms', [])
                if synonyms:
                    print(f"  Sinonimi: {', '.join(synonyms[:3])}")

        # Esporta le entità in un file
        print("\nEsportazione entità in file...")
        try:
            output_file_path = export_entities_to_txt(
                entities,
                stats,
                "entita_complete.txt",
                spacy_recognizer=linker.wikidata.spacy_recognizer,
                italian_recognizer=linker.italian_recognizer,
                transformers_recognizer=linker.transformers_recognizer if hasattr(linker, 'transformers_recognizer') else None,
                zero_shot_classifier=linker.zero_shot_classifier if hasattr(linker, 'zero_shot_classifier') else None,
                wordnet_kb=linker.wordnet_kb if hasattr(linker, 'wordnet_kb') else None,
                original_text=text
            )
            print(f"File salvato in: {output_file_path}")
        except Exception as e:
            print(f"Errore nell'esportazione del file: {e}")
            traceback.print_exc()

        # Verifica il file di output
        import os
        if os.path.exists(output_file_path):
            file_size = os.path.getsize(output_file_path)
            print(f"Verifica file: '{output_file_path}' esiste e ha dimensione {file_size} bytes")
            print("Analisi delle entità completata con successo!")
        else:
            print(f"ATTENZIONE: Il file '{output_file_path}' non è stato creato!")

    except Exception as e:
        print(f"Errore generale: {e}")
        traceback.print_exc()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


PyTorch versione: 2.6.0+cu124
Transformers versione: 4.49.0
CUDA disponibile: False
Inizializzazione EntityLinkerItalian avanzato...
Modello spaCy it_core_news_lg caricato con successo.


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: it (Italian) ...
INFO:stanza:File exists: /root/stanza_resources/it/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| ner       | fbk               |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Riconoscitore NLP italiano inizializzato correttamente con Stanza.
DEBUG: Moduli transformers e torch disponibili. Inizializzazione riconoscitore Transformer NER...
DEBUG: Selezionando modello per lingua: it
DEBUG: Proverò questi modelli nell'ordine: ['dbmdz/bert-base-italian-xxl-cased', 'Babelscape/wikineural-multilingual-ner', 'Davlan/xlm-roberta-base-ner-hrl', 'MilaNLProc/bert-italian-cased-ner', '5had3/bert-base-italian-cased-ner', 'dslim/bert-base-NER']
DEBUG: Verifica disponibilità modello: dbmdz/bert-base-italian-xxl-cased
DEBUG: Modello dbmdz/bert-base-italian-xxl-cased disponibile e selezionato
DEBUG: Tentativo di caricare il modello: dbmdz/bert-base-italian-xxl-cased
DEBUG: Usando device CPU


...

DEBUG: Modello dbmdz/bert-base-italian-xxl-cased caricato con successo!
DEBUG: Test modello superato - entità trovate: 6
DEBUG: Riconoscitore Transformer NER inizializzato con successo
DEBUG: Inizializzazione WordNet knowledge base...
DEBUG: Inizializzazione Zero-Shot Classifier...
DEBUG: Tentativo di caricare il classificatore zero-shot: facebook/bart-large-mnli


Device set to use cpu


DEBUG: Classificatore zero-shot caricato con successo: facebook/bart-large-mnli
DEBUG: Test zero-shot completato con successo
DEBUG: Zero-Shot Classifier inizializzato con successo
DEBUG: Inizializzazione WordNet knowledge base...
DEBUG: NLTK e WordNet importati con successo
DEBUG: WordNet corpus non trovato, tentativo di download...
DEBUG: Open Multilingual WordNet non trovato, tentativo di download...
DEBUG: Test WordNet in italiano per 'casa': 8 synsets trovati
DEBUG: Scaricamento di WordNet...
DEBUG: Scaricamento di Open Multilingual WordNet...
DEBUG: WordNet inizializzato con successo. Lingua: ita
DEBUG: Esempio per 'casa': 8 synsets trovati
DEBUG: WordNet KB disponibilità: True

Disponibilità riconoscitori:
- SpaCy: True
- Italian NLP: True
- Transformer NER: True
  Modello: dbmdz/bert-base-italian-xxl-cased
- Zero-Shot Classifier: True
  Modello: facebook/bart-large-mnli
- WordNet KB: True
  Lingua: it

Analisi del testo di esempio...

Ricerca entità nel testo principale...
DEBU

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


spaCy ha trovato 22 entità nel testo
Di cui 0 sono uniche (non trovate da Wikidata o dal dizionario locale)
Riconoscitore italiano ha trovato 27 entità nel testo
Entità unica dal riconoscitore italiano: Dante (PROPN)
Di cui 1 sono uniche (non trovate da altre fonti)
DEBUG: Tentativo di trovare entità Transformer NER
DEBUG: Modello Transformer NER in uso: dbmdz/bert-base-italian-xxl-cased
DEBUG: Analisi testo con modello dbmdz/bert-base-italian-xxl-cased
DEBUG: Testo diviso in 10 frasi
DEBUG: Analisi frase 1/10
DEBUG: Mismatch - tokenizer ha '##ti' ma il testo ha 'ti'
DEBUG: Mismatch - tokenizer ha '##nar' ma il testo ha 'nar'
DEBUG: Mismatch - tokenizer ha '##ia' ma il testo ha 'ia'
DEBUG: Analisi frase 2/10
DEBUG: Mismatch - tokenizer ha '##sse' ma il testo ha 'sse'
DEBUG: Mismatch - tokenizer ha '##o' ma il testo ha 'o'
DEBUG: Analisi frase 3/10
DEBUG: Mismatch - tokenizer ha '##u' ma il testo ha 'u'
DEBUG: Mismatch - tokenizer ha '##vio' ma il testo ha 'vio'
DEBUG: Analisi frase 4/1